this notebook creates address label files

In [ ]:
import copy
import os
import typing
import ctc

import cryo
import numpy as np
import polars as pl
import polars_evm
import matplotlib.pyplot as plt
import toolplot
import tooltime
import toolstr
import IPython
import state_growth
import shutil
import glob
import polars_evm
import cryo

import ctc.rpc
import asyncio
import requests

import state_growth.datasets.io_per_contract.agg_by_contract
from state_growth.datasets import io_per_contract
from state_growth.datasets import address_labels

In [ ]:
toolplot.setup_plot_formatting()
polars_evm.set_column_display_width()

In [ ]:
context = {
    'data_root': '/home/storm/data',
    'network': 'ethereum',
}

In [ ]:
label_root = '/home/storm/data/ethereum_state_growth/address_labels'

def get_label_path(label_type, min_block, max_block):
    filename = "ethereum__{label_type}__{min_block:08}_to_{max_block:08}.parquet".format(
        label_type=label_type,
        min_block=min_block,
        max_block=max_block,
    )
    
    return os.path.join(label_root, filename)

# extract labels

In [ ]:
min_block = 0
max_block = 19_340_000

In [ ]:
for datatype, extractor in [
    ['erc20s', address_labels.extract_erc20s],
    ['erc721s', address_labels.extract_erc721s],
    ['erc1155s', address_labels.extract_erc1155s],
    ['chainlink_feeds', address_labels.extract_chainlink_feeds],
]:
    address_labels.extract_labels(
        datatype=datatype,
        min_block=min_block,
        max_block=max_block,
        f=extractor,
        **context
    )

# load labels

In [ ]:
labels = address_labels.load_address_labels(
    ['erc20s', 'erc721s', 'erc1155s', 'chainlink_feeds', 'manual'],
    **context
)

labels

In [ ]:
labels = address_labels.load_address_labels(
    ['erc20s', 'erc721s', 'erc1155s', 'chainlink_feeds', 'manual'],
    **context
)

labels

# erc20 metadata

In [ ]:
erc20s = address_labels.load_address_labels('erc20s', **context)
erc20s = erc20s.evm.binary_to_hex()

erc20s

In [ ]:
erc20_metadata = cryo.collect('erc20_metadata', contract=erc20s['address'])

erc20_metadata

In [ ]:
path = get_label_path('erc20s', min_block, max_block)

erc20_metadata.drop("block_number").rename({"erc20": "address"}).with_columns(
    label=pl.lit("erc20")
).write_parquet(path)

# erc721 metadata

In [ ]:
erc721s = address_labels.load_address_labels('erc721s', **context)
erc721s = erc721s.evm.binary_to_hex()

erc721s

In [ ]:
erc721_metadata = cryo.collect('erc20_metadata', contract=erc721s['address'])

erc721_metadata

In [ ]:
path = get_label_path('erc721s', min_block, max_block)

erc721_metadata.drop("block_number", "decimals").rename({"erc20": "address"}).with_columns(
    label=pl.lit("erc721")
).write_parquet(path)

In [ ]:
path = get_label_path('erc721s', min_block, max_block)

erc721_metadata = pl.read_parquet(path)

erc721_metadata

In [ ]:
path = get_label_path('erc721s', min_block, max_block)

erc721_metadata.with_columns(
    name=pl.when(
        pl.col.address
        == bytes.fromhex("0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85"[2:])
    )
    .then(pl.lit("ENS"))
    .otherwise(pl.col.name),
    symbol=pl.when(
        pl.col.address
        == bytes.fromhex("0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85"[2:])
    )
    .then(pl.lit("ENS"))
    .otherwise(pl.col.symbol),
).write_parquet(path)

# erc1155

In [ ]:
erc1155s = address_labels.load_address_labels('erc1155s', **context)
erc1155s = erc1155s.evm.binary_to_hex()

erc1155s

In [ ]:
erc1155_metadata = cryo.collect('erc20_metadata', contract=erc1155s['address'])

erc1155_metadata

In [ ]:
path = get_label_path('erc1155s', min_block, max_block)

erc1155_metadata.drop("block_number", "decimals").rename({"erc20": "address"}).with_columns(
    label=pl.lit("erc1155")
).write_parquet(path)

# uniswap pools

In [ ]:
erc20s = address_labels.load_address_labels('erc20s', **context)

erc20s

In [ ]:
uniswap_v2_pools = pl.read_parquet('/home/storm/data/ethereum/uniswap_v2_pools/*.parquet')

uniswap_v2_pools

In [ ]:
uniswap_v2_pools = uniswap_v2_pools.join(
    erc20s.drop('chain_id', 'label').rename(
        {
            'name': 'token0_name',
            'symbol': 'token0_symbol',
            'decimals': 'token0_decimals',
        }
    ),
    left_on="event__token0",
    right_on='address',
    how="left",
).join(
    erc20s.drop('chain_id', 'label').rename(
        {
            'name': 'token1_name',
            'symbol': 'token1_symbol',
            'decimals': 'token1_decimals',
        }
    ),
    left_on="event__token1",
    right_on='address',
    how="left",
).select(
    address='event__pair',
    label=pl.lit('uniswap_v2_pool'),
    token0_address='event__token0',
    token0_name='token0_name',
    token0_symbol='token0_symbol',
    token0_decimals='token0_decimals',
    token1_address='event__token1',
    token1_name='token1_name',
    token1_symbol='token1_symbol',
    token1_decimals='token1_decimals',
)

In [ ]:
uniswap_v2_pools.write_parquet(
    get_label_path('uniswap_v2_pools', min_block, max_block)
)

In [ ]:
uniswap_v3_pools = pl.read_parquet('/home/storm/data/ethereum/uniswap_v3_pools/*.parquet')

uniswap_v3_pools

In [ ]:
uniswap_v3_pools = uniswap_v3_pools.join(
    erc20s.drop('chain_id', 'label').rename(
        {
            'name': 'token0_name',
            'symbol': 'token0_symbol',
            'decimals': 'token0_decimals',
        }
    ),
    left_on="event__token0",
    right_on='address',
    how="left",
).join(
    erc20s.drop('chain_id', 'label').rename(
        {
            'name': 'token1_name',
            'symbol': 'token1_symbol',
            'decimals': 'token1_decimals',
        }
    ),
    left_on="event__token1",
    right_on='address',
    how="left",
).select(
    address='event__pool',
    label=pl.lit('uniswap_v3_pool'),
    token0_address='event__token0',
    token0_name='token0_name',
    token0_symbol='token0_symbol',
    token0_decimals='token0_decimals',
    token1_address='event__token1',
    token1_name='token1_name',
    token1_symbol='token1_symbol',
    token1_decimals='token1_decimals',
    fee='event__fee',
)

In [ ]:
uniswap_v3_pools.write_parquet(
    get_label_path('uniswap_v3_pools', min_block, max_block)
)

# chainlink feeds

In [ ]:
chainlink_feeds = address_labels.load_address_labels('chainlink_feeds', **context)
chainlink_feeds = chainlink_feeds.evm.binary_to_hex()

chainlink_feeds

In [ ]:
# add names of each feed

description_abi = await ctc.async_get_function_abi(
    contract_address='0x8d0003e5c1c8eb67e04023a21291cf01cfd2e4a1',
    function_name='description',
)

tasks = [
    ctc.rpc.async_eth_call(
        to_address=address,
        function_abi=description_abi,
        convert_reverts_to_none=True,
        decode_response=False,
    )
    for address in chainlink_feeds['address']
]

raw_descriptions = await asyncio.gather(*tasks)
descriptions = []
for description in raw_descriptions:
    if description is not None:
        description = ctc.binary_to_text(description).strip('\'\x06\x07&\x1a\x11\x0f\x0e#+\x1c\x16\x1b\x0c\x08\x1d\x00 \x15\t\n\r" !$()').strip('"')
        if description == '':
            description = None
        descriptions.append(description)
    else:
        descriptions.append(None)

chainlink_feeds = chainlink_feeds.with_columns(pl.Series('name', descriptions))

chainlink_feeds

In [ ]:
# from chainlink's website

response = requests.get('https://cl-docs-addresses.web.app/addresses.json')
raw_chainlink_data = response.json()
for item in raw_chainlink_data['ethereum']['networks']:
    if item['name'] == 'Ethereum Mainnet' and item['dataType'] == 'default':
        break
else:
    raise Exception('could not find')

chainlink_df = pl.DataFrame(item['proxies'])
chainlink_df = chainlink_df.with_columns(chainlink_df['proxy'].str.to_lowercase())

aggregator_abi = {
    "inputs": [],
    "name": "aggregator",
    "outputs": [{"internalType": "address", "name": "", "type": "address"}],
    "stateMutability": "view",
    "type": "function",
}
tasks = [
    ctc.rpc.async_eth_call(address, function_abi=aggregator_abi)
    for address in chainlink_df['proxy']
]
aggregators = await asyncio.gather(*tasks)
chainlink_df = chainlink_df.with_columns(pl.Series('aggregator', aggregators))

chainlink_df

In [ ]:
chainlink_feeds.join(
    chainlink_df[['pair', 'aggregator']],
    left_on='address',
    right_on='aggregator',
    how='left',
).filter(
    (pl.col.name != pl.col.pair) & (~pl.col.pair.is_null())
)

In [ ]:
chainlink_df.filter(~pl.col.aggregator.is_in(chainlink_feeds["address"])).filter(
    pl.col.aggregator != "0x0000000000000000000000000000000000000000"
)

In [ ]:
ls /home/storm/data/ethereum_state_growth/address_labels/

In [ ]:
chainlink_feeds = chainlink_feeds.with_columns(
    chainlink_feeds['address'].evm.hex_to_binary()
)

In [ ]:
chainlink_feeds.write_parquet(
    
    get_label_path('chainlink_feeds', min_block, max_block)
#     "/home/storm/data/ethereum_state_growth/address_labels/ethereum__chainlink_feeds__00000000_to_18999999.parquet"
)

# gnosis safes

In [ ]:
gnosis_factory__1_1_1 = "0x76e2cfc1f5fa8f6a5b3fc4c8f4788f0116861f9b"
gnosis_factory__1_3_0 = '0xa6b71e26c5e0845f74c812102ca7114b6a896ab2'
gnosis_factory__1_4_1 = '0x4e1DCf7AD4e460CfD30791CCC4F9c8a4f820ec67'

gnosis_factory__1_1_1__bytes = bytes.fromhex(gnosis_factory__1_1_1[2:])
gnosis_factory__1_3_0__bytes = bytes.fromhex(gnosis_factory__1_3_0[2:])
gnosis_factory__1_4_1__bytes = bytes.fromhex(gnosis_factory__1_4_1[2:])

proxy_creation_abi__1_1_1 = await ctc.async_get_event_abi(
    contract_address=gnosis_factory__1_1_1,
    event_abi='ProxyCreation',
)
proxy_creation_abi__1_3_0 = await ctc.async_get_event_abi(
    contract_address=gnosis_factory__1_3_0,
    event_abi='ProxyCreation',
)
proxy_creation_abi__1_4_1 = await ctc.async_get_event_abi(
    contract_address=gnosis_factory__1_4_1,
    event_abi='ProxyCreation',
)

In [ ]:
ctc.get_event_hash(proxy_creation_abi__1_1_1) == ctc.get_event_hash(proxy_creation_abi__1_3_0)

In [ ]:
ctc.get_event_hash(proxy_creation_abi__1_1_1) == ctc.get_event_hash(proxy_creation_abi__1_4_1)

In [ ]:
logs = (
    state_growth.scan_raw_dataset(datatype="logs", **context)
    .filter(pl.col.address == bytes.fromhex(gnosis_factory__1_4_1[2:]))
    .collect()
)

logs

In [ ]:
logs['topic0'].value_counts(sort=True)

In [ ]:
safes__1_1_1 = (
    state_growth.scan_raw_dataset(datatype="logs", **context)
    .filter(
        (pl.col.address == bytes.fromhex(gnosis_factory__1_1_1[2:]))
        & (pl.col.topic0 == bytes.fromhex('0xa38789425dbeee0239e16ff2d2567e31720127fbc6430758c1a4efc6aef29f80'[2:]))
    )
    .collect()
)

safes__1_1_1

In [ ]:
safes__1_3_0 = (
    state_growth.scan_raw_dataset(datatype="logs", **context)
    .filter(
        (pl.col.address == bytes.fromhex(gnosis_factory__1_3_0[2:]))
        & (pl.col.topic0 == bytes.fromhex('0x4f51faf6c4561ff95f067657e43439f0f856d97c04d9ec9070a6199ad418e235'[2:]))
    )
    .collect()
)

safes__1_3_0

In [ ]:
safe_addresses_1_1_1 = pl.DataFrame({
    'address': '0x' + safes__1_1_1['data'].evm.binary_to_hex().str.slice(26),
    'safe_version': '1.1.1',
})

safe_addresses_1_3_0 = pl.DataFrame({
    'address': '0x' + safes__1_3_0['data'].evm.binary_to_hex().str.slice(26, 40),
    'safe_version': '1.3.0',
})

safe_addresses_1_4_1 = pl.DataFrame({
    'address': '0x' + logs['topic1'].evm.binary_to_hex().str.slice(26),
    'safe_version': '1.4.1',
})




safe_addresses = pl.concat(
    [
        safe_addresses_1_1_1,
        safe_addresses_1_3_0,
        safe_addresses_1_4_1,
    ]
).with_columns(
    label=pl.lit('gnosis_safe'),
)

safe_addresses = safe_addresses.with_columns(
    safe_addresses['address'].evm.hex_to_binary()
)


safe_addresses.write_parquet(
    get_label_path('gnosis_safes', min_block, max_block)
)

# curve

In [ ]:
from ctc.protocols import curve_utils

In [ ]:
curve_contracts = [
    ['3Pool', '0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7'],
    ['3Pool', '0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490'],
    ['AAVE', '0xFd2a8fA60Abd58Efe3EeE34dd494cD491dC14900'],
    ['AAVE', '0xDeBF20617708857ebe4F679508E7b7863a8A8EeE'],
    ['ankrETH', '0xA96A65c051bF88B4095Ee1f2451C2A9d43F53Ae2'],
    ['ankrETH', '0xaA17A236F2bAdc98DDc0Cf999AbB47D47Fc0A6Cf'],
    ['BUSD', '0x79a8C46DeA5aDa233ABaFFD40F3A0A2B1e5A4F27'],
    ['BUSD', '0xb6c057591E073249F2D9D88Ba59a46CFC9B59EdB'],
    ['BUSD', '0x3B3Ac5386837Dc563660FB6a0937DFAa5924333B'],
    ['Compound', '0xA2B47E3D5c44877cca798226B7B8118F9BFb7A56'],
    ['Compound', '0xeB21209ae4C2c9FF2a86ACA31E123764A3B6Bc06'],
    ['Compound', '0x845838DF265Dcd2c412A1Dc9e959c7d08537f8a2'],
    ['EURS', '0x0Ce6a5fF5217e38315f87032CF90686C96627CAA'],
    ['EURS', '0x194eBd173F6cDacE046C53eACcE9B953F28411d1'],
    ['hBTC', '0x4CA9b3063Ec5866A4B82E437059D2C43d1be596F'],
    ['hBTC', '0xb19059ebb43466C323583928285a49f558E572Fd'],
    ['IronBank', '0x2dded6Da1BF5DBdF597C45fcFaa3194e53EcfeAF'],
    ['IronBank', '0x5282a4eF67D9C33135340fB3289cc1711c13638C'],
    ['Link', '0xF178C0b5Bb7e7aBF4e12A4838C7b7c5bA2C623c0'],
    ['Link', '0xcee60cfa923170e4f8204ae08b4fa6a3f5656f3a'],
    ['PAX', '0xA50cCc70b6a011CffDdf45057E39679379187287'],
    ['PAX', '0x06364f10B501e868329afBc005b3492902d6C763'],
    ['PAX', '0xD905e2eaeBe188fc92179b6350807D8bd91Db0D8'],
    ['renBTC', '0x93054188d876f558f4a66B2EF1d97d16eDf0895B'],
    ['renBTC', '0x49849C98ae39Fff122806C06791Fa73784FB3675'],
    ['rETH', '0xF9440930043eb3997fc70e1339dBb11F341de7A8'],
    ['rETH', '0x53a901d48795C58f485cBB38df08FA96a24669D5'],
    ['sAAVE', '0xEB16Ae0052ed37f479f7fe63849198Df1765a733'],
    ['sAAVE', '0x02d341CcB60fAaf662bC0554d13778015d1b285C'],
    ['sBTC', '0x7fC77b5c7614E1533320Ea6DDc2Eb61fa00A9714'],
    ['sBTC', '0x075b1bb99792c9E1041bA13afEf80C91a1e70fB3'],
    ['sETH', '0xc5424B857f758E906013F3555Dad202e4bdB4567'],
    ['sETH', '0xA3D87FffcE63B53E0d54fAa1cc983B7eB0b74A9c'],
    ['stETH', '0xDC24316b9AE028F1497c275EB9192a3Ea0f67022'],
    ['stETH', '0x06325440D014e39736583c165C2963BA99fAf14E'],
    ['sUSD', '0xFCBa3E75865d2d561BE8D220616520c171F12851'],
    ['sUSD', '0xA5407eAE9Ba41422680e2e00537571bcC53efBfD'],
    ['sUSD', '0xC25a3A3b969415c80451098fa907EC722572917F'],
    ['TriCrypto', '0x80466c64868E1ab14a1Ddf27A676C3fcBE638Fe5'],
    ['TriCrypto', '0x331aF2E331bd619DefAa5DAc6c038f53FCF9F785'],
    ['TriCrypto', '0xcA3d75aC011BF5aD07a98d02f18225F9bD9A6BDF'],
    ['USDT', '0xac795D2c97e60DF6a99ff1c814727302fD747a80'],
    ['USDT', '0x52EA46506B9CC5Ef470C5bf89f17Dc28bB35D85C'],
    ['USDT', '0x9fC689CCaDa600B6DF723D9E47D84d76664a1F23'],
    ['Y', '0xbBC81d23Ea2c3ec7e56D39296F0cbB648873a5d3'],
    ['Y', '0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51'],
    ['Y', '0xdF5e0e81Dff6FAF3A7e52BA697820c5e32D806A8'],
    ['Yv2', '0x8925D9d9B4569D737a48499DeF3f67BaA5a144b9'],
    ['Yv2', '0x571FF5b7b346F706aa48d696a9a4a288e9Bb4091'],
    ['bBTC', '0x071c661B4DeefB59E2a3DdB20Db036821eeE8F4b'],
    ['bBTC', '0xC45b2EEe6e09cA176Ca3bB5f7eEe7C47bF93c756'],
    ['bBTC', '0x410e3E86ef427e30B9235497143881f717d93c2A'],
    ['DUSD', '0x61E10659fe3aa93d036d099405224E4Ac24996d0'],
    ['DUSD', '0x8038C01A0390a8c547446a0b2c18fc9aEFEcc10c'],
    ['DUSD', '0x3a664Ab939FD8482048609f652f9a0B0677337B9'],
    ['GUSD', '0x4f062658EaAF2C1ccf8C8e36D6824CDf41167956'],
    ['GUSD', '0x64448B78561690B70E17CBE8029a3e5c1bB7136e'],
    ['GUSD', '0xD2967f45c4f384DEEa880F807Be904762a3DeA07'],
    ['HUSD', '0x09672362833d8f703D5395ef3252D4Bfa51c15ca'],
    ['HUSD', '0x3eF6A01A0f81D6046290f3e2A8c5b843e738E604'],
    ['HUSD', '0x5B5CFE992AdAC0C9D48E05854B2d91C73a003858'],
    ['LinkUSD', '0x1de7f0866e2c4adAC7b457c58Cc25c8688CDa1f2'],
    ['LinkUSD', '0xE7a24EF0C5e95Ffb0f6684b813A78F2a3AD7D171'],
    ['LinkUSD', '0x6D65b498cb23deAba52db31c93Da9BFFb340FB8F'],
    ['MUSD', '0x803A2B40c5a9BB2B86DD630B274Fa2A9202874C2'],
    ['MUSD', '0x8474DdbE98F5aA3179B3B3F5942D724aFcdec9f6'],
    ['MUSD', '0x1AEf73d49Dedc4b1778d0706583995958Dc862e6'],
    ['oBTC', '0xd5BCf53e2C81e1991570f33Fa881c49EEa570C8D'],
    ['oBTC', '0xd81dA8D904b52208541Bade1bD6595D8a251F8dd'],
    ['oBTC', '0x2fE94ea3d5d4a175184081439753DE15AeF9d614'],
    ['pBTC', '0x11F419AdAbbFF8d595E7d5b223eee3863Bb3902C'],
    ['pBTC', '0x7F55DDe206dbAD629C080068923b36fe9D6bDBeF'],
    ['pBTC', '0xDE5331AC4B3630f94853Ff322B66407e0D6331E8'],
    ['RSV', '0xBE175115BF33E12348ff77CcfEE4726866A0Fbd5'],
    ['RSV', '0xC18cC39da8b11dA8c3541C598eE022258F9744da'],
    ['RSV', '0xC2Ee6b0334C261ED60C72f6054450b61B8f18E35'],
    ['tBTC', '0xaa82ca713D94bBA7A89CEAB55314F9EfFEdDc78c'],
    ['tBTC', '0xC25099792E9349C7DD09759744ea681C7de2cb66'],
    ['tBTC', '0x64eda51d3Ad40D56b9dFc5554E06F94e1Dd786Fd'],
    ['USDK', '0xF1f85a74AD6c64315F85af52d3d46bF715236ADc'],
    ['USDK', '0x3E01dD8a5E1fb3481F0F589056b428Fc308AF0Fb'],
    ['USDK', '0x97E2768e8E73511cA874545DC5Ff8067eB19B787'],
    ['USDN', '0x094d12e5b541784701FD8d65F11fc0598FBC6332'],
    ['USDN', '0x0f9cb53Ebe405d49A0bbdBD291A65Ff571bC83e1'],
    ['USDN', '0x4f3E8F405CF5aFC05D68142F3783bDfE13811522'],
    ['USDP', '0x3c8cAee4E09296800f8D29A68Fa3837e2dae4940'],
    ['USDP', '0x42d7025938bEc20B69cBae5A77421082407f053A'],
    ['USDP', '0x7Eb40E450b9655f4B3cC4259BCC731c63ff55ae6'],
    ['UST', '0xB0a0716841F2Fc03fbA72A891B8Bb13584F52F2d'],
    ['UST', '0x890f4e345B1dAED0367A877a1612f86A1f86985f'],
    ['UST', '0x94e131324b6054c0D789b190b2dAC504e4361b53'],
    ['3pool', '0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A'],
    ['AAVE', '0xd662908ADA2Ea1916B3318327A97eB18aD588b5d'],
    ['alUSD', '0x9582C4ADACB3BCE56Fea3e590F05c3ca2fb9C477'],
    ['ankrETH', '0x6d10ed2cF043E6fcf51A0e7b4C2Af3Fa06695707'],
    ['bBTC', '0xdFc7AdFa664b08767b735dE28f9E84cd30492aeE'],
    ['BUSD', '0x69Fb7c45726cfE2baDeE8317005d3F94bE838840'],
    ['Compound', '0x7ca5b0a2910B33e9759DC7dDB0413949071D7575'],
    ['DUSD', '0xAEA6c312f4b3E04D752946d329693F7293bC2e6D'],
    ['EURS', '0x90Bb609649E0451E5aD952683D64BD2d1f245840'],
    ['FRAX', '0x72e158d38dbd50a483501c24f792bdaaa3e7d55c'],
    ['GUSD', '0xC5cfaDA84E902aD92DD40194f0883ad49639b023'],
    ['hBTC', '0x4c18E409Dc8619bFb6a1cB56D114C3f592E0aE79'],
    ['HUSD', '0x2db0E83599a91b508Ac268a6197b8B14F5e72840'],
    ['MUSD', '0x5f626c30EC1215f4EdCc9982265E8b1F411D1352'],
    ['oBTC', '0x11137B10C210b579405c21A07489e28F3c040AB1'],
    ['PAX', '0x64E3C23bfc40722d3B649844055F1D51c1ac041d'],
    ['IronBank', '0xF5194c3325202F456c95c1Cf0cA36f8475C1949F'],
    ['Link', '0xFD4D8a17df4C27c1dD245d153ccf4499e806C87D'],
    ['pBTC', '0xd7d147c6Bb90A718c3De8C0568F9B560C79fa416'],
    ['renBTC', '0xB1F2cdeC61db658F091671F5f199635aEF202CAC'],
    ['RSV', '0x4dC4A289a8E33600D8bD4cf5F6313E43a37adec7'],
    ['sAAVE', '0x462253b8F74B72304c145DB0e4Eebd326B22ca39'],
    ['sBTC', '0x705350c4BcD35c9441419DdD5d2f097d7a55410F'],
    ['sETH', '0x3C0FFFF15EA30C35d7A85B85c0782D6c94e1d238'],
    ['stETH', '0x182B723a58739a9c974cFDB385ceaDb237453c28'],
    ['sUSDv2', '0xA90996896660DEcC6E997655E065b23788857849'],
    ['rETH', '0x824F13f1a2F29cFEEa81154b46C0fc820677A637'],
    ['tBTC', '0x6828bcF74279eE32f2723eC536c22c51Eed383C6'],
    ['TriCrypto', '0x6955a55416a06839309018A8B0cB72c4DDC11f15'],
    ['USDK', '0xC2b1DF84112619D190193E48148000e3990Bf627'],
    ['USDN', '0xF98450B5602fa59CC66e1379DFfB6FDDc724CfC4'],
    ['USDP', '0x055be5DDB7A925BfEF3417FC157f53CA77cA7222'],
    ['USDT', '0xBC89cd85491d81C6AD2954E6d0362Ee29fCa8F53'],
    ['UST', '0x3B7020743Bc2A4ca9EaF9D0722d42E20d6935855'],
    ['Y', '0xFA712EE4788C042e2B7BB55E6cb8ec569C4530c1'],
    ['Yv2', '0x8101E6760130be2C8Ace79643AB73500571b7162'],
    ['CRV Token', '0xD533a949740bb3306d119CC777fa900bA034cd52'],
    ['Fee Distributor', '0xA464e6DCda8AC41e03616F95f4BC98a13b8922Dc'],
    ['Gauge Controller', '0x2F50D538606Fa9EDD2B11E2446BEb18C9D5846bB'],
    ['Minter', '0xd061D61a4d941c39E5453435B6345Dc261C2fcE0'],
    ['Voting Escrow', '0x5f3b5DfEb7B28CDbD7FAba78963EE202a494e2A2'],
    ['Vesting Escrow', '0x575ccd8e2d300e2377b43478339e364000318e2c'],
    ['Gauge Owner', '0x519AFB566c05E00cfB9af73496D00217A630e4D5'],
    ['Pool Owner', '0xeCb456EA5365865EbAb8a2661B0c503410e9B347'],
    ['Crypto Pool Owner', '0x3687367CcAEBBE89f1bc8Eae7592b4Eed44Ac0Bd'],
    ['Factory Pool Owner', '0x8cf8af108b3b46ddc6ad596aebb917e053f0d72b'],
    ['Ownership', '0xE478de485ad2fe566d49342Cbd03E49ed7DB3356'],
    ['Parameter', '0xBCfF8B0b9419b9A88c44546519b1e909cF330399'],
    ['Emergency', '0x1115c9b3168563354137cDc60efb66552dd50678'],
    ['Ownership', '0x40907540d8a6c65c637785e8f8b742ae6b0b9968'],
    ['Parameter', '0x4eeb3ba4f221ca16ed4a0cc7254e2e32df948c5f'],
    ['Emergency', '0x00669DF67E4827FCc0E48A1838a8d5AB79281909'],
    ['Ownership / Parameter', '0x5f3b5DfEb7B28CDbD7FAba78963EE202a494e2A2'],
    ['Emergency', '0x4c0947B16FB1f755A2D32EC21A0c4181f711C500'],
    ['ABurner', '0x12220a63a2013133d54558c9d03c35288eac9b34'],
    ['CryptoLPBurner', '0x0B5B9210d5015fD0c97FB19B32675b19703b0453'],
    ['CBurner', '0xdd0e10857d952c73b2fa39ce86308299df8774b8'],
    ['LPBurner', '0xaa42C0CD9645A58dfeB699cCAeFBD30f19B1ff81'],
    ['MetaBurner', '0xE4b65889469ad896e866331f0AB5652C1EcfB3E6'],
    ['SynthBurner', '0x67a0213310202DBc2cbE788f4349B72fbA90f9Fa'],
    ['USDNBurner', '0x06534b0BF7Ff378F162d4F348390BDA53b15fA35'],
    ['UnderlyingBurner', '0x786b374b5eef874279f4b7b4de16940e57301a58'],
    ['UniswapBurner', '0xf3b64840b39121b40d8685f1576b64c157ce2e24'],
    ['YBurner', '0xd16ea3e5681234da84419512eb597362135cd8c9'],
    ['Address Provider', '0x0000000022d53366457f9d5e68ec105046fc4383'],
    ['Curve Calculator', '0xc1DB00a8E5Ef7bfa476395cdbcc98235477cDE4E'],
    ['Pool Info', '0xe64608E223433E8a03a1DaaeFD8Cb638C14B552C'],
    ['Registry', '0x90E00ACe148ca3b23Ac1bC8C240C2a7Dd9c2d7f5'],
    ['Factory', '0xB9fC157394Af804a3578134A6585C0dc9cc990d4'],
    ['Migrator', '0xd6930b7f661257DA36F93160149b031735237594'],
    ['3pool', '0x5F890841f657d90E081bAbdB532A05996Af79Fe6'],
    ['sBTC', '0x2f956eee002b0debd468cf2e0490d1aec65e027f'],
    ['3pool Deposit Zap', '0xA79828DF1850E8a3A3064576f380D90aECDD3359'],
    ['sBTC Deposit Zap', '0x7AbDBAf29929e7F8621B757D2a7c04d78d633834'],
    ['alUSD', '0x43b4FdFD4Ff969587185cDB6f0BD875c5Fc83f8c'],
    ['FRAX', '0xd632f22692FaC7611d2AA1C0D552930D43CAEd3B'],
]

labels, addresses = zip(*curve_contracts)
addresses = [address.lower() for address in addresses]

In [ ]:
curve_labels = pl.DataFrame([addresses, labels], schema=['address', 'label'])

curve_labels

In [ ]:
factories = [
    '0xb9fc157394af804a3578134a6585c0dc9cc990d4',
    '0x90e00ace148ca3b23ac1bc8c240c2a7dd9c2d7f5',
    '0x0959158b6040d32d04c301a72cbfd6b39e21c9ae',
    '0x8f942c20d02befc377d41445793068908e2250d0',
    '0xf18056bbd320e96a48e3fbf8bc061322531aac99',
]

# get data from each factory
coroutines = [
    curve_utils.async_get_factory_pool_data(factory, include_balances=False)
    for factory in factories
]
factories_data = await asyncio.gather(*coroutines)
items = [item for factory_data in factories_data for item in factory_data]
curve_pools = pl.DataFrame(items)

curve_pools

In [ ]:
curve_addresses = pl.concat(
    [
        curve_labels,
        curve_pools.select(
            address="address", label=curve_pools["symbols"].list.join(" / ")
        ),
    ]
).group_by("address").agg(pl.last("label")).select(
    address='address',
    label=pl.lit('curve'),
    name='label',
)

curve_addresses = curve_addresses.with_columns(
    curve_addresses['address'].evm.hex_to_binary()
)

curve_addresses.write_parquet(
    get_label_path('curve', min_block, max_block)
)

# yearn

In [ ]:
from ctc.protocols import yearn_utils

In [ ]:
api_vaults = await yearn_utils.async_get_yearn_api_vaults(network='ethereum')
label, address = zip(*[[vault['display_name'], vault['address']] for vault in api_vaults])
yearn_vaults = pl.DataFrame({'name': label, 'address': address})

yearn_vaults

In [ ]:
yearn_aux = yearn_utils.get_yearn_addresses(network='ethereum')
yearn_meta = pl.DataFrame(list(yearn_aux.items()), orient='row', schema=['name', 'address'])

yearn_meta

In [ ]:
# https://docs.yearn.finance/getting-started/products/addresses
yearn_official = {
    "0x50c1a2eA0a861A967D9d0FFE2AE4012c2E053804": "V2 Registry",
#     "0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52": "Yearn multisig",  # a gnosis safe
#     "0x16388463d60FFE0661Cf7F1f31a7D658aC790ff7": "Strategist multisig",  # a gnosis safe
#     "0x846e211e8ba920B353FB717631C015cf04061Cc9": "Core Dev multisig",  # a gnosis safe
    "0x93A62dA5a14C80f265DAbC077fCEE437B1a0Efde": "Treasury",
    "0xDDCea799fF1699e98EDF118e0629A974Df7DF012": "Health Check",
    "0x83d95e0D5f402511dB06817Aff3f9eA88224B030": "Oracle",
    "0xf64e58Ee8C7BadC741A7ea98FB65488084385674": "ManagementList",
    "0x240315db938d44bb124ae619f5Fd0269A02d1271": "V2",
    "0xFF0bd2d0C7E9424ccB149ED3757155eEf41a793D": "Iron Bank",
    "0x14d6E0908baE40A2487352B2a9Cb1A6232DA8785": "V2",
    "0xF4fB8903A41fC78686b26DE55502cdE42a4c6c78": "V1",
    "0xec7Ac8AC897f5082B2c3d4e8D2173F992A097F24": "Iron Bank",
    "0x1007eD6fdFAC72bbea9c719cf1Fa9C355D248691": "Earn",
    "0x560144C25E53149aC410E5D33BDB131e49A850e5": "veCRV",
    "0x5AACD0D03096039aC4381CD814637e9FB7C34a6f": "Helper",
    "0xC3A0bEF4A47bA579Cbba510aE2c59d9b9bf9467C": "UniqueAddressesHelper",
    "0xae813841436fe29b95a14ac701afb1502c4cb789": "Strategies helper",
    "0x4218E20DB87023049Fc582Aaa4bD47a3611A20Ab": "AllowancesHelper",
    "0x957e3ae7983155a9f9e08da555b8084448be26e4": "AddressMergeHelper",
    "0x5d63a8584d91ebc5033d022afd6c5a7c7fddc99b": "PricesHelper",
    "0x855ffe28019106d089bC018Df18838f8d241c402": "BalancesHelper",
    "0xce29d34C8e88A2E1eDde10AD4eEE4f3e379fc041": "AddressesGeneratorV1Vaults",
    "0x437758D475F70249e03EDa6bE23684aD1FC375F0": "AddressesGeneratorV2Vaults",
    "0xF536399c04683D7ab0BCb1308C245b4bbb539344": "AddressesGeneratorEarn",
    "0xa0B57619A980DFEfD50f24F310EE1b55A40A9D46": "AddressesGeneratorIronBank",
    "0xC01a529c01f9399c928a4aFc50E25f12D1e5b142": "DelegatedBalanceMapping",
    "0xE9cfF16bdac9729F18CBAC451EcCA6c50b372207": "CalculationsCurve",
    "0x55e9B18fefFF7E00548d54480373Fc8843De8eA4": "CalculationsIronBank",
    "0x88dE7d7F7b9597C86b8cD195374FbF602934F334": "CalculationsSushiswap",
    "0x38477F2159638956d33E18951d98238a53b9aa3C": "CalculationsYearnVaults",
    "0x62a4e0E7574E5407656A65CC8DbDf70f3C6EB04B": "Earn (new)",
    "0x50c1a2eA0a861A967D9d0FFE2AE4012c2E053804": "V2 (existing)",
    "0x3eE41C098f9666ed2eA246f4D2558010e59d63A0": "V1 (existing)",
    "0xAB1c342C7bf5Ec5F02ADEA1c2270670bCa144CbB": "IB (existing)",
}

yearn_official = pl.DataFrame(
    list(yearn_official.items()), orient="row", schema=["address", "name"]
).select("name", "address")

In [ ]:
yearn_addresses = (
    pl.concat(
        [
            yearn_vaults,
            yearn_meta,
            yearn_official,
        ]
    )
    .group_by("address")
    .agg(pl.last("name"))
    .with_columns(pl.col.address.str.to_lowercase())
).select(
    address='address',
    label=pl.lit('yearn'),
    name='name',
).group_by('address').agg(pl.last('*'))

yearn_addresses

In [ ]:
yearn_addresses = yearn_addresses.with_columns(
    yearn_addresses['address'].evm.hex_to_binary()
)

In [ ]:
yearn_addresses.write_parquet(    
    get_label_path('yearn', min_block, max_block)
)

# balancer

https://docs.balancer.fi/reference/contracts/deployment-addresses/mainnet.html

In [ ]:
balancer_addresses = {
    'AaveLinearPoolFactory (v5)': '0x0b576c1245F479506e7C8bbc4dB4db07C1CD31F9',
    'ComposableStablePoolFactory (v5)': '0xDB8d758BCb971e482B2C45f7F8a7740283A1bd3A',
    'ERC4626LinearPoolFactory (v4)': '0x813EE7a840CE909E7Fea2117A44a90b8063bd4fd',
    'EulerLinearPoolFactory': '0x5F43FBa61f63Fa6bFF101a0A0458cEA917f6B347',
    'GearboxLinearPoolFactory (v2)': '0x39A79EB449Fc05C92c39aA6f0e9BfaC03BE8dE5B',
    'ManagedPoolFactory (v2)': '0xBF904F9F340745B4f0c4702c7B6Ab1e808eA6b93',
    'SiloLinearPoolFactory (v2)': '0x4E11AEec21baF1660b1a46472963cB3DA7811C89',
    'WeightedPoolFactory (v4)': '0x897888115Ada5773E02aA29F775430BFB5F34c51',
    'YearnLinearPoolFactory (v2)': '0x5F5222Ffa40F2AEd6380D022184D6ea67C776eE0',
    'BalancerQueries': '0xE39B5e3B6D74016b2F6A9673D7d7493B6DF549d5',
    'BalancerRelayer (v6)': '0x35Cea9e57A393ac66Aaa7E25C391D52C74B5648f',
    'BatchRelayerLibrary (v6)': '0xeA66501dF1A00261E3bB79D1E90444fc6A186B62',
    'ProtocolFeePercentagesProvider': '0x97207B095e4D5C9a6e4cfbfcd2C3358E03B90c4A',
    'Vault': '0xBA12222222228d8Ba445958a75a0704d566BF2C8',
    'Authorizer': '0xA331D84eC860Bf466b4CdCcFb4aC09a1B43F3aE6',
    'AuthorizerAdaptor': '0x8F42aDBbA1B16EaAE3BB5754915E0D06059aDd75',
    'AuthorizerAdaptorEntrypoint': '0xf5dECDB1f3d1ee384908Fbe16D2F0348AE43a9eA',
    'AuthorizerWithAdaptorValidation': '0x6048A8c631Fb7e77EcA533Cf9C29784e482391e7',
    'ArbitrumRootGauge (v2)': '0x6337949cbC4825Bbd09242c811770F6F6fee9FfC',
    'BALTokenHolderFactory': '0xB848f50141F3D4255b37aC288C25C109104F2158',
    'BalancerMinter': '0x239e55F427D44C3cc793f49bFB507ebe76638a2b',
    'BalancerTokenAdmin': '0xf302f9F50958c5593770FDf4d4812309fF77414f',
    'DistributionScheduler': '0xBd35248F8325DD1cB2bBf9D01E80A6bb99a792Dd',
    'FeeDistributor (v2)': '0xD3cf852898b21fc233251427c2DC93d3d604F3BB',
    'GaugeAdder (v4)': '0x5DbAd78818D4c8958EfF2d5b95b28385A22113Cd',
    'GaugeController': '0xC128468b7Ce63eA702C1f104D55A2566b13D3ABD',
    'GnosisRootGauge': '0x05277CE7D1e365d660624612d8b8b9B55bFD4518',
    'GnosisRootGaugeFactory': '0x2a18B396829bc29F66a1E59fAdd7a0269A6605E8',
    'OptimismRootGauge (v2)': '0xBC230b1a66A138cD9cFC7b352390025978dFAdde',
    'OptimismRootGaugeFactory (v2)': '0x866D4B65694c66fbFD15Dd6fa933D0A6b3940A36',
    'PolygonRootGauge (v2)': '0xfeb1A24C2752E53576133cdb718F25bC64eBDD52',
    'PolygonRootGaugeFactory (v2)': '0xa98Bce70c92aD2ef3288dbcd659bC0d6b62f8F13',
    'ProtocolFeesCollector': '0xce88686553686DA562CE7Cea497CE749DA109f9F',
    'ProtocolFeesWithdrawer': '0x5ef4c5352882b10893b70DbcaA0C000965bd23c5',
    'SingleRecipientGauge (v2)': '0xb2007B8B7E0260042517f635CFd8E6dD2Dd7f007',
    'SingleRecipientGaugeFactory (v2)': '0x4fb47126Fa83A8734991E41B942Ac29A3266C968',
    'SmartWalletChecker': '0x7869296Efd0a76872fEE62A058C8fBca5c1c826C',
    'VeBoostV2 (v2)': '0x67F8DF125B796B05895a6dc8Ecf944b9556ecb0B',
    'VotingEscrow': '0xC128a9954e6c874eA3d62ce62B468bA073093F25',
    'VotingEscrowDelegation': '0x2E96068b3D5B5BAE3D7515da4A1D2E52d08A2647',
    'VotingEscrowDelegationProxy': '0x6f5a2eE11E7a772AeB5114A20d0D7c0ff61EB8A0',
    'ArbitrumRootGaugeFactory (v2)': '0x1c99324EDC771c82A0DCCB780CC7DDA0045E50e7',
    'AvalancheRootGauge (v2)': '0x3Eae4a1c2E36870A006E816930d9f55DF0a72a13',
    'AvalancheRootGaugeFactory (v2)': '0x22625eEDd92c81a219A83e1dc48f88d54786B017',
    'BalancerHelpers': '0x5aDDCCa35b7A0D07C74063c48700C8590E87864E',
    'BaseRootGauge': '0x9a4d642b9876231BB9062559521A48097abFe6CB',
    'BaseRootGaugeFactory': '0x8e3B64b3737097F283E965869e3503AA20F31E4D',
    'BatchRelayerQueryLibrary (v6)': '0x481Ca759BABB6fF011E11890e183bE00de3714e7',
    'ChainlinkRateProviderFactory': '0x1311Fbc9F60359639174c1e7cC2032DbDb5Cc4d1',
    'DoubleEntrypointFixRelayer': '0xcA96C4f198d343E251b1a01F3EBA061ef3DA73C1',
    'GaugeWorkingBalanceHelper': '0xdAB2583911E872a00A851fB80dCC78a4B46BA57c',
    'LiquidityGaugeFactory (v2)': '0xf1665E19bc105BE4EDD3739F88315cC699cc5b65',
    'LiquidityGaugeV5 (v2)': '0xe5F96070CA00cd54795416B1a4b4c2403231c548',
    'MerkleOrchard (v2)': '0xE3881627B8DeeBCCF9c23B291430a549Fc0bE5F7',
    'NoProtocolFeeLiquidityBootstrappingPoolFactory': '0x0F3e0c4218b7b0108a3643cFe9D3ec0d4F57c54e',
    'OmniVotingEscrowAdaptor': '0x96484f2aBF5e58b15176dbF1A799627B53F13B6d',
    'PolygonZkEVMRootGauge': '0x45bF48D996d22afc9BC150DF7FB4d13A49088602',
    'PolygonZkEVMRootGaugeFactory': '0x9bF951848288cCD87d06FaC426150262cD3447De',
    'PreseededVotingEscrowDelegation': '0xB496FF44746A8693A060FafD984Da41B253f6790',
    'ProtocolIdRegistry': '0xc3ccacE87f6d3A81724075ADcb5ddd85a8A1bB68',
    'StakelessGaugeCheckpointer (v2)': '0x0C8f71D19f87c0bD1b9baD2484EcC3388D5DbB98',
    'VotingEscrowRemapper': '0x83E443EF4f9963C77bd860f94500075556668cb8',
    'WstETHRateProvider': '0x72D07D7DcA67b8A406aD1Ec34ce969c90bFEE768',
    'AaveLinearPoolFactory': '0xD7FAD3bd59D6477cbe1BE7f646F7f1BA25b230f8',
    'AaveLinearPoolFactory (v2)': '0x6A0AC04f5C2A10297D5FA79FA6358837a8770041',
    'AaveLinearPoolFactory (v3)': '0x7d833FEF5BB92ddb578DA85fc0c35cD5Cc00Fb3e',
    'AaveLinearPoolFactory (v4)': '0xb9F8AB3ED3F3aCBa64Bc6cd2DcA74B7F38fD7B88',
    'ArbitrumRootGauge': '0x715C777C6149C31A75D0d29633bbBE0A14370bDC',
    'ArbitrumRootGaugeFactory': '0xad901309d9e9DbC5Df19c84f729f429F0189a633',
    'AvalancheRootGauge': '0xc57b4BE50BF6BBe5f6a4fE379DA342B32b37bF2f',
    'AvalancheRootGaugeFactory': '0x10f3e79911A490aa5B5D5CDA6F111029c4Eab5AC',
    'BalancerRelayer': '0xAc9f49eF3ab0BbC929f7b1bb0A17E1Fca5786251',
    'BalancerRelayer (v2)': '0x51CC53375A8920aE54C0561E73a9d0423A74832e',
    'BalancerRelayer (v3)': '0x886A3Ec7bcC508B8795990B60Fa21f85F9dB7948',
    'BalancerRelayer (v4)': '0x2536dfeeCB7A0397CF98eDaDA8486254533b1aFA',
    'BalancerRelayer (v5)': '0xfeA793Aa415061C483D2390414275AD314B3F621',
    'BatchRelayerLibrary': '0x41B953164995c11C81DA73D212ED8Af25741b7Ac',
    'BatchRelayerLibrary (v2)': '0xd45369c11870e2057D5be17Cc106d32Ea416F7c4',
    'BatchRelayerLibrary (v3)': '0xD966d712F470067B60D37246404D6DFe5Bf0B419',
    'BatchRelayerLibrary (v4)': '0xd02992266BB6a6324A3aB8B62FeCBc9a3C58d1F9',
    'BatchRelayerLibrary (v5)': '0xf77018c0d817dA22caDbDf504C00c0d32cE1e5C2',
    'CircuitBreakerLib': '0x956CCab09898C0AF2aCa5e6C229c3aD4E93d9288',
    'ComposableStablePoolFactory': '0xf9ac7B9dF2b3454E841110CcE5550bD5AC6f875F',
    'ComposableStablePoolFactory (v2)': '0x85a80afee867aDf27B50BdB7b76DA70f1E853062',
    'ComposableStablePoolFactory (v3)': '0xdba127fBc23fb20F5929C546af220A991b5C6e01',
    'ComposableStablePoolFactory (v4)': '0xfADa0f4547AB2de89D1304A668C39B3E09Aa7c76',
    'ERC4626LinearPoolFactory (v2)': '0xE061bF85648e9FA7b59394668CfEef980aEc4c66',
    'ERC4626LinearPoolFactory (v3)': '0x67A25ca2350Ebf4a0C475cA74C257C94a373b828',
    'FeeDistributor': '0x26743984e3357eFC59f2fd6C1aFDC310335a61c9',
    'GaugeAdder': '0xEd5ba579bB5D516263ff6E1C10fcAc1040075Fe2',
    'GaugeAdder (v2)': '0x2fFB7B215Ae7F088eC2530C7aa8E1B24E398f26a',
    'GaugeAdder (v3)': '0x5efBb12F01f27F0E020565866effC1dA491E91A4',
    'GearboxLinearPoolFactory': '0x2EbE41E1aa44D61c206A94474932dADC7D3FD9E3',
    'InvestmentPoolFactory': '0x48767F9F868a4A7b86A90736632F6E44C2df7fa9',
    'L2GaugeCheckpointer': '0x343688C5cB92115a52cA485af7f62B4B7A2e9CcC',
    'LidoRelayer': '0xdcdbf71A870cc60C6F9B621E28a7D3Ffd6Dd4965',
    'LiquidityBootstrappingPoolFactory': '0x751A0bC0e3f75b38e01Cf25bFCE7fF36DE1C87DE',
    'LiquidityGaugeFactory': '0x4E7bBd911cf1EFa442BC1b2e9Ea01ffE785412EC',
    'LiquidityGaugeV5': '0x3b8cA519122CdD8efb272b0D3085453404B25bD0',
    'ManagedPoolAddRemoveTokenLib': '0xE42FFA682A26EF8F25891db4882932711D42e467',
    'ManagedPoolFactory': '0x9Ac3E70dB606659Bf32D4BdFbb687AD193FD1F5B',
    'MerkleOrchard': '0xdAE7e32ADc5d490a43cCba1f0c736033F2b4eFca',
    'MerkleRedeem': '0x884226c9f7b7205f607922E0431419276a64CF8f',
    'MetaStablePoolFactory': '0x67d27634E44793fE63c467035E31ea8635117cd4',
    'MockAaveLendingPool (v3)': '0x9061D0D56F162D3de7f855828A34ace1eEd3a5BE',
    'MockAaveLendingPool (v4)': '0x9dd5Db2d38b50bEF682cE532bCca5DfD203915E1',
    'MockAaveLinearPool (v3)': '0x813E3fE1761f714c502d1d2d3a7CCEB33f37F59D',
    'MockAaveLinearPool (v4)': '0x331d50e0b00fc1C32742F151E56B9B616227E23E',
    'MockComposableStablePool (v2)': '0x373b347bc87998b151A5E9B6bB6ca692b766648a',
    'MockComposableStablePool (v3)': '0x222bc81C6F3C17e9e9Aba47a12f55a1Dea42f163',
    'MockComposableStablePool (v4)': '0x5537f945D8c3FCFDc1b8DECEEBD220FAD26aFdA8',
    'MockERC4626LinearPool (v3)': '0xfeF969638C52899f91781f1Be594aF6f40B99BAd',
    'MockERC4626Token (v3)': '0xE605Dbe1cA85dCdb8F43CEfA427f3B0fC06f6ba6',
    'MockGearboxDieselToken': '0x4bE0E4d6184348c5BA845a4010528CFC779610b8',
    'MockGearboxLinearPool': '0xa8b103A10A94f4f2D7Ed2FdCd5545E8075573307',
    'MockGearboxVault': '0x7ADbdabaA80F654568421887c12F09E0C7BD9629',
    'MockShareToken': '0xBc4099300E69Ff4e52F3b18728054EF00449b9e5',
    'MockSilo': '0xE719Aef17468c7e10c0c205be62C990754DFF7E5',
    'MockSiloLinearPool': '0x395d8A1D9aD82B5ABE558F8AbbFe183B27138af4',
    'MockSiloRepository': '0xdB6dF721A6E7Fdb97363079B01F107860Ac156F9',
    'MockStaticAToken (v3)': '0xf9F5E0f3f74F57755a8c8be7b2FeDcAa40673080',
    'MockStaticAToken (v4)': '0x9805dcfD25e6De36bad8fe9D3Fe2c9b44B764102',
    'MockWeightedPool (v3)': '0x1576d472d82A72a1Ec7C57FcA770BD752D124A62',
    'MockYearnLinearPool': '0x0a0fb4ff697de5ac5b6770cd8ee1b72af80b57cf',
    'MockYearnTokenVault': '0x5C5fCf8fBd4cd563cED27e7D066b88ee20E1867A',
    'OptimismRootGauge': '0xcA734aD0b286EA7dD6367a504a48FB75cd12a97F',
    'OptimismRootGaugeFactory': '0x3083A1C455ff38d39e58Dbac5040f465cF73C5c8',
    'PolygonRootGauge': '0xe53D8f8aAaFD95FCc7196E393EC5a907Df44495E',
    'PolygonRootGaugeFactory': '0x4C4287b07d293E361281bCeEe8715c8CDeB64E34',
    'QueryProcessor': '0x469b58680774AAc9Ad66447eFB4EF634756A2cC5',
    'SiloLinearPoolFactory': '0xfd1c0e6f02f71842b6ffF7CdC7A017eE1Fd3CdAC',
    'SingleRecipientGauge': '0x5D3d2D18148baAcE45D09254513BcF2C548C4480',
    'SingleRecipientGaugeFactory': '0x94f68b54191F62f781Fe8298A8A5Fa3ed772d227',
    'StablePhantomPoolFactory': '0xb08E16cFc07C684dAA2f93C70323BAdb2A6CBFd2',
    'StablePoolFactory': '0xc66Ba2B6595D3613CCab350C886aCE23866EDe24',
    'StablePoolFactory (v2)': '0x8df6EfEc5547e31B0eb7d1291B511FF8a2bf987c',
    'StakelessGaugeCheckpointer': '0x0f08eEf2C785AA5e7539684aF04755dEC1347b7c',
    'TimelockAuthorizer': '0x9E3cD0606Db55ac68845bB60121847823712ae05',
    'TimelockAuthorizerMigrator': '0xf8ee6f1F9B54F9b2C192D703ea2d22112cBC062b',
    'UnbuttonAaveLinearPoolFactory': '0x9588c26142e345f1A0d005CfC0C6DF29A8Fa010C',
    'WeightedPool2TokensFactory': '0xA5bf2ddF098bb0Ef6d120C98217dD6B141c74EE0',
    'WeightedPoolFactory': '0x8E9aa87E45e92bad84D5F8DD1bff34Fb92637dE9',
    'WeightedPoolFactory (v2)': '0xcC508a455F5b0073973107Db6a878DdBDab957bC',
    'WeightedPoolFactory (v3)': '0x5Dd94Da3644DDD055fcf6B3E1aa310Bb7801EB8b',
    'YearnLinearPoolFactory': '0x8b7854708c0C54f9D7d1FF351D4F84E6dE0E134C',
}


In [ ]:
balancer_addresses = pl.DataFrame(
    list(balancer_addresses.items()),
    orient="row",
    schema=["name", "address"],
).with_columns(label=pl.lit("balancer")).select(
    pl.col.address.str.to_lowercase(),
    pl.col.label,
    pl.col.name,
)

balancer_addresses

In [ ]:
balancer_addresses = balancer_addresses.with_columns(
    balancer_addresses['address'].evm.hex_to_binary()
)

In [ ]:
balancer_addresses.write_parquet(    
    get_label_path('balancer', min_block, max_block)
)

# sushi

https://dev.sushi.com/docs/Developers/Deployment%20Addresses

In [ ]:
sushi_addresses = [
    ['BentoBoxV1', '0xF5BCE5077908a1b7370B9ae04AdC565EBd643966'],
    ['MasterChef', '0xc2EdaD668740f1aA35E4D8f227fB8E17dcA888Cd'],
    ['MasterChefV2', '0xbE811A0D44E2553d25d11CB8DC0d3F0D0E6430E6'],
    ['CloneRewarderTime', '0x7519c93fc5073e15d89131fd38118d73a72370f8'],
    ['CloneRewarderTime', '0x9e01aac4b3e8781a85b21d9d9f848e72af77b362'],
    ['CloneRewarderTime', '0x7512105dbb4c0e0432844070a45b7ea0d83a23fd'],
    ['CloneRewarderTime', '0x083773a02311894a7e5710d62662bfbd268e10ec'],
    ['CloneRewarderTime', '0xc0482b9d1ff34f7cd98e3a0a4584298492acf550'],
    ['CloneRewarderTime', '0xc8aeb7eea2762e3bb16c6e146502fd9ceee410b1'],
    ['CloneRewarderTime', '0x50cce4bb91ee7e0897dd7f1de3010e436852dc71'],
    ['CloneRewarderTime', '0xf960e225f48e89e2754319f803c3103cc76865be'],
    ['CloneRewarderTime', '0xc56df8b6dad2555fe42961f2db0399f4cd3abcda'],
    ['CloneRewarderTime', '0x75ff3dd673ef9fc459a52e1054db5df2a1101212'],
    ['CloneRewarderTime', '0xe7ec4353fa1c8393281dcf94e069bc4b823d907c'],
    ['CloneRewarderTime', '0x7d2581f69ff44506672268c87793fc0255a8b335'],
    ['CloneRewarderTime', '0xe058d87fc1185e38ab68893136834715b30961e1'],
    ['CloneRewarderTime', '0x2af9b355a578e8bf422bb7ebb5ee7434f24d5ef3'],
    ['CloneRewarderTime', '0xf313455fa32db78a02aebf824624cf914abe2533'],
    ['CloneRewarderTime', '0xdf3754b114a523d8b0b3647018fe2b5ef28d15b6'],
    ['CloneRewarderTime', '0xf166427ee8175aa30d6b58d10e0b044eeed3b9de'],
    ['CloneRewarderTime', '0x316834fac49396d7951b4b0c14956591b51709f8'],
    ['CloneRewarderTime', '0x4e568bf651c4a17434943304d2425d05dcbae4bc'],
    ['CloneRewarderTime', '0xefa54bae2eaa3250f096217f9d78ca599e04c9ce'],
    ['CloneRewarderTime', '0x40610c4c136bfa63b98ec58f2f392958fe8a8f5d'],
    ['CloneRewarderTime', '0xf6a15e94ef40f5b927d301c8af06012411ecca69'],
    ['CloneRewarderTime', '0x157079347ce0ea0bea0dc87e504357e8480daddb'],
    ['CloneRewarderTime', '0x5b627d6cae2991504d6af5b9eda34e8fbafbd727'],
    ['CloneRewarderTime', '0xa4be652e8a5bf0b5f4870d110c7177a2393544c8'],
    ['CloneRewarderTime', '0x900ec205c8ebda41ace176dfb0876d80502344a6'],
    ['CloneRewarderTime', '0x51d8a431f8b6c25f3b987e0bb8dd3cfb01366b79'],
    ['CloneRewarderTime', '0x5f9de45c76d67947300fa61060509433d4fc9739'],
    ['CloneRewarderTime', '0x6558e44e58aee09fb24cb63a6a8270baebd6122f'],
    ['CloneRewarderTime', '0x474a00c140d7dfea674fd67241d8368b2569d12e'],
    ['CloneRewarderTime', '0x2646cbb689bce70dd7d8e041ea35b1879412059c'],
    ['CloneRewarderTime', '0xf0660b9a3eaf3a821966ff5a7a1f93e19aa03c00'],
    ['CloneRewarderTime', '0xf3312e3d9c55993cc0dc95eb87e0b3edca8a87f7'],
    ['CloneRewarderTime', '0x8c662fcbad0d66e8c9bc824e8709de80969daa70'],
    ['CloneRewarderTime', '0x4abfe1e0e87fa22416ebbf35bba6910cb755798e'],
    ['CloneRewarderTime', '0xd254a8f7ffeb6ecd76bedb2acbcf04f196c9d7a2'],
    ['CloneRewarderTime', '0x330045c1ea0d7eea6ff8c1e9c8f7a461e125d199'],
    ['CloneRewarderTime', '0x754e3f0123a96eb7f8d81a8e067375115bbb82f7'],
    ['CloneRewarderTime', '0x173ea048c2b5a3b234ffe40eb3c8859802778bf3'],
    ['CloneRewarderTime', '0x34c8ab3bda8e1b68a4f69766e2d4f71f39c8ec09'],
    ['CloneRewarderTime', '0xbec5a9803ff6f66e4287611a5c146a812d1456e2'],
    ['CloneRewarderTime', '0x9708daec19fef81a0f21badb198cb6e255c2c627'],
    ['V2Factory', '0xC0AEe478e3658e2610c5F7A4A2E1777cE9e4f2Ac'],
    ['V2Router02', '0xd9e1cE17f2641f24aE83637ab66a2cca9C378B9F'],
    ['FuroStream', '0x4ab2FC6e258a0cA7175D05fF10C5cF798A672cAE'],
    ['FuroStreamRouter', '0x95bd909A9cB3002992E2993846b035545F15ff37'],
    ['FuroVesting', '0x0689640d190b10765f09310fCfE9C670eDe4E25B'],
    ['FuroVestingRouter', '0x8E190A2173334c67734119791CF63299A7570877'],
    ['RouteProcessor', '0x19dBa5df5383168f760617aaDD23322BC5F9Ff7b'],
    ['RouteProcessor3', '0x827179dD56d07A7eeA32e3873493835da2866976'],
    ['ConstantProductPoolFactory', '0xD75F5369724b513b497101fb15211160c1d96550'],
    ['StablePoolFactory', '0xC040F84Cf7046409f92d578eF9040fE45E6ef4be'],
    ['TridentRouter', '0xDdC1b5920723F774d2Ec2C3c9355251A20819776'],
    ['V3Factory', '0xbACEB8eC6b9355Dfc0269C18bac9d6E2Bdc29C4F'],
    ['NFTDescriptor', '0x67468E6c4418d58B1b41bc0A795BaCB824F70792'],
    ['NonfungiblePositionManager', '0x2214A42d8e2A1d20635c2cb0664422c528B6A432'],
    ['NonfungibleTokenPositionDescriptor', '0x1C4369df5732ccF317fef479B26A56e176B18ABb'],
    ['QuoterV2', '0x64e8802FE490fa7cc61d3463958199161Bb608A7'],
    ['TickLens', '0xFB70AD5a200d784E7901230E6875d91d5Fa6B68c'],
    ['V3Migrator', '0xc4817DEC4e969F7Ea0c8b5bF9913697869A98e47'],
    ['SushiXSwap', '0x011E52E4E40CF9498c79273329E8827b21E2e581'],
]

sushi_addresses = pl.DataFrame(
    sushi_addresses,
    orient="row",
    schema=["name", "address"],
).with_columns(label=pl.lit("sushi")).select(
    pl.col.address.str.to_lowercase(),
    pl.col.label,
    pl.col.name,
)

sushi_addresses = sushi_addresses.with_columns(
    sushi_addresses['address'].evm.hex_to_binary()
)

sushi_addresses

In [ ]:
erc20s = address_labels.load_address_labels('erc20s', **context)

In [ ]:
v2_factory_events = await ctc.async_get_events(
    '0xC0AEe478e3658e2610c5F7A4A2E1777cE9e4f2Ac',
    event_name='PairCreated',
)

sushi_v2_pools = v2_factory_events[['arg__token0', 'arg__token1', 'arg__pair']].evm.hex_to_binary().join(
    erc20s[['address', 'symbol']].rename({'symbol': 'symbol0'}),
    left_on='arg__token0',
    right_on='address',
    how='left',
).join(
    erc20s[['address', 'symbol']].rename({'symbol': 'symbol1'}),
    left_on='arg__token1',
    right_on='address',
    how='left',
).select(
    address='arg__pair',
    label=pl.lit('sushi'),
    name='pool__' + pl.col.symbol0 + '__' + pl.col.symbol1,
)

sushi_v2_pools

In [ ]:
v3_factory_events = await ctc.async_get_events(
    '0xbACEB8eC6b9355Dfc0269C18bac9d6E2Bdc29C4F',
    event_name='PoolCreated',
)

In [ ]:
v3_factory_events


sushi_v3_pools = v3_factory_events[['arg__token0', 'arg__token1', 'arg__pool']].evm.hex_to_binary().join(
    erc20s[['address', 'symbol']].rename({'symbol': 'symbol0'}),
    left_on='arg__token0',
    right_on='address',
    how='left',
).join(
    erc20s[['address', 'symbol']].rename({'symbol': 'symbol1'}),
    left_on='arg__token1',
    right_on='address',
    how='left',
).select(
    address='arg__pool',
    label=pl.lit('sushi'),
    name='pool__' + pl.col.symbol0 + '__' + pl.col.symbol1,
)

sushi_v3_pools

In [ ]:
sushi_addresses = pl.concat(
    [
        sushi_addresses,
        sushi_v2_pools,
        sushi_v3_pools,
    ]
)

In [ ]:
sushi_addresses.write_parquet(    
    get_label_path('sushi', min_block, max_block)
)

# compound
https://github.com/compound-finance/compound-config/blob/master/networks/mainnet.json

In [ ]:
compound_addresses = {
    "cFEI": "0x7713DD9Ca933848F6819F38B8352D9A15EA73F67",
    "cUSDPDelegate": "0xa035b9e130f2b1aedc733eefb1c67ba4c503491f",
    "cUSDP": "0x041171993284df560249b57358f931d9eb7b925d",
    "Proposal72PriceFeed": "0x046728da7cb8272284238bd3e47909823d63a58d",
    "cAAVEDelegate": "0xa035b9e130f2b1aedc733eefb1c67ba4c503491f",
    "cAAVE": "0xe65cdb6479bac1e22340e4e755fae7e509ecd06c",
    "cSUSHIDelegate": "0xa035b9e130f2b1aedc733eefb1c67ba4c503491f",
    "cSUSHI": "0x4b0181102a0112a2ef11abee5563bb4a3176c9d7",
    "cYFIDelegate": "0xa035b9e130f2b1aedc733eefb1c67ba4c503491f",
    "cYFI": "0x80a2ae356fc9ef4305676f7a3e2ed04e12c33946",
    "cTUSDDelegate": "0xce526fa199d2f772abbc5b40b3284cdab1b8e6de",
    "cTUSD": "0x12392f67bdf24fae0af363c24ac620a2f67dad86",
    "cLINKDelegate": "0x24aa720906378bb8364228bddb8cabbc1f6fe1ba",
    "cLINK": "0xface851a4921ce59e912d19329929ce6da6eb0c7",
    "GovernorBravo": "0xc0da02939e1441f497fd74f78ce7decb17b66529",
    "cWBTC2Delegate": "0x24aa720906378bb8364228bddb8cabbc1f6fe1ba",
    "cWBTC2": "0xccF4429DB6322D5C611ee964527D42E5d685DD6a",
    "wbtc2_irm": "0xF2e5dB36B0682f2CD6bC805c3a4236194e01f4D5",
    "IRM_COMP_Updateable": "0xd956188795ca6F4A74092ddca33E0Ea4cA3a1395",
    "cCOMPDelegate": "0x338f7e5d19d9953b76dd81446b142c2d9fe03482",
    "cCOMP": "0x70e36f6bf80a52b3b46b3af8e106cc0ed743e8e4",
    "cUSDC": "0x39AA39c021dfbaE8faC545936693aC917d5E7563",
    "PriceOracle": "0x02557a5e05defeffd4cae6d83ea3d173b272c904",
    "PriceOracleProxy": "0xDDc46a3B076aec7ab3Fc37420A8eDd2959764Ec4",
    "Maximillion": "0xf859A1AD94BcF445A406B892eF0d3082f4174088",
    "Base200bps_Slope2000bps_Jump20000bps_Kink90": "0x6bc8fE27D0c7207733656595e73c0D5Cf7AfaE36",
    "GovernorAlpha": "0xc0dA01a04C3f3E0be433606045bB7017A7323E38",
    "cDAI": "0x5d3a536E4D6DbD6114cc1Ead35777bAB948E3643",
    "CompoundLens": "0xdCbDb7306c6Ff46f77B349188dC18cEd9DF30299",
    "StdComptroller": "0x62F18C451af964197341d3c86D27e98C41BB8fcC",
    "Poster": "0x3c6809319201b978d821190ba03fa19a3523bd96",
    "Unitroller": "0x3d9819210A31b4961b30EF54bE2aeD79B9c9Cd3B",
    "Reservoir": "0x2775b1c75658Be0F640272CCb8c72ac986009e38",
    "Comptroller": "0x3d9819210A31b4961b30EF54bE2aeD79B9c9Cd3B",
    "cUSDT": "0xf650C3d88D12dB855b8bf7D11Be6C55A4e07dCC9",
    "cBAT": "0x6C8c6b02E7b2BE14d4fA6022Dfd6d75921D90E4E",
    "Base500bps_Slope1500bps": "0xd928c8ead620bb316d2cefe3caf81dc2dec6ff63",
    "DSR_Kink_9000bps_Jump_12000bps_AssumedRF_500bps": "0xec163986cC9a6593D6AdDcBFf5509430D348030F",
    "DSR_Updateable": "0xfeD941d39905B23D6FAf02C8301d40bD4834E27F",
    "Base0bps_Slope2000bps": "0xc64C4cBA055eFA614CE01F4BAD8A9F519C4f8FaB",
    "StdComptroller_2_6": "0x97BD4Cc841FC999194174cd1803C543247a014fe",
    "Base200bps_Slope1000bps": "0x0c3f8df27e1a00b47653fde878d68d35f00714c0",
    "cETH": "0x4Ddc2D193948926D02f9B1fE9e1daa0718270ED5",
    "cDaiDelegate": "0xbB8bE4772fAA655C255309afc3c5207aA7b896Fd",
    "cUsdtDelegate": "0x976aa93ca5Aaa569109f4267589c619a097f001D",
    "cUniDelegate": "0x338f7e5d19d9953b76dd81446b142c2d9fe03482",
    "cUNI": "0x35a18000230da775cac24873d00ff85bccded550",
    "StdComptrollerG2": "0xf592eF673057a451c49c9433E278c5d59b56132c",
    "DSR_Kink_9000bps_Jump_12000bps_AssumedRF_20000bps": "0x000000007675b5E1dA008f037A0800B309e0C493",
    "Base500bps_Slope1200bps": "0xa1046abfc2598F48C44Fb320d281d3F3c0733c9a",
    "cSAI": "0xF5DCe57282A584D2746FaF1593d3121Fcac444dC",
    "StdComptrollerG3": "0x9D0a0443FF4bB04391655B8cD205683d9fA75550",
    "Timelock": "0x6d903f6003cca6255D85CcA4D3B5E5146dC33925",
    "Base200bps_Slope3000bps": "0xBAE04CbF96391086dC643e842b517734E214D698",
    "StdComptrollerG4": "0xAf601CbFF871d0BE62D18F79C31e387c76fa0374",
    "StdComptrollerG5": "0x7b5e3521a049C8fF88e6349f33044c6Cc33c113c",
    "cREP": "0x158079Ee67Fce2f58472A96584A73C7Ab9AC95c1",
    "Base200bps_Slope2000bps_Jump8000bps_Kink90": "0x40C0C2c565335fa9C4235aC8E1CbFE2c97BAC13A",
    "WBTC": "0x2260fac5e5542a773aa44fbcfedf7c193bc2c599",
    "Base200bps_Slope222bps_Kink90_Jump40": "0x5562024784cc914069d67D89a28e3201bF7b57E7",
    "cZRX": "0xB3319f5D18Bc0D84dD1b4825Dcde5d5f7266d407",
    "cWBTC": "0xC11b1268C1A384e55C48c2391d8d480264A3A7F4",
    "PriceData": "0xc629c26dced4277419cde234012f8160a0278a79",
    "PriceFeed": "0x6D2299C48a8dD07a872FDd0F8233924872Ad1071",
    "cMKR": "0x95b4eF2869eBD94BEb4eEE400a99824BF5DC325b",
    "cMKRDelegate": "0xa035b9e130f2b1aedc733eefb1c67ba4c503491f",
    "CrowdProposalFactory": "0x54a06047087927D9B0fb21c1cf0ebd792764dDB8",
    "IRM_USDT_Updateable": "0xFB564da37B41b2F6B6EDcc3e56FbF523bD9F2012",
    "IRM_USDC_Updateable": "0xD8EC56013EA119E7181d231E5048f90fBbe753c0",
    "IRM_UNI_Updateable": "0xd88b94128ff2b8cf2d7886cd1c1e46757418ca2a",
    "cUSDCv3": "0xc3d688B66703497DAA19211EEdff47f25384cdc3",
    "Configurator": "0x316f9708bB98af7dA9c68C1C3b5e79039cD336E3",
    "Compoundv3Admin": "0x1EC63B5883C3481134FD50D5DAebc83Ecd2E8779",
    "Compoundv3Rewards": "0x1B0e765F6224C21223AeA2af16c1C46E38885a40",
    "Bulker": "0x74a81F84268744a40FEBc48f8b812a1f188D80C3",
}

compound_addresses = (
    pl.DataFrame(
        list(compound_addresses.items()),
        orient="row",
        schema=["name", "address"],
    )
    .with_columns(label=pl.lit("compound"))
    .select(
        pl.col.address.str.to_lowercase(),
        pl.col.label,
        pl.col.name,
    )
    .group_by("address")
    .agg(pl.first("*"))
)

compound_addresses

In [ ]:
compound_addresses = compound_addresses.with_columns(
    compound_addresses['address'].evm.hex_to_binary()
)

In [ ]:
compound_addresses.write_parquet(    
    get_label_path('compound', min_block, max_block)
)

# aave

In [ ]:
aave_addresses = {
    # https://github.com/studydefi/money-legos/blob/master/src/aave/contracts.ts
    'v1 LendingPoolAddressesProvider': "0x24a42fd28c976a61df5d00d0599c34c4f90748c8",
    'v1 LendingPool': "0x398ec7346dcd622edc5ae82352f02be94c62d119",
    'v1 LendingPoolCore': "0x3dfd23a6c5e8bbcfc9581d2e864a68feb6a076d3",
    'v1 aETH': "0x3a3a65aab0dd2a17e3f1947ba16138cd37d08c04",
    'v1 aDAI': "0xfc1e690f61efd961294b3e1ce3313fbd8aa4f85d",
    'v1 aUSDC': "0x9ba00d6856a4edf4665bca2c2309936572473b7e",
    'v1 aSUSD': "0x625ae63000f46200499120b906716420bd059240",
    'v1 aTUSD': "0x4da9b813057d04baef4e5800e36083717b4a0341",
    'v1 aUSDT': "0x71fc860f7d3a592a4a98740e39db31d25db65ae8",
    'v1 aBUSD': "0x6ee0f7bb50a54ab5253da0667b0dc2ee526c30a8",
    'v1 aBAT': "0xe1ba0fb44ccb0d11b80f92f4f8ed94ca3ff51d00",
    'v1 aKNC': "0x9d91be44c06d373a8a226e1f3b146956083803eb",
    'v1 aLEND': "0x7d2d3688df45ce7c552e19c27e007673da9204b8",
    'v1 aLINK': "0xa64bd6c70cb9051f6a9ba1f163fdc07e0dfb5f84",
    'v1 aMANA': "0x6fce4a401b6b80ace52baaefe4421bd188e76f6f",
    'v1 aMKR': "0x7deb5e830be29f91e298ba5ff1356bb7f8146998",
    'v1 aREP': "0x71010a9d003445ac60c4e6a7017c1e89a477b438",
    'v1 aSNX': "0x328c4c80bc7aca0834db37e6600a6c49e12da4de",
    'v1 aWBTC': "0xfc4b8ed459e00e5400be803a9bb3954234fd50e3",
    'v1 aZRX': "0x6fb0855c404e09c47c3fbca25f08d4e41f9f062f",
    'v1 aENJ': '0x712db54daa836b53ef1ecbb9c6ba3b9efb073f40',
    'v1 aAAVE': '0xba3d9687cf50fe253cd2e1cfeede1d6787344ed5',
    'v1 aREN': '0x69948cc03f478b95283f7dbf1ce764d0fc7ec54c',
    'v1 aUNI': '0xB124541127A0A657f056D9Dd06188c4F1b0e5aab',
    'v1 aYFI': '0x12e51E77DAAA58aA0E9247db7510Ea4B46F9bEAd',

    # https://docs.aave.com/developers/v/1.0/deployed-contracts/uniswap-market
    'v1 amm LendingPoolAddressProvider': '0x7fd53085b9a29d236235d6fc593b47c9c33429f1',
    'v1 amm LendingPool': '0x2F60C3EB259D63dcCa81fDE7Eaa216D9983D7C60',
    'v1 amm LendingPoolCore': '0x1012cfF81A1582ddD0616517eFB97D02c5c17E25',
    'v1 amm aUNI_USDC_ETH': '0x1D0e53A0e524E3CC92C1f0f33Ae268FfF8D7E7a5',
    'v1 amm aUNI_LINK_ETH': '0x9548DB8b1cA9b6c757485e7861918b640390169c',
    'v1 amm aUNI_DAI_ETH': '0xBbBb7F2aC04484F7F04A2C2C16f20479791BbB44',
    'v1 amm aUNI_LEND_ETH': '0xc88ebbf7c523f38ef3eb8a151273c0f0da421e63',
    'v1 amm aUNI_MKR_ETH': '0x8c69f7A4C9B38F1b48005D216c398Efb2F1Ce3e4',
    'v1 amm aUNI_SETH_ETH': '0x84BBcaB430717ff832c3904fa6515f97fc63C76F',
    'v1 amm aUniDAI': '0x048930eec73c91B44b0844aEACdEBADC2F2b6efb',
    'v1 amm aUniUSDC': '0xe02b2Ad63eFF3Ac1D5827cBd7AB9DD3DaC4f4AD0',
    'v1 amm aUniUSDT': '0xb977ee318010A5252774171494a1bCB98E7fab65',
    'v1 amm aUniETH': '0x6179078872605396Ee62960917128F9477a5DdbB',
    
     # https://github.com/aave/aave-address-book/blob/main/src/AaveV2Ethereum.sol
    'v2 POOL_ADDRESSES_PROVIDER': '0xB53C1a33016B2DC2fF3653530bfF1848a515c8c5',
    'v2 POOL': '0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9',
    'v2 POOL_CONFIGURATOR': '0x311Bb771e4F8952E6Da169b425E7e92d6Ac45756',
    'v2 ORACLE': '0xA50ba011c48153De246E5192C8f9258A2ba79Ca9',
    'v2 LENDING_RATE_ORACLE': '0x8A32f49FFbA88aba6EFF96F45D8BD1D4b3f35c7D',
    'v2 AAVE_PROTOCOL_DATA_PROVIDER': '0x057835Ad21a177dbdd3090bB1CAE03EaCF78Fc6d',
    'v2 POOL_ADMIN': '0x5300A1a15135EA4dc7aD5a167152C01EFc9b192A',
#     'v2 EMERGENCY_ADMIN': '0xCA76Ebd8617a03126B6FB84F9b1c1A0fB71C2633',  # a gnosis safe
    'v2 COLLECTOR': '0x464C71f6c2F760DdA6093dCB91C24c39e5d6e18c',
    'v2 DEFAULT_INCENTIVES_CONTROLLER': '0xd784927Ff2f95ba542BfC824c8a8a98F3495f6b5',
    'v2 EMISSION_MANAGER': '0xEE56e2B3D491590B5b31738cC34d5232F378a8D5',
    'v2 LENDING_POOL_COLLATERAL_MANAGER': '0x368e6441bB27159c6e8e6d3bbd9147BEcBA915E3',
    'v2 DEBT_SWAP_ADAPTER': '0x6A6FA664D4Fa49a6a780a1D6143f079f8dd7C33d',
    'v2 CONFIG_ENGINE': '0x9eCed0293e7B73CFf4a2b4F9C82aAc8346158bd9',
    'v2 MIGRATION_HELPER': '0xB748952c7BC638F31775245964707Bcc5DDFabFC',
    'v2 POOL_ADDRESSES_PROVIDER_REGISTRY': '0x52D306e36E3B6B02c153d0266ff0f85d18BCD413',
    'v2 RATES_FACTORY': '0xbD37610BBB1ddc2a22797F7e3f531B59902b7bA7',
    'v2 REPAY_WITH_COLLATERAL_ADAPTER': '0x80Aca0C645fEdABaa20fd2Bf0Daf57885A309FE6',
    'v2 SWAP_COLLATERAL_ADAPTER': '0x135896DE8421be2ec868E0b811006171D9df802A',
    'v2 UI_INCENTIVE_DATA_PROVIDER': '0xD01ab9a6577E1D84F142e44D49380e23A340387d',
    'v2 UI_POOL_DATA_PROVIDER': '0x00e50FAB64eBB37b87df06Aa46b8B35d5f1A4e1A',
    'v2 WALLET_BALANCE_PROVIDER': '0x8E8dAd5409E0263a51C0aB5055dA66Be28cFF922',
    'v2 WETH_GATEWAY': '0xa0d9C1E9E48Ca30c8d8C3B5D69FF5dc1f6DFfC24',
    'v2 USDT_A_TOKEN': '0x3Ed3B47Dd13EC9a98b44e6204A523E766B225811',
    'v2 USDT_V_TOKEN': '0x531842cEbbdD378f8ee36D171d6cC9C4fcf475Ec',
    'v2 USDT_S_TOKEN': '0xe91D55AB2240594855aBd11b3faAE801Fd4c4687',
    'v2 USDT_INTEREST_RATE_STRATEGY': '0xF02EC9CA513ca505f0F5347E7D784Fa6165F8a99',
    'v2 WBTC_A_TOKEN': '0x9ff58f4fFB29fA2266Ab25e75e2A8b3503311656',
    'v2 WBTC_V_TOKEN': '0x9c39809Dec7F95F5e0713634a4D0701329B3b4d2',
    'v2 WBTC_S_TOKEN': '0x51B039b9AFE64B78758f8Ef091211b5387eA717c',
    'v2 WBTC_INTEREST_RATE_STRATEGY': '0x32f3A6134590fc2d9440663d35a2F0a6265F04c4',
    'v2 WETH_A_TOKEN': '0x030bA81f1c18d280636F32af80b9AAd02Cf0854e',
    'v2 WETH_V_TOKEN': '0xF63B34710400CAd3e044cFfDcAb00a0f32E33eCf',
    'v2 WETH_S_TOKEN': '0x4e977830ba4bd783C0BB7F15d3e243f73FF57121',
    'v2 WETH_INTEREST_RATE_STRATEGY': '0xb8975328Aa52c00B9Ec1e11e518C4900f2e6C62a',
    'v2 YFI_A_TOKEN': '0x5165d24277cD063F5ac44Efd447B27025e888f37',
    'v2 YFI_V_TOKEN': '0x7EbD09022Be45AD993BAA1CEc61166Fcc8644d97',
    'v2 YFI_S_TOKEN': '0xca823F78C2Dd38993284bb42Ba9b14152082F7BD',
    'v2 YFI_INTEREST_RATE_STRATEGY': '0x2c206fa2127aB7f1CE3dc987daf683Ed5B9CF069',
    'v2 ZRX_A_TOKEN': '0xDf7FF54aAcAcbFf42dfe29DD6144A69b629f8C9e',
    'v2 ZRX_V_TOKEN': '0x85791D117A392097590bDeD3bD5abB8d5A20491A',
    'v2 ZRX_S_TOKEN': '0x071B4323a24E73A5afeEbe34118Cd21B8FAAF7C3',
    'v2 ZRX_INTEREST_RATE_STRATEGY': '0x2c206fa2127aB7f1CE3dc987daf683Ed5B9CF069',
    'v2 UNI_A_TOKEN': '0xB9D7CB55f463405CDfBe4E90a6D2Df01C2B92BF1',
    'v2 UNI_V_TOKEN': '0x5BdB050A92CADcCfCDcCCBFC17204a1C9cC0Ab73',
    'v2 UNI_S_TOKEN': '0xD939F7430dC8D5a427f156dE1012A56C18AcB6Aa',
    'v2 UNI_INTEREST_RATE_STRATEGY': '0x87cCa171a2125efca9868F9Bc1FfD7B7c80dC5a3',
    'v2 AAVE_A_TOKEN': '0xFFC97d72E13E01096502Cb8Eb52dEe56f74DAD7B',
    'v2 AAVE_V_TOKEN': '0xF7DBA49d571745D9d7fcb56225B05BEA803EBf3C',
    'v2 AAVE_S_TOKEN': '0x079D6a3E844BcECf5720478A718Edb6575362C5f',
    'v2 AAVE_INTEREST_RATE_STRATEGY': '0xd4cA26F2496195C4F886D464D8578368236bB747',
    'v2 BAT_A_TOKEN': '0x05Ec93c0365baAeAbF7AefFb0972ea7ECdD39CF1',
    'v2 BAT_V_TOKEN': '0xfc218A6Dfe6901CB34B1a5281FC6f1b8e7E56877',
    'v2 BAT_S_TOKEN': '0x277f8676FAcf4dAA5a6EA38ba511B7F65AA02f9F',
    'v2 BAT_INTEREST_RATE_STRATEGY': '0x2c206fa2127aB7f1CE3dc987daf683Ed5B9CF069',
    'v2 BUSD_A_TOKEN': '0xA361718326c15715591c299427c62086F69923D9',
    'v2 BUSD_V_TOKEN': '0xbA429f7011c9fa04cDd46a2Da24dc0FF0aC6099c',
    'v2 BUSD_S_TOKEN': '0x4A7A63909A72D268b1D8a93a9395d098688e0e5C',
    'v2 BUSD_INTEREST_RATE_STRATEGY': '0xF1AafF9a4Da6Bf4Fb8fc18d39C8ffdafbAACce69',
    'v2 DAI_A_TOKEN': '0x028171bCA77440897B824Ca71D1c56caC55b68A3',
    'v2 DAI_V_TOKEN': '0x6C3c78838c761c6Ac7bE9F59fe808ea2A6E4379d',
    'v2 DAI_S_TOKEN': '0x778A13D3eeb110A4f7bb6529F99c000119a08E92',
    'v2 DAI_INTEREST_RATE_STRATEGY': '0xab7f8ca781C736EB624A9E321cd043F8E6292C96',
    'v2 ENJ_A_TOKEN': '0xaC6Df26a590F08dcC95D5a4705ae8abbc88509Ef',
    'v2 ENJ_V_TOKEN': '0x38995F292a6E31b78203254fE1cdd5Ca1010A446',
    'v2 ENJ_S_TOKEN': '0x943DcCA156b5312Aa24c1a08769D67FEce4ac14C',
    'v2 ENJ_INTEREST_RATE_STRATEGY': '0x2c206fa2127aB7f1CE3dc987daf683Ed5B9CF069',
    'v2 KNC_A_TOKEN': '0x39C6b3e42d6A679d7D776778Fe880BC9487C2EDA',
    'v2 KNC_V_TOKEN': '0x6B05D1c608015Ccb8e205A690cB86773A96F39f1',
    'v2 KNC_S_TOKEN': '0x9915dfb872778B2890a117DA1F35F335eb06B54f',
    'v2 KNC_INTEREST_RATE_STRATEGY': '0xeB65a3163Fd8900F4fE9a044e18966706b69A74D',
    'v2 LINK_A_TOKEN': '0xa06bC25B5805d5F8d82847D191Cb4Af5A3e873E0',
    'v2 LINK_V_TOKEN': '0x0b8f12b1788BFdE65Aa1ca52E3e9F3Ba401be16D',
    'v2 LINK_S_TOKEN': '0xFB4AEc4Cc858F2539EBd3D37f2a43eAe5b15b98a',
    'v2 LINK_INTEREST_RATE_STRATEGY': '0xED6547b83276B076B771B88FcCbD68BDeDb3927f',
    'v2 MANA_A_TOKEN': '0xa685a61171bb30d4072B338c80Cb7b2c865c873E',
    'v2 MANA_V_TOKEN': '0x0A68976301e46Ca6Ce7410DB28883E309EA0D352',
    'v2 MANA_S_TOKEN': '0xD86C74eA2224f4B8591560652b50035E4e5c0a3b',
    'v2 MANA_INTEREST_RATE_STRATEGY': '0x2c206fa2127aB7f1CE3dc987daf683Ed5B9CF069',
    'v2 MKR_A_TOKEN': '0xc713e5E149D5D0715DcD1c156a020976e7E56B88',
    'v2 MKR_V_TOKEN': '0xba728eAd5e496BE00DCF66F650b6d7758eCB50f8',
    'v2 MKR_S_TOKEN': '0xC01C8E4b12a89456a9fD4e4e75B72546Bf53f0B5',
    'v2 MKR_INTEREST_RATE_STRATEGY': '0x2c206fa2127aB7f1CE3dc987daf683Ed5B9CF069',
    'v2 REN_A_TOKEN': '0xCC12AbE4ff81c9378D670De1b57F8e0Dd228D77a',
    'v2 REN_V_TOKEN': '0xcd9D82d33bd737De215cDac57FE2F7f04DF77FE0',
    'v2 REN_S_TOKEN': '0x3356Ec1eFA75d9D150Da1EC7d944D9EDf73703B7',
    'v2 REN_INTEREST_RATE_STRATEGY': '0x2c206fa2127aB7f1CE3dc987daf683Ed5B9CF069',
    'v2 SNX_A_TOKEN': '0x35f6B052C598d933D69A4EEC4D04c73A191fE6c2',
    'v2 SNX_V_TOKEN': '0x267EB8Cf715455517F9BD5834AeAE3CeA1EBdbD8',
    'v2 SNX_S_TOKEN': '0x8575c8ae70bDB71606A53AeA1c6789cB0fBF3166',
    'v2 SNX_INTEREST_RATE_STRATEGY': '0xB2D822cAdb9040F3164829BC34e41a93cA3E01e5',
    'v2 sUSD_A_TOKEN': '0x6C5024Cd4F8A59110119C56f8933403A539555EB',
    'v2 sUSD_V_TOKEN': '0xdC6a3Ab17299D9C2A412B0e0a4C1f55446AE0817',
    'v2 sUSD_S_TOKEN': '0x30B0f7324feDF89d8eff397275F8983397eFe4af',
    'v2 sUSD_INTEREST_RATE_STRATEGY': '0x28fc7a5F0e7652B6f5E398F5E656A3eB36a1F3DE',
    'v2 TUSD_A_TOKEN': '0x101cc05f4A51C0319f570d5E146a8C625198e636',
    'v2 TUSD_V_TOKEN': '0x01C0eb1f8c6F1C1bF74ae028697ce7AA2a8b0E92',
    'v2 TUSD_S_TOKEN': '0x7f38d60D94652072b2C44a18c0e14A481EC3C0dd',
    'v2 TUSD_INTEREST_RATE_STRATEGY': '0xb70e28437Aec70a8cfE5240F54c463cF849bE17C',
    'v2 USDC_A_TOKEN': '0xBcca60bB61934080951369a648Fb03DF4F96263C',
    'v2 USDC_V_TOKEN': '0x619beb58998eD2278e08620f97007e1116D5D25b',
    'v2 USDC_S_TOKEN': '0xE4922afAB0BbaDd8ab2a88E0C79d884Ad337fcA6',
    'v2 USDC_INTEREST_RATE_STRATEGY': '0xb72F23adE9b9980c2E731Ca504105fC860643619',
    'v2 CRV_A_TOKEN': '0x8dAE6Cb04688C62d939ed9B68d32Bc62e49970b1',
    'v2 CRV_V_TOKEN': '0x00ad8eBF64F141f1C81e9f8f792d3d1631c6c684',
    'v2 CRV_S_TOKEN': '0x9288059a74f589C919c7Cf1Db433251CdFEB874B',
    'v2 CRV_INTEREST_RATE_STRATEGY': '0x35FE8C9924718f286463e659be35b4122eEe907F',
    'v2 GUSD_A_TOKEN': '0xD37EE7e4f452C6638c96536e68090De8cBcdb583',
    'v2 GUSD_V_TOKEN': '0x279AF5b99540c1A3A7E3CDd326e19659401eF99e',
    'v2 GUSD_S_TOKEN': '0xf8aC64ec6Ff8E0028b37EB89772d21865321bCe0',
    'v2 GUSD_INTEREST_RATE_STRATEGY': '0x999F449fEAfaAd78CE5702A9ebc460633478D4a4',
    'v2 BAL_A_TOKEN': '0x272F97b7a56a387aE942350bBC7Df5700f8a4576',
    'v2 BAL_V_TOKEN': '0x13210D4Fe0d5402bd7Ecbc4B5bC5cFcA3b71adB0',
    'v2 BAL_S_TOKEN': '0xe569d31590307d05DA3812964F1eDd551D665a0b',
    'v2 BAL_INTEREST_RATE_STRATEGY': '0xb873ec463CAC50112B0e14d060fbc4192406279f',
    'v2 xSUSHI_A_TOKEN': '0xF256CC7847E919FAc9B808cC216cAc87CCF2f47a',
    'v2 xSUSHI_V_TOKEN': '0xfAFEDF95E21184E3d880bd56D4806c4b8d31c69A',
    'v2 xSUSHI_S_TOKEN': '0x73Bfb81D7dbA75C904f430eA8BAe82DB0D41187B',
    'v2 xSUSHI_INTEREST_RATE_STRATEGY': '0x6855E5544Cd803BF24c9612b3F12C009116B0ee1',
    'v2 renFIL_A_TOKEN': '0x514cd6756CCBe28772d4Cb81bC3156BA9d1744aa',
    'v2 renFIL_V_TOKEN': '0x348e2eBD5E962854871874E444F4122399c02755',
    'v2 renFIL_S_TOKEN': '0xcAad05C49E14075077915cB5C820EB3245aFb950',
    'v2 renFIL_INTEREST_RATE_STRATEGY': '0x311C866D55456e465e314A3E9830276B438A73f0',
    'v2 RAI_A_TOKEN': '0xc9BC48c72154ef3e5425641a3c747242112a46AF',
    'v2 RAI_V_TOKEN': '0xB5385132EE8321977FfF44b60cDE9fE9AB0B4e6b',
    'v2 RAI_S_TOKEN': '0x9C72B8476C33AE214ee3e8C20F0bc28496a62032',
    'v2 RAI_INTEREST_RATE_STRATEGY': '0xc83cDb49F234EC6666D7857B3EAa1302b109394C',
    'v2 AMPL_A_TOKEN': '0x1E6bb68Acec8fefBD87D192bE09bb274170a0548',
    'v2 AMPL_V_TOKEN': '0xf013D90E4e4E3Baf420dFea60735e75dbd42f1e1',
    'v2 AMPL_S_TOKEN': '0x18152C9f77DAdc737006e9430dB913159645fa87',
    'v2 AMPL_INTEREST_RATE_STRATEGY': '0xB2D822cAdb9040F3164829BC34e41a93cA3E01e5',
    'v2 USDP_A_TOKEN': '0x2e8F4bdbE3d47d7d7DE490437AeA9915D930F1A3',
    'v2 USDP_V_TOKEN': '0xFDb93B3b10936cf81FA59A02A7523B6e2149b2B7',
    'v2 USDP_S_TOKEN': '0x2387119bc85A74e0BBcbe190d80676CB16F10D4F',
    'v2 USDP_INTEREST_RATE_STRATEGY': '0x40a76E018dd4aFe1391F51f0C8c8E4eA70476e15',
    'v2 DPI_A_TOKEN': '0x6F634c6135D2EBD550000ac92F494F9CB8183dAe',
    'v2 DPI_V_TOKEN': '0x4dDff5885a67E4EffeC55875a3977D7E60F82ae0',
    'v2 DPI_S_TOKEN': '0xa3953F07f389d719F99FC378ebDb9276177d8A6e',
    'v2 DPI_INTEREST_RATE_STRATEGY': '0xA12D7bB07379ad6de4497B80088e28D4C00932D1',
    'v2 FRAX_A_TOKEN': '0xd4937682df3C8aEF4FE912A96A74121C0829E664',
    'v2 FRAX_V_TOKEN': '0xfE8F19B17fFeF0fDbfe2671F248903055AFAA8Ca',
    'v2 FRAX_S_TOKEN': '0x3916e3B6c84b161df1b2733dFfc9569a1dA710c2',
    'v2 FRAX_INTEREST_RATE_STRATEGY': '0x28fc7a5F0e7652B6f5E398F5E656A3eB36a1F3DE',
    'v2 FEI_A_TOKEN': '0x683923dB55Fead99A79Fa01A27EeC3cB19679cC3',
    'v2 FEI_V_TOKEN': '0xC2e10006AccAb7B45D9184FcF5b7EC7763f5BaAe',
    'v2 FEI_S_TOKEN': '0xd89cF9E8A858F8B4b31Faf793505e112d6c17449',
    'v2 FEI_INTEREST_RATE_STRATEGY': '0x795dC59EA6472Dfa4298A454C6E8Dcb005643A13',
    'v2 stETH_A_TOKEN': '0x1982b2F5814301d4e9a8b0201555376e62F82428',
    'v2 stETH_V_TOKEN': '0xA9DEAc9f00Dc4310c35603FCD9D34d1A750f81Db',
    'v2 stETH_S_TOKEN': '0x66457616Dd8489dF5D0AFD8678F4A260088aAF55',
    'v2 stETH_INTEREST_RATE_STRATEGY': '0xff04ed5f7a6C3a0F1e5Ea20617F8C6f513D5A77c',
    'v2 ENS_A_TOKEN': '0x9a14e23A58edf4EFDcB360f68cd1b95ce2081a2F',
    'v2 ENS_V_TOKEN': '0x176808047cc9b7A2C9AE202c593ED42dDD7C0D13',
    'v2 ENS_S_TOKEN': '0x34441FFD1948E49dC7a607882D0c38Efd0083815',
    'v2 ENS_INTEREST_RATE_STRATEGY': '0x6855E5544Cd803BF24c9612b3F12C009116B0ee1',
    'v2 UST_A_TOKEN': '0xc2e2152647F4C26028482Efaf64b2Aa28779EFC4',
    'v2 UST_V_TOKEN': '0xaf32001cf2E66C4C3af4205F6EA77112AA4160FE',
    'v2 UST_S_TOKEN': '0x7FDbfB0412700D94403c42cA3CAEeeA183F07B26',
    'v2 UST_INTEREST_RATE_STRATEGY': '0xc83cDb49F234EC6666D7857B3EAa1302b109394C',
    'v2 CVX_A_TOKEN': '0x952749E07d7157bb9644A894dFAF3Bad5eF6D918',
    'v2 CVX_V_TOKEN': '0x4Ae5E4409C6Dbc84A00f9f89e4ba096603fb7d50',
    'v2 CVX_S_TOKEN': '0xB01Eb1cE1Da06179136D561766fc2d609C5F55Eb',
    'v2 CVX_INTEREST_RATE_STRATEGY': '0x2c206fa2127aB7f1CE3dc987daf683Ed5B9CF069',
    'v2 ONE_INCH_A_TOKEN': '0xB29130CBcC3F791f077eAdE0266168E808E5151e',
    'v2 ONE_INCH_V_TOKEN': '0xD7896C1B9b4455aFf31473908eB15796ad2295DA',
    'v2 ONE_INCH_S_TOKEN': '0x1278d6ED804d59d2d18a5Aa5638DfD591A79aF0a',
    'v2 ONE_INCH_INTEREST_RATE_STRATEGY': '0x6855E5544Cd803BF24c9612b3F12C009116B0ee1',
    'v2 LUSD_A_TOKEN': '0xce1871f791548600cb59efbefFC9c38719142079',
    'v2 LUSD_V_TOKEN': '0x411066489AB40442d6Fc215aD7c64224120D33F2',
    'v2 LUSD_S_TOKEN': '0x39f010127274b2dBdB770B45e1de54d974974526',
    'v2 LUSD_INTEREST_RATE_STRATEGY': '0xab7f8ca781C736EB624A9E321cd043F8E6292C96',

    # https://github.com/aave/aave-address-book/blob/main/src/AaveV2EthereumArc.sol
    'v2 arc POOL_ADDRESSES_PROVIDER': '0x6FdfafB66d39cD72CFE7984D3Bbcc76632faAb00',
    'v2 arc POOL': '0x37D7306019a38Af123e4b245Eb6C28AF552e0bB0',
    'v2 arc POOL_CONFIGURATOR': '0x4e1c7865e7BE78A7748724Fa0409e88dc14E67aA',
    'v2 arc ORACLE': '0xB8a7bc0d13B1f5460513040a97F404b4fea7D2f3',
    'v2 arc LENDING_RATE_ORACLE': '0xfA3c34d734fe0106C87917683ca45dffBe3b3B00',
    'v2 arc AAVE_PROTOCOL_DATA_PROVIDER': '0x71B53fC437cCD988b1b89B1D4605c3c3d0C810ea',
    'v2 arc POOL_ADMIN': '0xAce1d11d836cb3F51Ef658FD4D353fFb3c301218',
#     'v2 arc EMERGENCY_ADMIN': '0x33B09130b035d6D7e57d76fEa0873d9545FA7557',  # a gnosis safe
    'v2 arc COLLECTOR': '0x464C71f6c2F760DdA6093dCB91C24c39e5d6e18c',
    'v2 arc LENDING_POOL_COLLATERAL_MANAGER': '0x837696219C9a3775a856BEBC02DB1fA918C8a46e',
    'v2 arc PERMISSION_MANAGER': '0xF4a1F5fEA79C3609514A417425971FadC10eCfBE',

    # https://github.com/aave/aave-address-book/blob/main/src/AaveV2EthereumAMM.sol
    'v2 amm POOL_ADDRESSES_PROVIDER': '0xAcc030EF66f9dFEAE9CbB0cd1B25654b82cFA8d5',
    'v2 amm POOL': '0x7937D4799803FbBe595ed57278Bc4cA21f3bFfCB',
    'v2 amm POOL_CONFIGURATOR': '0x23A875eDe3F1030138701683e42E9b16A7F87768',
    'v2 amm ORACLE': '0xA50ba011c48153De246E5192C8f9258A2ba79Ca9',
    'v2 amm LENDING_RATE_ORACLE': '0x8A32f49FFbA88aba6EFF96F45D8BD1D4b3f35c7D',
    'v2 amm AAVE_PROTOCOL_DATA_PROVIDER': '0xc443AD9DDE3cecfB9dfC5736578f447aFE3590ba',
    'v2 amm POOL_ADMIN': '0x5300A1a15135EA4dc7aD5a167152C01EFc9b192A',
#     'v2 amm EMERGENCY_ADMIN': '0xCA76Ebd8617a03126B6FB84F9b1c1A0fB71C2633',  # a gnosis safe
    'v2 amm COLLECTOR': '0x464C71f6c2F760DdA6093dCB91C24c39e5d6e18c',
    'v2 amm LENDING_POOL_COLLATERAL_MANAGER': '0x368e6441bB27159c6e8e6d3bbd9147BEcBA915E3',
    'v2 amm CONFIG_ENGINE': '0xcfC26009618ec2Ca8787180116a37Caa354a465C',
    'v2 amm MIGRATION_HELPER': '0x3cd1dFB81C50A5300C60a181ED145a7286d81e0a',
    'v2 amm POOL_ADDRESSES_PROVIDER_REGISTRY': '0x52D306e36E3B6B02c153d0266ff0f85d18BCD413',
    'v2 amm RATES_FACTORY': '0x6e4D068105052C3877116DCF86f5FF36B7eCa2B8',
    'v2 amm UI_INCENTIVE_DATA_PROVIDER': '0xD01ab9a6577E1D84F142e44D49380e23A340387d',
    'v2 amm UI_POOL_DATA_PROVIDER': '0x00e50FAB64eBB37b87df06Aa46b8B35d5f1A4e1A',
    'v2 amm WALLET_BALANCE_PROVIDER': '0x8E8dAd5409E0263a51C0aB5055dA66Be28cFF922',
    'v2 amm WETH_GATEWAY': '0xbe9a7B3F2f54E18D7C0a17B03ad84Ac2D1D28eAC',
    'v2 amm WETH_A_TOKEN': '0xf9Fb4AD91812b704Ba883B11d2B576E890a6730A',
    'v2 amm WETH_V_TOKEN': '0xA4C273d9A0C1fe2674F0E845160d6232768a3064',
    'v2 amm WETH_S_TOKEN': '0x118Ee405c6be8f9BA7cC7a98064EB5DA462235CF',
    'v2 amm WETH_INTEREST_RATE_STRATEGY': '0x8d02bac65cd84343eF8239d277794bad455cE889',
    'v2 amm DAI_A_TOKEN': '0x79bE75FFC64DD58e66787E4Eae470c8a1FD08ba4',
    'v2 amm DAI_V_TOKEN': '0x3F4fA4937E72991367DC32687BC3278f095E7EAa',
    'v2 amm DAI_S_TOKEN': '0x8da51a5a3129343468a63A96ccae1ff1352a3dfE',
    'v2 amm DAI_ORACLE': '0x773616E4d11A78F511299002da57A0a94577F1f4',
    'v2 amm DAI_INTEREST_RATE_STRATEGY': '0x79F40CDF9f491f148E522D7845c3fBF61E56c33F',
    'v2 amm USDC_A_TOKEN': '0xd24946147829DEaA935bE2aD85A3291dbf109c80',
    'v2 amm USDC_V_TOKEN': '0xCFDC74b97b69319683fec2A4Ef95c4Ab739F1B12',
    'v2 amm USDC_S_TOKEN': '0xE5971a8a741892F3b3ac3E9c94d02588190cE220',
    'v2 amm USDC_ORACLE': '0x986b5E1e1755e3C2440e960477f25201B0a8bbD4',
    'v2 amm USDC_INTEREST_RATE_STRATEGY': '0x79F40CDF9f491f148E522D7845c3fBF61E56c33F',
    'v2 amm USDT_A_TOKEN': '0x17a79792Fe6fE5C95dFE95Fe3fCEE3CAf4fE4Cb7',
    'v2 amm USDT_V_TOKEN': '0xDcFE9BfC246b02Da384de757464a35eFCa402797',
    'v2 amm USDT_S_TOKEN': '0x04A0577a89E1b9E8f6c87ee26cCe6a168fFfC5b5',
    'v2 amm USDT_ORACLE': '0xEe9F2375b4bdF6387aa8265dD4FB8F16512A1d46',
    'v2 amm USDT_INTEREST_RATE_STRATEGY': '0x79F40CDF9f491f148E522D7845c3fBF61E56c33F',
    'v2 amm WBTC_A_TOKEN': '0x13B2f6928D7204328b0E8E4BCd0379aA06EA21FA',
    'v2 amm WBTC_V_TOKEN': '0x3b99fdaFdfE70d65101a4ba8cDC35dAFbD26375f',
    'v2 amm WBTC_S_TOKEN': '0x55E575d092c934503D7635A837584E2900e01d2b',
    'v2 amm WBTC_ORACLE': '0xFD858c8bC5ac5e10f01018bC78471bb0DC392247',
    'v2 amm WBTC_INTEREST_RATE_STRATEGY': '0x8d02bac65cd84343eF8239d277794bad455cE889',
    'v2 amm UNI_DAI_WETH_A_TOKEN': '0x9303EabC860a743aABcc3A1629014CaBcc3F8D36',
    'v2 amm UNI_DAI_WETH_V_TOKEN': '0x23bcc861b989762275165d08B127911F09c71628',
    'v2 amm UNI_DAI_WETH_S_TOKEN': '0xE9562bf0A11315A1e39f9182F446eA58002f010E',
    'v2 amm UNI_DAI_WETH_ORACLE': '0x66A6b87A18DB78086acda75b7720DC47CdABcC05',
    'v2 amm UNI_DAI_WETH_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm UNI_WBTC_WETH_A_TOKEN': '0xc58F53A8adff2fB4eb16ED56635772075E2EE123',
    'v2 amm UNI_WBTC_WETH_V_TOKEN': '0x02aAeB4C7736177242Ee0f71f6f6A0F057Aba87d',
    'v2 amm UNI_WBTC_WETH_S_TOKEN': '0xeef7d082D9bE2F5eC73C072228706286dea1f492',
    'v2 amm UNI_WBTC_WETH_ORACLE': '0x7004BB6F2013F13C54899309cCa029B49707E547',
    'v2 amm UNI_WBTC_WETH_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm UNI_AAVE_WETH_A_TOKEN': '0xe59d2FF6995a926A574390824a657eEd36801E55',
    'v2 amm UNI_AAVE_WETH_V_TOKEN': '0x859ED7D9E92d1fe42fF95C3BC3a62F7cB59C373E',
    'v2 amm UNI_AAVE_WETH_S_TOKEN': '0x997b26eFf106f138e71160022CaAb0AFC5814643',
    'v2 amm UNI_AAVE_WETH_ORACLE': '0xB525547968610395B60085bDc8033FFeaEaa5F64',
    'v2 amm UNI_AAVE_WETH_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm UNI_BAT_WETH_A_TOKEN': '0xA1B0edF4460CC4d8bFAA18Ed871bFF15E5b57Eb4',
    'v2 amm UNI_BAT_WETH_V_TOKEN': '0x3Fbef89A21Dc836275bC912849627b33c61b09b4',
    'v2 amm UNI_BAT_WETH_S_TOKEN': '0x27c67541a4ea26a436e311b2E6fFeC82083a6983',
    'v2 amm UNI_BAT_WETH_ORACLE': '0xB394D8a1CE721630Cbea8Ec110DCEf0D283EDE3a',
    'v2 amm UNI_BAT_WETH_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm UNI_DAI_USDC_A_TOKEN': '0xE340B25fE32B1011616bb8EC495A4d503e322177',
    'v2 amm UNI_DAI_USDC_V_TOKEN': '0x925E3FDd927E20e33C3177C4ff6fb72aD1133C87',
    'v2 amm UNI_DAI_USDC_S_TOKEN': '0x6Bb2BdD21920FcB2Ad855AB5d523222F31709d1f',
    'v2 amm UNI_DAI_USDC_ORACLE': '0x3B148Fa5E8297DB64262442052b227328730EA81',
    'v2 amm UNI_DAI_USDC_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm UNI_CRV_WETH_A_TOKEN': '0x0ea20e7fFB006d4Cfe84df2F72d8c7bD89247DB0',
    'v2 amm UNI_CRV_WETH_V_TOKEN': '0xF3f1a76cA6356a908CdCdE6b2AC2eaace3739Cd0',
    'v2 amm UNI_CRV_WETH_S_TOKEN': '0xd6035f8803eE9f173b1D3EBc3BDE0Ea6B5165636',
    'v2 amm UNI_CRV_WETH_ORACLE': '0x10F7078e2f29802D2AC78045F61A69aE0883535A',
    'v2 amm UNI_CRV_WETH_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm UNI_LINK_WETH_A_TOKEN': '0xb8db81B84d30E2387de0FF330420A4AAA6688134',
    'v2 amm UNI_LINK_WETH_V_TOKEN': '0xeDe4052ed8e1F422F4E5062c679f6B18693fEcdc',
    'v2 amm UNI_LINK_WETH_S_TOKEN': '0xeb32b3A1De9a1915D2b452B673C53883b9Fa6a97',
    'v2 amm UNI_LINK_WETH_ORACLE': '0x30adCEfA5d483284FD79E1eFd54ED3e0A8eaA632',
    'v2 amm UNI_LINK_WETH_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm UNI_MKR_WETH_A_TOKEN': '0x370adc71f67f581158Dc56f539dF5F399128Ddf9',
    'v2 amm UNI_MKR_WETH_V_TOKEN': '0xf36C394775285F89bBBDF09533421E3e81e8447c',
    'v2 amm UNI_MKR_WETH_S_TOKEN': '0x6E7E38bB73E19b62AB5567940Caaa514e9d85982',
    'v2 amm UNI_MKR_WETH_ORACLE': '0xEBF4A448ff3D835F8FA883941a3E9D5E74B40B5E',
    'v2 amm UNI_MKR_WETH_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm UNI_REN_WETH_A_TOKEN': '0xA9e201A4e269d6cd5E9F0FcbcB78520cf815878B',
    'v2 amm UNI_REN_WETH_V_TOKEN': '0x2A8d5B1c1de15bfcd5EC41368C0295c60D8Da83c',
    'v2 amm UNI_REN_WETH_S_TOKEN': '0x312edeADf68E69A0f53518bF27EAcD1AbcC2897e',
    'v2 amm UNI_REN_WETH_ORACLE': '0xe2f7C06906A9dB063C28EB5c71B6Ab454e5222dD',
    'v2 amm UNI_REN_WETH_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm UNI_SNX_WETH_A_TOKEN': '0x38E491A71291CD43E8DE63b7253E482622184894',
    'v2 amm UNI_SNX_WETH_V_TOKEN': '0xfd15008efA339A2390B48d2E0Ca8Abd523b406d3',
    'v2 amm UNI_SNX_WETH_S_TOKEN': '0xef62A0C391D89381ddf8A8C90Ba772081107D287',
    'v2 amm UNI_SNX_WETH_ORACLE': '0x29bfee7E90572Abf1088a58a145a10D051b78E46',
    'v2 amm UNI_SNX_WETH_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm UNI_UNI_WETH_A_TOKEN': '0x3D26dcd840fCC8e4B2193AcE8A092e4a65832F9f',
    'v2 amm UNI_UNI_WETH_V_TOKEN': '0x0D878FbB01fbEEa7ddEFb896d56f1D3167af919F',
    'v2 amm UNI_UNI_WETH_S_TOKEN': '0x6febCE732191Dc915D6fB7Dc5FE3AEFDDb85Bd1B',
    'v2 amm UNI_UNI_WETH_ORACLE': '0xC2E93e8121237A885A00627975eB06C7BF9808d6',
    'v2 amm UNI_UNI_WETH_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm UNI_USDC_WETH_A_TOKEN': '0x391E86e2C002C70dEe155eAceB88F7A3c38f5976',
    'v2 amm UNI_USDC_WETH_V_TOKEN': '0x26625d1dDf520fC8D975cc68eC6E0391D9d3Df61',
    'v2 amm UNI_USDC_WETH_S_TOKEN': '0xfAB4C9775A4316Ec67a8223ecD0F70F87fF532Fc',
    'v2 amm UNI_USDC_WETH_ORACLE': '0x71c4a2173CE3620982DC8A7D870297533360Da4E',
    'v2 amm UNI_USDC_WETH_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm UNI_WBTC_USDC_A_TOKEN': '0x2365a4890eD8965E564B7E2D27C38Ba67Fec4C6F',
    'v2 amm UNI_WBTC_USDC_V_TOKEN': '0x36dA0C5dC23397CBf9D13BbD74E93C04f99633Af',
    'v2 amm UNI_WBTC_USDC_S_TOKEN': '0xc66bfA05cCe646f05F71DeE333e3229cE24Bbb7e',
    'v2 amm UNI_WBTC_USDC_ORACLE': '0x11f4ba2227F21Dc2A9F0b0e6Ea740369d580a212',
    'v2 amm UNI_WBTC_USDC_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm UNI_YFI_WETH_A_TOKEN': '0x5394794Be8b6eD5572FCd6b27103F46b5F390E8f',
    'v2 amm UNI_YFI_WETH_V_TOKEN': '0xDf70Bdf01a3eBcd0D918FF97390852A914a92Df7',
    'v2 amm UNI_YFI_WETH_S_TOKEN': '0x9B054B76d6DE1c4892ba025456A9c4F9be5B1766',
    'v2 amm UNI_YFI_WETH_ORACLE': '0x664223b8Bb0934aE0970e601F452f75AaCe9Aa2A',
    'v2 amm UNI_YFI_WETH_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm BPT_WBTC_WETH_A_TOKEN': '0x358bD0d980E031E23ebA9AA793926857703783BD',
    'v2 amm BPT_WBTC_WETH_V_TOKEN': '0xF655DF3832859cfB0AcfD88eDff3452b9Aa6Db24',
    'v2 amm BPT_WBTC_WETH_S_TOKEN': '0x46406eCd20FDE1DF4d80F15F07c434fa95CB6b33',
    'v2 amm BPT_WBTC_WETH_ORACLE': '0x4CA8D8fC2b4fCe8A2dcB71Da884bba042d48E067',
    'v2 amm BPT_WBTC_WETH_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm BPT_BAL_WETH_A_TOKEN': '0xd109b2A304587569c84308c55465cd9fF0317bFB',
    'v2 amm BPT_BAL_WETH_V_TOKEN': '0xF41A5Cc7a61519B08056176d7B4b87AB34dF55AD',
    'v2 amm BPT_BAL_WETH_S_TOKEN': '0x6474d116476b8eDa1B21472a599Ff76A829AbCbb',
    'v2 amm BPT_BAL_WETH_ORACLE': '0x2e4e78936b100be6Ef85BCEf7FB25bC770B02B85',
    'v2 amm BPT_BAL_WETH_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm GUNI_DAI_USDC_A_TOKEN': '0xd145c6ae8931ed5Bca9b5f5B7dA5991F5aB63B5c',
    'v2 amm GUNI_DAI_USDC_V_TOKEN': '0x40533CC601Ec5b79B00D76348ADc0c81d93d926D',
    'v2 amm GUNI_DAI_USDC_S_TOKEN': '0x460Fd61bBDe7235C3F345901ad677854c9330c86',
    'v2 amm GUNI_DAI_USDC_ORACLE': '0x7843eA2E3e60b24cc12B56C5627Adc7F9f0749D6',
    'v2 amm GUNI_DAI_USDC_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',
    'v2 amm GUNI_USDC_USDT_A_TOKEN': '0xCa5DFDABBfFD58cfD49A9f78Ca52eC8e0591a3C5',
    'v2 amm GUNI_USDC_USDT_V_TOKEN': '0x0B7c7d9c5548A23D0455d1edeC541cc2AD955a9d',
    'v2 amm GUNI_USDC_USDT_S_TOKEN': '0xFEaeCde9Eb0cd43FDE13427C6C7ef406780a8136',
    'v2 amm GUNI_USDC_USDT_ORACLE': '0x399e3bb2BBd49c570aa6edc6ac390E0D0aCbbD5e',
    'v2 amm GUNI_USDC_USDT_INTEREST_RATE_STRATEGY': '0x52E39422cd86a12a13773D86af5FdBF5665989aD',

    # https://github.com/aave/aave-address-book/blob/main/src/MiscEthereum.sol
    'AAVE_ECOSYSTEM_RESERVE_CONTROLLER': '0x3d569673dAa0575c936c7c67c4E6AedA69CC630C',
    'ECOSYSTEM_RESERVE': '0x25F2226B597E8F9514B3F68F00f494cF4f286491',
    'PROXY_ADMIN_LONG': '0x86C3FfeE349A7cFf7cA88C449717B1b133bfb517',
    'AAVE_SWAPPER': '0x3ea64b1C0194524b48F9118462C8E9cd61a243c7',
    'AAVE_POL_ETH_BRIDGE': '0x1C2BA5b8ab8e795fF44387ba6d251fa65AD20b36',
    'PARASWAP_FEE_CLAIMER': '0x9abf798f5314BFd793A9E57A654BEd35af4A1D60',
    'TRANSPARENT_PROXY_FACTORY': '0xB4e496f70602fE2AC6Ae511D028BA4D194773B29',
    'PROXY_ADMIN': '0xD3cF979e676265e4f6379749DECe4708B9A22476',
    'CREATE_3_FACTORY': '0xcc3C54B95f3f1867A43009B80ed4DD930E3cE2F7',
#     'PROTOCOL_GUARDIAN': '0xCA76Ebd8617a03126B6FB84F9b1c1A0fB71C2633',  # a gnosis safe
    'AAVE_MERKLE_DISTRIBUTOR': '0xa88c6D90eAe942291325f9ae3c66f3563B93FE10',
    'GHO_TOKEN': '0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f',
    'GHO_FLASHMINTER_FACILITATOR': '0xb639D208Bcf0589D54FaC24E655C79EC529762B8',
    'GSM_REGISTRY': '0x0d8eFfC11dF3F229AA1EA0509BC9DFa632A13578',
    'GSM_USDC': '0x0d8eFfC11dF3F229AA1EA0509BC9DFa632A13578',
    'GSM_USDT': '0x686F8D21520f4ecEc7ba577be08354F4d1EB8262',

    #gov  https://github.com/aave/aave-address-book/blob/main/src/GovernanceV3Ethereum.sol
    'gov CROSS_CHAIN_CONTROLLER': '0xEd42a7D8559a463722Ca4beD50E0Cc05a386b0e1',
    'gov GOVERNANCE': '0x9AEE0B04504CeF83A65AC3f0e838D0593BCb2BC7',
    'gov PAYLOADS_CONTROLLER': '0xdAbad81aF85554E9ae636395611C58F7eC1aAEc5',
    'gov VOTING_MACHINE': '0x617332a777780F546261247F621051d0b98975Eb',
    'gov VOTING_PORTAL_ETH_ETH': '0xf23f7De3AC42F22eBDA17e64DC4f51FB66b8E21f',
    'gov VOTING_PORTAL_ETH_AVAX': '0x33aCEf7365809218485873B7d0d67FeE411B5D79',
    'gov VOTING_PORTAL_ETH_POL': '0x9b24C168d6A76b5459B1d47071a54962a4df36c3',
    'gov PC_DATA_HELPER': '0xE3B770Dc4ae3f8bECaB3Ed12dE692c741603e16A',
    'gov GOV_DATA_HELPER': '0x971c82c8316aD611904F95616c21ce90837f1856',
    'gov VM_DATA_HELPER': '0x77976B51569896523EE215962Ee91ff236Fa50E8',
    'gov META_DELEGATE_HELPER': '0x94363B11b37BC3ffe43AB09cff5A010352FE85dC',
    'gov EMERGENCY_REGISTRY': '0x73C6Fb358dDA8e84D50e98A98F7c0dF32e15C7e9',
    'gov GOVERNANCE_POWER_STRATEGY': '0xa198Fac58E02A5C5F8F7e877895d50cFa9ad1E04',
    'gov EXECUTOR_LVL_1': '0x5300A1a15135EA4dc7aD5a167152C01EFc9b192A',
    'gov EXECUTOR_LVL_2': '0x17Dd33Ed0e3dD2a80E37489B8A63063161BE6957',
    'gov VOTING_STRATEGY': '0x5642A5A5Ec284B4145563aBF319620204aCCA7f4',
    'gov DATA_WAREHOUSE': '0x1699FE9CaDC8a0b6c93E06B62Ab4592a0fFEcF61',
    
    # https://github.com/aave/aave-address-book/blob/main/src/AaveV3Ethereum.sol
    'v3 POOL_ADDRESSES_PROVIDER': '0x2f39d218133AFaB8F2B819B1066c7E434Ad94E9e',
    'v3 POOL': '0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2',
    'v3 POOL_CONFIGURATOR': '0x64b761D848206f447Fe2dd461b0c635Ec39EbB27',
    'v3 ORACLE': '0x54586bE62E3c3580375aE3723C145253060Ca0C2',
#     'v3 PRICE_ORACLE_SENTINEL': '0x0000000000000000000000000000000000000000',
    'v3 AAVE_PROTOCOL_DATA_PROVIDER': '0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3',
    'v3 ACL_MANAGER': '0xc2aaCf6553D20d1e9d78E365AAba8032af9c85b0',
    'v3 ACL_ADMIN': '0x5300A1a15135EA4dc7aD5a167152C01EFc9b192A',
    'v3 COLLECTOR': '0x464C71f6c2F760DdA6093dCB91C24c39e5d6e18c',
    'v3 DEFAULT_INCENTIVES_CONTROLLER': '0x8164Cc65827dcFe994AB23944CBC90e0aa80bFcb',
    'v3 DEFAULT_A_TOKEN_IMPL_REV_1': '0x7EfFD7b47Bfd17e52fB7559d3f924201b9DbfF3d',
    'v3 DEFAULT_VARIABLE_DEBT_TOKEN_IMPL_REV_1': '0xaC725CB59D16C81061BDeA61041a8A5e73DA9EC6',
    'v3 DEFAULT_STABLE_DEBT_TOKEN_IMPL_REV_1': '0x15C5620dfFaC7c7366EED66C20Ad222DDbB1eD57',
    'v3 EMISSION_MANAGER': '0x223d844fc4B006D67c0cDbd39371A9F73f69d974',
    'v3 CAPS_PLUS_RISK_STEWARD': '0x82dcCF206Ae2Ab46E2099e663F70DeE77caE7778',
    'v3 FREEZING_STEWARD': '0x2eE68ACb6A1319de1b49DC139894644E424fefD6',
    'v3 DEBT_SWAP_ADAPTER': '0x8761e0370f94f68Db8EaA731f4fC581f6AD0Bd68',
    'v3 DELEGATION_AWARE_A_TOKEN_IMPL_REV_1': '0x21714092D90c7265F52fdfDae068EC11a23C6248',
    'v3 CONFIG_ENGINE': '0xA3e44d830440dF5098520F62Ebec285B1198c51E',
    'v3 POOL_ADDRESSES_PROVIDER_REGISTRY': '0xbaA999AC55EAce41CcAE355c77809e68Bb345170',
    'v3 RATES_FACTORY': '0xcC47c4Fe1F7f29ff31A8b62197023aC8553C7896',
    'v3 REPAY_WITH_COLLATERAL_ADAPTER': '0x02e7B8511831B1b02d9018215a0f8f500Ea5c6B3',
    'v3 STATIC_A_TOKEN_FACTORY': '0x411D79b8cC43384FDE66CaBf9b6a17180c842511',
    'v3 SWAP_COLLATERAL_ADAPTER': '0xADC0A53095A0af87F3aa29FE0715B5c28016364e',
    'v3 UI_GHO_DATA_PROVIDER': '0x379c1EDD1A41218bdbFf960a9d5AD2818Bf61aE8',
    'v3 UI_INCENTIVE_DATA_PROVIDER': '0x162A7AC02f547ad796CA549f757e2b8d1D9b10a6',
    'v3 UI_POOL_DATA_PROVIDER': '0x91c0eA31b49B69Ea18607702c5d9aC360bf3dE7d',
    'v3 WALLET_BALANCE_PROVIDER': '0xC7be5307ba715ce89b152f3Df0658295b3dbA8E2',
    'v3 WETH_GATEWAY': '0x893411580e590D62dDBca8a703d61Cc4A8c7b2b9',
    'v3 WITHDRAW_SWAP_ADAPTER': '0x78F8Bd884C3D738B74B420540659c82f392820e0',
    'v3 SAVINGS_DAI_TOKEN_WRAPPER': '0xE28E2c8d240dd5eBd0adcab86fbD79df7a052034',
    'v3 WETH_A_TOKEN': '0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8',
    'v3 WETH_V_TOKEN': '0xeA51d7853EEFb32b6ee06b1C12E6dcCA88Be0fFE',
    'v3 WETH_S_TOKEN': '0x102633152313C81cD80419b6EcF66d14Ad68949A',
    'v3 WETH_INTEREST_RATE_STRATEGY': '0x06B1Ec378618EA736a65395eA5CAB69A2410493B',
    'v3 WETH_STATA_TOKEN': '0x252231882FB38481497f3C767469106297c8d93b',
    'v3 wstETH_A_TOKEN': '0x0B925eD163218f6662a35e0f0371Ac234f9E9371',
    'v3 wstETH_V_TOKEN': '0xC96113eED8cAB59cD8A66813bCB0cEb29F06D2e4',
    'v3 wstETH_S_TOKEN': '0x39739943199c0fBFe9E5f1B5B160cd73a64CB85D',
    'v3 wstETH_INTEREST_RATE_STRATEGY': '0x7b8Fa4540246554e77FCFf140f9114de00F8bB8D',
    'v3 wstETH_STATA_TOKEN': '0x322AA5F5Be95644d6c36544B6c5061F072D16DF5',
    'v3 WBTC_A_TOKEN': '0x5Ee5bf7ae06D1Be5997A1A72006FE6C607eC6DE8',
    'v3 WBTC_V_TOKEN': '0x40aAbEf1aa8f0eEc637E0E7d92fbfFB2F26A8b7B',
    'v3 WBTC_S_TOKEN': '0xA1773F1ccF6DB192Ad8FE826D15fe1d328B03284',
    'v3 WBTC_INTEREST_RATE_STRATEGY': '0x07Fa3744FeC271F80c2EA97679823F65c13CCDf4',
    'v3 USDC_A_TOKEN': '0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c',
    'v3 USDC_V_TOKEN': '0x72E95b8931767C79bA4EeE721354d6E99a61D004',
    'v3 USDC_S_TOKEN': '0xB0fe3D292f4bd50De902Ba5bDF120Ad66E9d7a39',
    'v3 USDC_INTEREST_RATE_STRATEGY': '0x642a8DAcC59b73491Dcaa3BCeF046D660901fCc1',
    'v3 USDC_STATA_TOKEN': '0x73edDFa87C71ADdC275c2b9890f5c3a8480bC9E6',
    'v3 DAI_A_TOKEN': '0x018008bfb33d285247A21d44E50697654f754e63',
    'v3 DAI_V_TOKEN': '0xcF8d0c70c850859266f5C338b38F9D663181C314',
    'v3 DAI_S_TOKEN': '0x413AdaC9E2Ef8683ADf5DDAEce8f19613d60D1bb',
    'v3 DAI_INTEREST_RATE_STRATEGY': '0x2402C25e7E45b1466c53Ef7766AAd878A4CbC237',
    'v3 DAI_STATA_TOKEN': '0xaf270C38fF895EA3f95Ed488CEACe2386F038249',
    'v3 LINK_A_TOKEN': '0x5E8C8A7243651DB1384C0dDfDbE39761E8e7E51a',
    'v3 LINK_V_TOKEN': '0x4228F8895C7dDA20227F6a5c6751b8Ebf19a6ba8',
    'v3 LINK_S_TOKEN': '0x63B1129ca97D2b9F97f45670787Ac12a9dF1110a',
    'v3 LINK_INTEREST_RATE_STRATEGY': '0x24701A6368Ff6D2874d6b8cDadd461552B8A5283',
    'v3 AAVE_A_TOKEN': '0xA700b4eB416Be35b2911fd5Dee80678ff64fF6C9',
    'v3 AAVE_V_TOKEN': '0xBae535520Abd9f8C85E58929e0006A2c8B372F74',
    'v3 AAVE_S_TOKEN': '0x268497bF083388B1504270d0E717222d3A87D6F2',
    'v3 AAVE_INTEREST_RATE_STRATEGY': '0x24701A6368Ff6D2874d6b8cDadd461552B8A5283',
    'v3 cbETH_A_TOKEN': '0x977b6fc5dE62598B08C85AC8Cf2b745874E8b78c',
    'v3 cbETH_V_TOKEN': '0x0c91bcA95b5FE69164cE583A2ec9429A569798Ed',
    'v3 cbETH_S_TOKEN': '0x82bE6012cea6D147B968eBAea5ceEcF6A5b4F493',
    'v3 cbETH_INTEREST_RATE_STRATEGY': '0x24701A6368Ff6D2874d6b8cDadd461552B8A5283',
    'v3 USDT_A_TOKEN': '0x23878914EFE38d27C4D67Ab83ed1b93A74D4086a',
    'v3 USDT_V_TOKEN': '0x6df1C1E379bC5a00a7b4C6e67A203333772f45A8',
    'v3 USDT_S_TOKEN': '0x822Fa72Df1F229C3900f5AD6C3Fa2C424D691622',
    'v3 USDT_INTEREST_RATE_STRATEGY': '0xc77576b02D74BBF9CdC26F3B86FD09d134416df2',
    'v3 USDT_STATA_TOKEN': '0x862c57d48becB45583AEbA3f489696D22466Ca1b',
    'v3 rETH_A_TOKEN': '0xCc9EE9483f662091a1de4795249E24aC0aC2630f',
    'v3 rETH_V_TOKEN': '0xae8593DD575FE29A9745056aA91C4b746eee62C8',
    'v3 rETH_S_TOKEN': '0x1d1906f909CAe494c7441604DAfDDDbD0485A925',
    'v3 rETH_INTEREST_RATE_STRATEGY': '0x24701A6368Ff6D2874d6b8cDadd461552B8A5283',
    'v3 LUSD_A_TOKEN': '0x3Fe6a295459FAe07DF8A0ceCC36F37160FE86AA9',
    'v3 LUSD_V_TOKEN': '0x33652e48e4B74D18520f11BfE58Edd2ED2cEc5A2',
    'v3 LUSD_S_TOKEN': '0x37A6B708FDB1483C231961b9a7F145261E815fc3',
    'v3 LUSD_INTEREST_RATE_STRATEGY': '0xC0B875907514131C2Fd43f0FBf59EdaB84C7e260',
    'v3 LUSD_STATA_TOKEN': '0xDBf5E36569798D1E39eE9d7B1c61A7409a74F23A',
    'v3 CRV_A_TOKEN': '0x7B95Ec873268a6BFC6427e7a28e396Db9D0ebc65',
    'v3 CRV_V_TOKEN': '0x1b7D3F4b3c032a5AE656e30eeA4e8E1Ba376068F',
    'v3 CRV_S_TOKEN': '0x90D9CD005E553111EB8C9c31Abe9706a186b6048',
    'v3 CRV_INTEREST_RATE_STRATEGY': '0x76884cAFeCf1f7d4146DA6C4053B18B76bf6ED14',
    'v3 MKR_A_TOKEN': '0x8A458A9dc9048e005d22849F470891b840296619',
    'v3 MKR_V_TOKEN': '0x6Efc73E54E41b27d2134fF9f98F15550f30DF9B1',
    'v3 MKR_S_TOKEN': '0x0496372BE7e426D28E89DEBF01f19F014d5938bE',
    'v3 MKR_INTEREST_RATE_STRATEGY': '0x27eFE5db315b71753b2a38ED3d5dd7E9362ba93F',
    'v3 SNX_A_TOKEN': '0xC7B4c17861357B8ABB91F25581E7263E08DCB59c',
    'v3 SNX_V_TOKEN': '0x8d0de040e8aAd872eC3c33A3776dE9152D3c34ca',
    'v3 SNX_S_TOKEN': '0x478E1ec1A2BeEd94c1407c951E4B9e22d53b2501',
    'v3 SNX_INTEREST_RATE_STRATEGY': '0xA6459195d60A797D278f58Ffbd2BA62Fb3F7FA1E',
    'v3 BAL_A_TOKEN': '0x2516E7B3F76294e03C42AA4c5b5b4DCE9C436fB8',
    'v3 BAL_V_TOKEN': '0x3D3efceb4Ff0966D34d9545D3A2fa2dcdBf451f2',
    'v3 BAL_S_TOKEN': '0xB368d45aaAa07ee2c6275Cb320D140b22dE43CDD',
    'v3 BAL_INTEREST_RATE_STRATEGY': '0xd9d85499449f26d2A2c240defd75314f23920089',
    'v3 UNI_A_TOKEN': '0xF6D2224916DDFbbab6e6bd0D1B7034f4Ae0CaB18',
    'v3 UNI_V_TOKEN': '0xF64178Ebd2E2719F2B1233bCb5Ef6DB4bCc4d09a',
    'v3 UNI_S_TOKEN': '0x2FEc76324A0463c46f32e74A86D1cf94C02158DC',
    'v3 UNI_INTEREST_RATE_STRATEGY': '0x27eFE5db315b71753b2a38ED3d5dd7E9362ba93F',
    'v3 LDO_A_TOKEN': '0x9A44fd41566876A39655f74971a3A6eA0a17a454',
    'v3 LDO_V_TOKEN': '0xc30808705C01289A3D306ca9CAB081Ba9114eC82',
    'v3 LDO_S_TOKEN': '0xa0a5bF5781Aeb548db9d4226363B9e89287C5FD2',
    'v3 LDO_INTEREST_RATE_STRATEGY': '0x27eFE5db315b71753b2a38ED3d5dd7E9362ba93F',
    'v3 ENS_A_TOKEN': '0x545bD6c032eFdde65A377A6719DEF2796C8E0f2e',
    'v3 ENS_V_TOKEN': '0xd180D7fdD4092f07428eFE801E17BC03576b3192',
    'v3 ENS_S_TOKEN': '0x7617d02E311CdE347A0cb45BB7DF2926BBaf5347',
    'v3 ENS_INTEREST_RATE_STRATEGY': '0xf6733B9842883BFE0e0a940eA2F572676af31bde',
    'v3 ONE_INCH_A_TOKEN': '0x71Aef7b30728b9BB371578f36c5A1f1502a5723e',
    'v3 ONE_INCH_V_TOKEN': '0xA38fCa8c6Bf9BdA52E76EB78f08CaA3BE7c5A970',
    'v3 ONE_INCH_S_TOKEN': '0x4b62bFAff61AB3985798e5202D2d167F567D0BCD',
    'v3 ONE_INCH_INTEREST_RATE_STRATEGY': '0xf6733B9842883BFE0e0a940eA2F572676af31bde',
    'v3 FRAX_A_TOKEN': '0xd4e245848d6E1220DBE62e155d89fa327E43CB06',
    'v3 FRAX_V_TOKEN': '0x88B8358F5BC87c2D7E116cCA5b65A9eEb2c5EA3F',
    'v3 FRAX_S_TOKEN': '0x219640546c0DFDDCb9ab3bcdA89B324e0a376367',
    'v3 FRAX_INTEREST_RATE_STRATEGY': '0x2402C25e7E45b1466c53Ef7766AAd878A4CbC237',
    'v3 FRAX_STATA_TOKEN': '0xEE66abD4D0f9908A48E08AE354B0f425De3e237E',
    'v3 GHO_A_TOKEN': '0x00907f9921424583e7ffBfEdf84F92B7B2Be4977',
    'v3 GHO_V_TOKEN': '0x786dBff3f1292ae8F92ea68Cf93c30b34B1ed04B',
    'v3 GHO_S_TOKEN': '0x3f3DF7266dA30102344A813F1a3D07f5F041B5AC',
    'v3 GHO_INTEREST_RATE_STRATEGY': '0x00524e8E4C5FD2b8D8aa1226fA16b39Cad69B8A0',
    'v3 RPL_A_TOKEN': '0xB76CF92076adBF1D9C39294FA8e7A67579FDe357',
    'v3 RPL_V_TOKEN': '0x8988ECA19D502fd8b9CCd03fA3bD20a6f599bc2A',
    'v3 RPL_S_TOKEN': '0x41e330fd8F7eA31E2e8F02cC0C9392D1403597B4',
    'v3 RPL_INTEREST_RATE_STRATEGY': '0xD87974E8ED49AB16d5053ba793F4e17078Be0426',
    'v3 sDAI_A_TOKEN': '0x4C612E3B15b96Ff9A6faED838F8d07d479a8dD4c',
    'v3 sDAI_V_TOKEN': '0x8Db9D35e117d8b93C6Ca9b644b25BaD5d9908141',
    'v3 sDAI_S_TOKEN': '0x48Bc45f084988bC01933EA93EeFfEBC0416534f6',
    'v3 sDAI_INTEREST_RATE_STRATEGY': '0xdef8F50155A6cf21181E29E400E8CffAE2d50968',
    'v3 STG_A_TOKEN': '0x1bA9843bD4327c6c77011406dE5fA8749F7E3479',
    'v3 STG_V_TOKEN': '0x655568bDd6168325EC7e58Bf39b21A856F906Dc2',
    'v3 STG_S_TOKEN': '0xc3115D0660b93AeF10F298886ae22E3Dd477E482',
    'v3 STG_INTEREST_RATE_STRATEGY': '0x27eFE5db315b71753b2a38ED3d5dd7E9362ba93F',
    'v3 KNC_A_TOKEN': '0x5b502e3796385E1e9755d7043B9C945C3aCCeC9C',
    'v3 KNC_V_TOKEN': '0x253127Ffc04981cEA8932F406710661c2f2c3fD2',
    'v3 KNC_S_TOKEN': '0xdfEE0C9eA1309cB9611F33972E72a72166fcF548',
    'v3 KNC_INTEREST_RATE_STRATEGY': '0xf6733B9842883BFE0e0a940eA2F572676af31bde',
    'v3 FXS_A_TOKEN': '0x82F9c5ad306BBa1AD0De49bB5FA6F01bf61085ef',
    'v3 FXS_V_TOKEN': '0x68e9f0aD4e6f8F5DB70F6923d4d6d5b225B83b16',
    'v3 FXS_S_TOKEN': '0x61dFd349140C239d3B61fEe203Efc811b518a317',
    'v3 FXS_INTEREST_RATE_STRATEGY': '0xf6733B9842883BFE0e0a940eA2F572676af31bde',
    'v3 crvUSD_A_TOKEN': '0xb82fa9f31612989525992FCfBB09AB22Eff5c85A',
    'v3 crvUSD_V_TOKEN': '0x028f7886F3e937f8479efaD64f31B3fE1119857a',
    'v3 crvUSD_S_TOKEN': '0xb55C604075D79486b8A329c396Fc711Be54B5330',
    'v3 crvUSD_INTEREST_RATE_STRATEGY': '0x44CaDF6E49895640D9De85ac01d97D44429Ad0A4',
    'v3 crvUSD_STATA_TOKEN': '0x848107491E029AFDe0AC543779c7790382f15929',
}

aave_addresses = pl.DataFrame(
    [
        [label, address]
        for label, address in aave_addresses.items()
    ],
    orient='row',
    schema=['name', 'address'],
).select(
    address='address',
    label=pl.lit('aave'),
    name='name',
).group_by('address').agg(
    label=pl.first('label'),
    name=pl.first('name'),
)

aave_addresses

In [ ]:
aave_addresses = aave_addresses.with_columns(
    aave_addresses['address'].evm.hex_to_binary()
)

In [ ]:
aave_addresses.write_parquet(    
    get_label_path('aave', min_block, max_block)
)

# maker

In [ ]:
raw_maker_addresses = {
    "CHANGELOG": "0xdA0Ab1e0017DEbCd72Be8599041a2aa3bA7e740F",
    "BAT": "0x0D8775F648430679A709E98d2b0Cb6250d2887EF",
    "CDP_MANAGER": "0x5ef30b9986345249bc32d8928B7ee64DE9435E39",
    "COMP": "0xc00e94Cb662C3520282E6f5717214004A7f26888",
    "DSR_MANAGER": "0x373238337Bfe1146fb49989fc222523f83081dDb",
    "ETH": "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2",
    "LERP_FAB": "0x9175561733D138326FDeA86CdFdF53e92b588276",
    "RWA015_A_INPUT_CONDUIT_JAR_USDC": "0xB9373C557f3aE8cDdD068c1644ED226CfB18A997",
    "GET_CDPS": "0x36a724Bd100c39f0Ea4D3A20F7097eE01A8Ff573",
    "GOV_GUARD": "0x6eEB68B2C7A918f36B78E2DB80dcF279236DDFb8",
    "ILK_REGISTRY": "0x5a464C28D19848f44199D003BeF5ecc87d090F87",
    "KNC": "0xdd974D5C2e2928deA5F71b9825b8b646686BD200",
    "LINK": "0x514910771AF9Ca656af840dff83E8264EcF986CA",
    "LRC": "0xBBbbCA6A901c926F240b89EacB641d8Aec7AEafD",
    "MANA": "0x0F5D2fB29fb7d3CFeE444a200298f468908cC942",
    "MCD_ADM": "0x0a3f6849f78076aefaDf113F5BED87720274dDC0",
    "RWA015_A_INPUT_CONDUIT_JAR_GUSD": "0x13C31b41E671401c7BC2bbd44eF33B6E9eaa1E7F",
    "MCD_DAI": "0x6B175474E89094C44Da98b954EedeAC495271d0F",
    "MCD_DEPLOY": "0xbaa65281c2FA2baAcb2cb550BA051525A480D3F4",
    "MCD_END": "0x0e2e8F1D1326A4B9633D96222Ce399c708B19c28",
    "MCD_ESM": "0x09e05fF6142F2f9de8B6B65855A1d56B6cfE4c58",
    "MCD_FLAP": "0x0c10Ae443cCB4604435Ba63DA80CCc63311615Bc",
    "MCD_CLIP_CALC_BAT_A": "0x2e118153D304a0d9C5838D5FCb70CEfCbEc81DC2",
    "MCD_CLIP_CALC_COMP_A": "0x1f546560EAa70985d962f1562B65D4B182341a63",
    "MCD_CLIP_CALC_ETH_A": "0x7d9f92DAa9254Bbd1f479DBE5058f74C2381A898",
    "MCD_CLIP_CALC_KNC_A": "0x82c41e2ADE28C066a5D3A1E3f5B444a4075C1584",
    "MCD_CLIP_CALC_LINK_A": "0x7B1696677107E48B152e9Bf400293e98B7D86Eb1",
    "MCD_CLIP_CALC_LRC_A": "0x6856CCA4c881CAf29B6563bA046C7Bb73121fb9d",
    "MCD_CLIP_CALC_MANA_A": "0xABbCd14FeDbb2D39038327055D9e615e178Fd64D",
    "MCD_CLIP_CALC_PAXUSD_A": "0xA2a4aeFEd398661B0a873d3782DA121c194a0201",
    "MCD_CLIP_CALC_TUSD_A": "0x9B207AfAAAD1ae300Ea659e71306a7Bd6D81C160",
    "MCD_CLIP_CALC_USDC_A": "0x00A0F90666c6Cd3E615cF8459A47e89A08817602",
    "MCD_CLIP_CALC_USDC_B": "0xD6FE411284b92d309F79e502Dd905D7A3b02F561",
    "MCD_CLIP_CALC_USDT_A": "0x1Cf3DE6D570291CDB88229E70037d1705d5be748",
    "MCD_CLIP_CALC_WBTC_A": "0x5f4CEa97ca1030C6Bd38429c8a0De7Cd4981C70A",
    "MCD_CLIP_CALC_ZRX_A": "0xebe5e9D77b9DBBA8907A197f4c2aB00A81fb0C4e",
    "MCD_FLOP": "0xA41B6EF151E06da0e34B009B86E828308986736D",
    "MCD_GOV": "0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2",
    "MCD_GOV_ACTIONS": "0x4F5f0933158569c026d617337614d00Ee6589B6E",
    "MCD_JOIN_BAT_A": "0x3D0B1912B66114d4096F48A8CEe3A56C231772cA",
    "MCD_JOIN_COMP_A": "0xBEa7cDfB4b49EC154Ae1c0D731E4DC773A3265aA",
    "MCD_JOIN_DAI": "0x9759A6Ac90977b93B58547b4A71c78317f391A28",
    "MCD_JOIN_ETH_A": "0x2F0b23f53734252Bda2277357e97e1517d6B042A",
    "MCD_JOIN_KNC_A": "0x475F1a89C1ED844A08E8f6C50A00228b5E59E4A9",
    "MCD_JOIN_LINK_A": "0xdFccAf8fDbD2F4805C174f856a317765B49E4a50",
    "MCD_JOIN_LRC_A": "0x6C186404A7A238D3d6027C0299D1822c1cf5d8f1",
    "MCD_JOIN_MANA_A": "0xA6EA3b9C04b8a38Ff5e224E7c3D6937ca44C0ef9",
    "MCD_JOIN_PAXUSD_A": "0x7e62B7E279DFC78DEB656E34D6a435cC08a44666",
    "MCD_JOIN_TUSD_A": "0x4454aF7C8bb9463203b66C816220D41ED7837f44",
    "MCD_JOIN_USDC_A": "0xA191e578a6736167326d05c119CE0c90849E84B7",
    "MCD_JOIN_USDC_B": "0x2600004fd1585f7270756DDc88aD9cfA10dD0428",
    "MCD_JOIN_USDT_A": "0x0Ac6A1D74E84C2dF9063bDDc31699FF2a2BB22A2",
    "MCD_JOIN_WBTC_A": "0xBF72Da2Bd84c5170618Fbe5914B0ECA9638d5eb5",
    "MCD_JOIN_ZRX_A": "0xc7e8Cd72BDEe38865b4F5615956eF47ce1a7e5D0",
    "MCD_JUG": "0x19c0976f590D67707E62397C87829d896Dc0f1F1",
    "MCD_PAUSE": "0xbE286431454714F511008713973d3B053A2d38f3",
    "MCD_PAUSE_PROXY": "0xBE8E3e3618f7474F8cB1d074A26afFef007E98FB",
    "MCD_POT": "0x197E90f9FAD81970bA7976f33CbD77088E5D7cf7",
    "MCD_SPOT": "0x65C79fcB50Ca1594B025960e539eD7A9a6D434A3",
    "MCD_VAT": "0x35D1b3F3D7966A1DFe207aa4514C12a259A0492B",
    "MCD_VOW": "0xA950524441892A31ebddF91d3cEEFa04Bf454466",
    "MULTICALL": "0x5e227AD1969Ea493B43F840cfF78d08a6fc17796",
    "OSM_MOM": "0x76416A4d5190d071bfed309861527431304aA14f",
    "PAXUSD": "0x8E870D67F660D95d5be530380D0eC0bd388289E1",
    "PIP_BAT": "0xB4eb54AF9Cc7882DF0121d26c5b97E802915ABe6",
    "PIP_COMP": "0xBED0879953E633135a48a157718Aa791AC0108E4",
    "PIP_ETH": "0x81FE72B5A8d1A857d176C3E7d5Bd2679A9B85763",
    "PIP_KNC": "0xf36B79BD4C0904A5F350F1e4f776B81208c13069",
    "PIP_LINK": "0x9B0C694C6939b5EA9584e9b61C7815E8d97D9cC7",
    "PIP_LRC": "0x9eb923339c24c40Bef2f4AF4961742AA7C23EF3a",
    "PIP_MANA": "0x8067259EA630601f319FccE477977E55C6078C13",
    "PIP_PAXUSD": "0x043B963E1B2214eC90046167Ea29C2c8bDD7c0eC",
    "PIP_TUSD": "0xeE13831ca96d191B688A670D47173694ba98f1e5",
    "PIP_USDC": "0x77b68899b99b686F415d074278a9a16b336085A0",
    "PIP_USDT": "0x7a5918670B0C390aD25f7beE908c1ACc2d314A3C",
    "PIP_WBTC": "0xf185d0682d50819263941e5f4EacC763CC5C6C42",
    "PIP_ZRX": "0x7382c066801E7Acb2299aC8562847B9883f5CD3c",
    "PROXY_ACTIONS": "0x82ecD135Dce65Fbc6DbdD0e4237E0AF93FFD5038",
    "PROXY_ACTIONS_DSR": "0x07ee93aEEa0a36FfF2A9B95dd22Bd6049EE54f26",
    "PROXY_ACTIONS_END": "0x7AfF9FC9faD225e3c88cDA06BC56d8Aca774bC57",
    "PROXY_DEPLOYER": "0x1b93556AB8dcCEF01Cd7823C617a6d340f53Fb58",
    "PROXY_FACTORY": "0xA26e15C895EFc0616177B7c1e7270A4C7D51C997",
    "PROXY_PAUSE_ACTIONS": "0x6bda13D43B7EDd6CAfE1f70fB98b5d40f61A1370",
    "PROXY_REGISTRY": "0x4678f0a6958e4D2Bc4F1BAF7Bc52E8F3564f3fE4",
    "TUSD": "0x0000000000085d4780B73119b644AE5ecd22b376",
    "USDC": "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48",
    "USDT": "0xdAC17F958D2ee523a2206206994597C13D831ec7",
    "WBTC": "0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599",
    "ZRX": "0xE41d2489571d322189246DaFA5ebDe1F4699F498",
    "MCD_JOIN_ETH_B": "0x08638eF1A205bE6762A8b935F5da9b700Cf7322c",
    "MCD_CLIP_CALC_ETH_B": "0x19E26067c4a69B9534adf97ED8f986c49179dE18",
    "BAL": "0xba100000625a3754423978a60c9317c58a424e3D",
    "PIP_BAL": "0x3ff860c0F28D69F392543A16A397D0dAe85D16dE",
    "MCD_JOIN_BAL_A": "0x4a03Aa7fb3973d8f0221B466EefB53D0aC195f55",
    "MCD_CLIP_CALC_BAL_A": "0x79564a41508DA86721eDaDac07A590b5A51B2c01",
    "YFI": "0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e",
    "PIP_YFI": "0x5F122465bCf86F45922036970Be6DD7F58820214",
    "MCD_JOIN_YFI_A": "0x3ff33d9162aD47660083D7DC4bC02Fb231c81677",
    "MCD_CLIP_CALC_YFI_A": "0x1f206d7916Fd3B1b5B0Ce53d5Cab11FCebc124DA",
    "RWA015_A_INPUT_CONDUIT_URN_USDC": "0xe08cb5E24862eA86328295D5E5c08972203C20D8",
    "GUSD": "0x056Fd409E1d7A124BD7017459dFEa2F387b6d5Cd",
    "MCD_JOIN_GUSD_A": "0xe29A14bcDeA40d83675aa43B72dF07f649738C8b",
    "MCD_CLIP_CALC_GUSD_A": "0xC287E4e9017259f3b21C86A0Ef7840243eC3f4d6",
    "PIP_GUSD": "0xf45Ae69CcA1b9B043dAE2C83A5B65Bc605BEc5F5",
    "VOTE_PROXY_FACTORY": "0x6FCD258af181B3221073A96dD90D1f7AE7eEc408",
    "UNI": "0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984",
    "MCD_JOIN_UNI_A": "0x3BC3A58b4FC1CbE7e98bB4aB7c99535e8bA9b8F1",
    "MCD_CLIP_CALC_UNI_A": "0xeA7FE6610e6708E2AFFA202948cA19ace3F580AE",
    "PIP_UNI": "0xf363c7e351C96b910b92b45d34190650df4aE8e7",
    "RENBTC": "0xEB4C2781e4ebA804CE9a9803C67d0893436bB27D",
    "MCD_JOIN_RENBTC_A": "0xFD5608515A47C37afbA68960c1916b79af9491D0",
    "MCD_CLIP_CALC_RENBTC_A": "0xcC89F368aad8D424d3e759c1525065e56019a0F4",
    "PIP_RENBTC": "0xf185d0682d50819263941e5f4EacC763CC5C6C42",
    "MCD_IAM_AUTO_LINE": "0xC7Bdd1F2B16447dcf3dE045C4a039A60EC2f0ba3",
    "AAVE": "0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9",
    "MCD_JOIN_AAVE_A": "0x24e459F61cEAa7b1cE70Dbaea938940A7c5aD46e",
    "MCD_CLIP_CALC_AAVE_A": "0x76024a8EfFCFE270e089964a562Ece6ea5f3a14C",
    "PIP_AAVE": "0x8Df8f06DC2dE0434db40dcBb32a82A104218754c",
    "UNIV2DAIETH": "0xA478c2975Ab1Ea89e8196811F51A7B7Ade33eB11",
    "MCD_JOIN_UNIV2DAIETH_A": "0x2502F65D77cA13f183850b5f9272270454094A08",
    "MCD_CLIP_CALC_UNIV2DAIETH_A": "0xf738C272D648Cc4565EaFb43c0C5B35BbA3bf29d",
    "PIP_UNIV2DAIETH": "0xFc8137E1a45BAF0030563EC4F0F851bd36a85b7D",
    "MCD_JOIN_PSM_USDC_A": "0x0A59649758aa4d66E25f08Dd01271e891fe52199",
    "MCD_CLIP_CALC_PSM_USDC_A": "0xbeE028b5Fa9eb0aDAC5eeF7E5B13383172b91A4E",
    "MCD_PSM_USDC_A": "0x89B78CfA322F6C5dE0aBcEecab66Aee45393cC5A",
    "UNIV2WBTCETH": "0xBb2b8038a1640196FbE3e38816F3e67Cba72D940",
    "MCD_JOIN_UNIV2WBTCETH_A": "0xDc26C9b7a8fe4F5dF648E314eC3E6Dc3694e6Dd2",
    "MCD_CLIP_CALC_UNIV2WBTCETH_A": "0xC94ee71e909DbE08d63aA9e6EFbc9976751601B4",
    "PIP_UNIV2WBTCETH": "0x8400D2EDb8B97f780356Ef602b1BdBc082c2aD07",
    "UNIV2USDCETH": "0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc",
    "MCD_JOIN_UNIV2USDCETH_A": "0x03Ae53B33FeeAc1222C3f372f32D37Ba95f0F099",
    "MCD_CLIP_CALC_UNIV2USDCETH_A": "0x022ff40643e8b94C43f0a1E54f51EF6D070AcbC4",
    "PIP_UNIV2USDCETH": "0xf751f24DD9cfAd885984D1bA68860F558D21E52A",
    "UNIV2DAIUSDC": "0xAE461cA67B15dc8dc81CE7615e0320dA1A9aB8D5",
    "MCD_JOIN_UNIV2DAIUSDC_A": "0xA81598667AC561986b70ae11bBE2dd5348ed4327",
    "MCD_CLIP_CALC_UNIV2DAIUSDC_A": "0xbEF2ab2aA5CC780A03bccf22AD3320c8CF35af6A",
    "PIP_UNIV2DAIUSDC": "0x25D03C2C928ADE19ff9f4FFECc07d991d0df054B",
    "UNIV2ETHUSDT": "0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852",
    "MCD_JOIN_UNIV2ETHUSDT_A": "0x4aAD139a88D2dd5e7410b408593208523a3a891d",
    "MCD_CLIP_CALC_UNIV2ETHUSDT_A": "0xA475582E3D6Ec35091EaE81da3b423C1B27fa029",
    "PIP_UNIV2ETHUSDT": "0x5f6dD5B421B8d92c59dC6D907C9271b1DBFE3016",
    "UNIV2LINKETH": "0xa2107FA5B38d9bbd2C461D6EDf11B11A50F6b974",
    "MCD_JOIN_UNIV2LINKETH_A": "0xDae88bDe1FB38cF39B6A02b595930A3449e593A6",
    "MCD_CLIP_CALC_UNIV2LINKETH_A": "0x8aCeC2d937a4A4cAF42565aFbbb05ac242134F14",
    "PIP_UNIV2LINKETH": "0xd7d31e62AE5bfC3bfaa24Eda33e8c32D31a1746F",
    "UNIV2UNIETH": "0xd3d2E2692501A5c9Ca623199D38826e513033a17",
    "MCD_JOIN_UNIV2UNIETH_A": "0xf11a98339FE1CdE648e8D1463310CE3ccC3d7cC1",
    "MCD_CLIP_CALC_UNIV2UNIETH_A": "0xad609Ed16157014EF955C94553E40e94A09049f0",
    "PIP_UNIV2UNIETH": "0x8462A88f50122782Cc96108F476deDB12248f931",
    "UNIV2WBTCDAI": "0x231B7589426Ffe1b75405526fC32aC09D44364c4",
    "MCD_JOIN_UNIV2WBTCDAI_A": "0xD40798267795Cbf3aeEA8E9F8DCbdBA9b5281fcC",
    "MCD_CLIP_CALC_UNIV2WBTCDAI_A": "0x863AEa7D2c4BF2B5Aa191B057240b6Dc29F532eB",
    "PIP_UNIV2WBTCDAI": "0x5bB72127a196392cf4aC00Cf57aB278394d24e55",
    "UNIV2AAVEETH": "0xDFC14d2Af169B0D36C4EFF567Ada9b2E0CAE044f",
    "MCD_JOIN_UNIV2AAVEETH_A": "0x42AFd448Df7d96291551f1eFE1A590101afB1DfF",
    "MCD_CLIP_CALC_UNIV2AAVEETH_A": "0x5396e541E1F648EC03faf338389045F1D7691960",
    "PIP_UNIV2AAVEETH": "0x32d8416e8538Ac36272c44b0cd962cD7E0198489",
    "UNIV2DAIUSDT": "0xB20bd5D04BE54f870D5C0d3cA85d82b34B836405",
    "MCD_JOIN_UNIV2DAIUSDT_A": "0xAf034D882169328CAf43b823a4083dABC7EEE0F4",
    "MCD_CLIP_CALC_UNIV2DAIUSDT_A": "0x4E88cE740F6bEa31C2b14134F6C5eB2a63104fcF",
    "PIP_UNIV2DAIUSDT": "0x9A1CD705dc7ac64B50777BcEcA3529E58B1292F1",
    "RWA001": "0x10b2aA5D77Aa6484886d8e244f0686aB319a270d",
    "PIP_RWA001": "0x76A9f30B45F4ebFD60Ce8a1c6e963b1605f7cB6d",
    "MCD_JOIN_RWA001_A": "0x476b81c12Dc71EDfad1F64B9E07CaA60F4b156E2",
    "MIP21_LIQUIDATION_ORACLE": "0x88f88Bb9E66241B73B84f3A6E197FbBa487b1E30",
    "RWA001_A_URN": "0xa3342059BcDcFA57a13b12a35eD4BBE59B873005",
    "RWA001_A_INPUT_CONDUIT": "0x486C85e2bb9801d14f6A8fdb78F5108a0fd932f2",
    "RWA001_A_OUTPUT_CONDUIT": "0xb3eFb912e1cbC0B26FC17388Dd433Cecd2206C3d",
    "MCD_JOIN_ETH_C": "0xF04a5cC80B1E94C69B48f5ee68a08CD2F09A7c3E",
    "MCD_CLIP_CALC_ETH_C": "0x1c4fC274D12b2e1BBDF97795193D3148fCDa6108",
    "RWA002": "0xAAA760c2027817169D7C8DB0DC61A2fb4c19AC23",
    "PIP_RWA002": "0xd2473237E20Bd52F8E7cE0FD79403A6a82fbAEC8",
    "MCD_JOIN_RWA002_A": "0xe72C7e90bc26c11d45dBeE736F0acf57fC5B7152",
    "RWA002_A_URN": "0x225B3da5BE762Ee52B182157E67BeA0b31968163",
    "RWA002_A_INPUT_CONDUIT": "0x2474F297214E5d96Ba4C81986A9F0e5C260f445D",
    "RWA002_A_OUTPUT_CONDUIT": "0x2474F297214E5d96Ba4C81986A9F0e5C260f445D",
    "MCD_DOG": "0x135954d155898D42C90D2a57824C690e0c7BEf1B",
    "CLIPPER_MOM": "0x79FBDF16b366DFb14F66cE4Ac2815Ca7296405A0",
    "MCD_CLIP_LINK_A": "0x832Dd5f17B30078a5E46Fdb8130A68cBc4a74dC0",
    "MCD_CLIP_YFI_A": "0x9daCc11dcD0aa13386D295eAeeBBd38130897E6f",
    "MCD_CLIP_ETH_A": "0xc67963a226eddd77B91aD8c421630A1b0AdFF270",
    "MCD_CLIP_ETH_B": "0x71eb894330e8a4b96b8d6056962e7F116F50e06F",
    "MCD_CLIP_ETH_C": "0xc2b12567523e3f3CBd9931492b91fe65b240bc47",
    "MCD_CLIP_WBTC_A": "0x0227b54AdbFAEec5f1eD1dFa11f54dcff9076e2C",
    "MCD_CLIP_BAT_A": "0x3D22e6f643e2F4c563fD9db22b229Cbb0Cd570fb",
    "MCD_CLIP_ZRX_A": "0xdc90d461E148552387f3aB3EBEE0Bdc58Aa16375",
    "MCD_CLIP_KNC_A": "0x006Aa3eB5E666D8E006aa647D4afAB212555Ddea",
    "MCD_CLIP_MANA_A": "0xF5C8176E1eB0915359E46DEd16E52C071Bb435c0",
    "MCD_CLIP_COMP_A": "0x2Bb690931407DCA7ecE84753EA931ffd304f0F38",
    "MCD_CLIP_LRC_A": "0x81C5CDf4817DBf75C7F08B8A1cdaB05c9B3f70F7",
    "MCD_CLIP_BAL_A": "0x6AAc067bb903E633A422dE7BE9355E62B3CE0378",
    "MCD_CLIP_UNI_A": "0x3713F83Ee6D138Ce191294C131148176015bC29a",
    "MCD_CLIP_RENBTC_A": "0x834719BEa8da68c46484E001143bDDe29370a6A3",
    "MCD_CLIP_AAVE_A": "0x8723b74F598DE2ea49747de5896f9034CC09349e",
    "MCD_CLIP_UNIV2DAIETH_A": "0x9F6981bA5c77211A34B76c6385c0f6FA10414035",
    "MCD_CLIP_UNIV2USDCETH_A": "0x93AE03815BAF1F19d7F18D9116E4b637cc32A131",
    "MCD_CLIP_UNIV2ETHUSDT_A": "0x2aC4C9b49051275AcB4C43Ec973082388D015D48",
    "MCD_CLIP_UNIV2WBTCDAI_A": "0x4fC53a57262B87ABDa61d6d0DB2bE7E9BE68F6b8",
    "MCD_CLIP_UNIV2WBTCETH_A": "0xb15afaB996904170f87a64Fe42db0b64a6F75d24",
    "MCD_CLIP_UNIV2LINKETH_A": "0x6aa0520354d1b84e1C6ABFE64a708939529b619e",
    "MCD_CLIP_UNIV2UNIETH_A": "0xb0ece6F5542A4577E2f1Be491A937Ccbbec8479e",
    "MCD_CLIP_UNIV2AAVEETH_A": "0x854b252BA15eaFA4d1609D3B98e00cc10084Ec55",
    "MCD_CLIP_UNIV2DAIUSDT_A": "0xe4B82Be84391b9e7c56a1fC821f47569B364dd4a",
    "MCD_CLIP_USDC_A": "0x046b1A5718da6A226D912cFd306BA19980772908",
    "MCD_CLIP_USDC_B": "0x5590F23358Fe17361d7E4E4f91219145D8cCfCb3",
    "MCD_CLIP_TUSD_A": "0x0F6f88f8A4b918584E3539182793a0C276097f44",
    "MCD_CLIP_USDT_A": "0xFC9D6Dd08BEE324A5A8B557d2854B9c36c2AeC5d",
    "MCD_CLIP_PAXUSD_A": "0xBCb396Cd139D1116BD89562B49b9D1d6c25378B0",
    "MCD_CLIP_GUSD_A": "0xa47D68b9dB0A0361284fA04BA40623fcBd1a263E",
    "MCD_CLIP_PSM_USDC_A": "0x66609b4799fd7cE12BA799AD01094aBD13d5014D",
    "MCD_CLIP_UNIV2DAIUSDC_A": "0x9B3310708af333f6F379FA42a5d09CBAA10ab309",
    "MCD_FLASH": "0x60744434d6339a6B27d73d9Eda62b6F66a0a04FA",
    "RWA003": "0x07F0A80aD7AeB7BfB7f139EA71B3C8f7E17156B9",
    "PIP_RWA003": "0xDeF7E88447F7D129420FC881B2a854ABB52B73B8",
    "MCD_JOIN_RWA003_A": "0x1Fe789BBac5b141bdD795A3Bc5E12Af29dDB4b86",
    "RWA003_A_URN": "0x7bF825718e7C388c3be16CFe9982539A7455540F",
    "RWA003_A_INPUT_CONDUIT": "0x2A9798c6F165B6D60Cfb923Fe5BFD6f338695D9B",
    "RWA003_A_OUTPUT_CONDUIT": "0x2A9798c6F165B6D60Cfb923Fe5BFD6f338695D9B",
    "RWA004": "0x873F2101047A62F84456E3B2B13df2287925D3F9",
    "PIP_RWA004": "0x5eEE1F3d14850332A75324514CcbD2DBC8Bbc566",
    "MCD_JOIN_RWA004_A": "0xD50a8e9369140539D1c2D113c4dC1e659c6242eB",
    "RWA004_A_URN": "0xeF1699548717aa4Cf47aD738316280b56814C821",
    "RWA004_A_INPUT_CONDUIT": "0xe1ed3F588A98bF8a3744f4BF74Fd8540e81AdE3f",
    "RWA004_A_OUTPUT_CONDUIT": "0xe1ed3F588A98bF8a3744f4BF74Fd8540e81AdE3f",
    "RWA005": "0x6DB236515E90fC831D146f5829407746EDdc5296",
    "PIP_RWA005": "0x8E6039C558738eb136833aB50271ae065c700d2B",
    "MCD_JOIN_RWA005_A": "0xA4fD373b93aD8e054970A3d6cd4Fd4C31D08192e",
    "RWA005_A_URN": "0xc40907545C57dB30F01a1c2acB242C7c7ACB2B90",
    "RWA005_A_INPUT_CONDUIT": "0x5b702e1fEF3F556cbe219eE697D7f170A236cc66",
    "RWA005_A_OUTPUT_CONDUIT": "0x5b702e1fEF3F556cbe219eE697D7f170A236cc66",
    "RWA006": "0x4EE03cfBF6E784c462839f5954d60f7C2B60b113",
    "PIP_RWA006": "0xB8AeCF04Fdf22Ef6C0c6b6536896e1F2870C41D3",
    "MCD_JOIN_RWA006_A": "0x5E11E34b6745FeBa9449Ae53c185413d6EdC66BE",
    "RWA006_A_URN": "0x0C185bf5388DdfDB288F4D875265d456D18FD9Cb",
    "RWA006_A_INPUT_CONDUIT": "0x8Fe38D1E4293181273E2e323e4c16e0D1d4861e3",
    "RWA006_A_OUTPUT_CONDUIT": "0x8Fe38D1E4293181273E2e323e4c16e0D1d4861e3",
    "VOTE_DELEGATE_PROXY_FACTORY": "0xD897F108670903D1d6070fcf818f9db3615AF272",
    "MCD_JOIN_PSM_PAX_A": "0x7bbd8cA5e413bCa521C2c80D8d1908616894Cf21",
    "MCD_CLIP_PSM_PAX_A": "0x5322a3551bc6a1b39d5D142e5e38Dc5B4bc5B3d2",
    "MCD_CLIP_CALC_PSM_PAX_A": "0xC19eAc21A4FccdD30812F5fF5FebFbD6817b7593",
    "MCD_PSM_PAX_A": "0x961Ae24a1Ceba861D1FDf723794f6024Dc5485Cf",
    "PIP_PAX": "0x043B963E1B2214eC90046167Ea29C2c8bDD7c0eC",
    "MATIC": "0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0",
    "MCD_JOIN_MATIC_A": "0x885f16e177d45fC9e7C87e1DA9fd47A9cfcE8E13",
    "MCD_CLIP_MATIC_A": "0x29342F530ed6120BDB219D602DaFD584676293d1",
    "MCD_CLIP_CALC_MATIC_A": "0xdF8C347B06a31c6ED11f8213C2366348BFea68dB",
    "PIP_MATIC": "0x8874964279302e6d4e523Fb1789981C39a1034Ba",
    "CALC_FAB": "0xE1820A2780193d74939CcA104087CADd6c1aA13A",
    "PAX": "0x8E870D67F660D95d5be530380D0eC0bd388289E1",
    "MCD_VEST_DAI": "0xa4c22f0e25C6630B2017979AcF1f865e94695C4b",
    "MCD_VEST_MKR": "0x0fC8D4f2151453ca0cA56f07359049c8f07997Bd",
    "GUNIV3DAIUSDC1": "0xAbDDAfB225e10B90D798bB8A886238Fb835e2053",
    "MCD_JOIN_GUNIV3DAIUSDC1_A": "0xbFD445A97e7459b0eBb34cfbd3245750Dba4d7a4",
    "MCD_CLIP_GUNIV3DAIUSDC1_A": "0x5048c5Cd3102026472f8914557A1FD35c8Dc6c9e",
    "MCD_CLIP_CALC_GUNIV3DAIUSDC1_A": "0x25B17065b94e3fDcD97d94A2DA29E7F77105aDd7",
    "PIP_GUNIV3DAIUSDC1": "0x7F6d78CC0040c87943a0e0c140De3F77a273bd58",
    "MCD_VEST_MKR_TREASURY": "0x6D635c8d08a1eA2F1687a5E46b666949c977B7dd",
    "OPTIMISM_DAI_BRIDGE": "0x10E6593CDda8c58a1d0f14C5164B376352a55f2F",
    "OPTIMISM_ESCROW": "0x467194771dAe2967Aef3ECbEDD3Bf9a310C76C65",
    "OPTIMISM_GOV_RELAY": "0x09B354CDA89203BB7B3131CC728dFa06ab09Ae2F",
    "ARBITRUM_DAI_BRIDGE": "0xD3B5b60020504bc3489D6949d545893982BA3011",
    "ARBITRUM_ESCROW": "0xA10c7CE4b876998858b1a9E12b10092229539400",
    "ARBITRUM_GOV_RELAY": "0x9ba25c289e351779E0D481Ba37489317c34A899d",
    "STETH": "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84",
    "WSTETH": "0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0",
    "PIP_WSTETH": "0xFe7a2aC0B945f12089aEEB6eCebf4F384D9f043F",
    "MCD_JOIN_WSTETH_A": "0x10CD5fbe1b404B7E19Ef964B63939907bdaf42E2",
    "MCD_CLIP_WSTETH_A": "0x49A33A28C4C7D9576ab28898F4C9ac7e52EA457A",
    "MCD_CLIP_CALC_WSTETH_A": "0x15282b886675cc1Ce04590148f456428E87eaf13",
    "ADAI": "0x028171bCA77440897B824Ca71D1c56caC55b68A3",
    "RWA013_A_INPUT_CONDUIT": "0x615984F33604011Fcd76E9b89803Be3816276E61",
    "RWA013_A_URN": "0x9C170dd80Ee2CA5bfDdF00cbE93e8faB2D05bA6D",
    "MCD_JOIN_RWA013_A": "0x779D0fD012815D4239BAf75140e6B2971BEd5113",
    "PIP_ADAI": "0x6A858592fC4cBdf432Fc9A1Bc8A0422B99330bdF",
    "DIRECT_MOM": "0x1AB3145E281c01a1597c8c62F9f060E8e3E02fAB",
    "JOIN_FAB": "0xf1738d22140783707Ca71CB3746e0dc7Bf2b0264",
    "MCD_JOIN_WBTC_B": "0xfA8c996e158B80D77FbD0082BB437556A65B96E0",
    "MCD_CLIP_WBTC_B": "0xe30663C6f83A06eDeE6273d72274AE24f1084a22",
    "MCD_CLIP_CALC_WBTC_B": "0xeb911E99D7ADD1350DC39d84D60835BA9B287D96",
    "MCD_JOIN_WBTC_C": "0x7f62f9592b823331E012D3c5DdF2A7714CfB9de2",
    "MCD_CLIP_WBTC_C": "0x39F29773Dcb94A32529d0612C6706C49622161D1",
    "MCD_CLIP_CALC_WBTC_C": "0x4fa2A328E7f69D023fE83454133c273bF5ACD435",
    "MCD_JOIN_PSM_GUSD_A": "0x79A0FA989fb7ADf1F8e80C93ee605Ebb94F7c6A5",
    "MCD_CLIP_PSM_GUSD_A": "0xf93CC3a50f450ED245e003BFecc8A6Ec1732b0b2",
    "MCD_CLIP_CALC_PSM_GUSD_A": "0x7f67a68a0ED74Ea89A82eD9F243C159ed43a502a",
    "MCD_PSM_GUSD_A": "0x204659B2Fd2aD5723975c362Ce2230Fba11d3900",
    "GUNIV3DAIUSDC2": "0x50379f632ca68D36E50cfBC8F78fe16bd1499d1e",
    "MCD_JOIN_GUNIV3DAIUSDC2_A": "0xA7e4dDde3cBcEf122851A7C8F7A55f23c0Daf335",
    "MCD_CLIP_GUNIV3DAIUSDC2_A": "0xB55da3d3100C4eBF9De755b6DdC24BF209f6cc06",
    "MCD_CLIP_CALC_GUNIV3DAIUSDC2_A": "0xef051Ca2A2d809ba47ee0FC8caaEd06E3D832225",
    "PIP_GUNIV3DAIUSDC2": "0xcCBa43231aC6eceBd1278B90c3a44711a00F4e93",
    "CLIP_FAB": "0x0716F25fBaAae9b63803917b6125c10c313dF663",
    "RWA013": "0xD6C7FD4392D328e4a8f8bC50F4128B64f4dB2d4C",
    "CDP_REGISTRY": "0xBe0274664Ca7A68d6b5dF826FB3CcB7c620bADF3",
    "MCD_CROPPER": "0x8377CD01a5834a6EaD3b7efb482f678f2092b77e",
    "MCD_CROPPER_IMP": "0xaFB21A0e9669cdbA539a4c91Bf6B94c5F013c0DE",
    "PROXY_ACTIONS_CROPPER": "0xa2f69F8B9B341CFE9BfBb3aaB5fe116C89C95bAF",
    "PROXY_ACTIONS_END_CROPPER": "0x38f7C166B5B22906f04D8471E241151BA45d97Af",
    "CRVV1ETHSTETH": "0x06325440D014e39736583c165C2963BA99fAf14E",
    "PIP_CRVV1ETHSTETH": "0xEa508F82728927454bd3ce853171b0e2705880D4",
    "MCD_JOIN_CRVV1ETHSTETH_A": "0x82D8bfDB61404C796385f251654F6d7e92092b5D",
    "MCD_CLIP_CRVV1ETHSTETH_A": "0x1926862F899410BfC19FeFb8A3C69C7Aed22463a",
    "MCD_CLIP_CALC_CRVV1ETHSTETH_A": "0x8a4780acABadcae1a297b2eAe5DeEbd7d50DEeB8",
    "MCD_VEST_DAI_LEGACY": "0x2Cc583c0AaCDaC9e23CB601fDA8F1A0c56Cdcb71",
    "MCD_JOIN_WSTETH_B": "0x248cCBf4864221fC0E840F29BB042ad5bFC89B5c",
    "MCD_CLIP_WSTETH_B": "0x3ea60191b7d5990a3544B6Ef79983fD67e85494A",
    "MCD_CLIP_CALC_WSTETH_B": "0x95098b29F579dbEb5c198Db6F30E28F7f3955Fbb",
    "MCD_CURE": "0x0085c9feAb2335447E1F4DC9bf3593a8e28bdfc7",
    "PIP_RWA013": "0x69Cf63ed6eD57Ad129bF67EB726Ae1bd293edbB0",
    "STARKNET_ESCROW_MOM": "0xc238E3D63DfD677Fa0FA9985576f0945C581A266",
    "STARKNET_ESCROW": "0x0437465dfb5B79726e35F08559B0cBea55bb585C",
    "STARKNET_DAI_BRIDGE": "0x9F96fE0633eE838D0298E8b8980E6716bE81388d",
    "STARKNET_GOV_RELAY": "0x2385C60D2756Ed8CA001817fC37FDa216d7466c0",
    "STARKNET_CORE": "0xc662c410C0ECf747543f5bA90660f6ABeBD9C8c4",
    "RWA008": "0xb9737098b50d7c536b6416dAeB32879444F59fCA",
    "PIP_RWA008": "0x2623dE50D8A6FdC2f0D583327142210b8b464bfd",
    "MCD_JOIN_RWA008_A": "0x56eDD5067d89D4E65Bf956c49eAF054e6Ff0b262",
    "RWA008_A_URN": "0x495215cabc630830071F80263a908E8826a66121",
    "RWA008_A_INPUT_CONDUIT": "0xa397a23dDA051186F202C67148c90683c413383C",
    "RWA008_A_OUTPUT_CONDUIT": "0x21CF5Ad1311788D762f9035829f81B9f54610F0C",
    "RWA009": "0x8b9734bbaA628bFC0c9f323ba08Ed184e5b88Da2",
    "PIP_RWA009": "0xdc7D370A089797Fe9556A2b0400496eBb3a61E44",
    "MCD_JOIN_RWA009_A": "0xEe0FC514280f09083a32AE906cCbD2FAc4c680FA",
    "RWA009_A_URN": "0x1818EE501cd28e01E058E7C283E178E9e04a1e79",
    "RWA009_A_JAR": "0x6C6d4Be2223B5d202263515351034861dD9aFdb6",
    "RWA009_A_OUTPUT_CONDUIT": "0x508D982e13263Fc8e1b5A4E6bf59b335202e36b4",
    "RWA_TOKEN_FAB": "0x2B3a4c18705e99bC29b22222dA7E10b643658552",
    "MCD_JOIN_TELEPORT_FW_A": "0x41Ca7a7Aa2Be78Cf7CB80C0F4a9bdfBC96e81815",
    "MCD_ORACLE_AUTH_TELEPORT_FW_A": "0x324a895625E7AE38Fc7A6ae91a71e7E937Caa7e6",
    "MCD_ROUTER_TELEPORT_FW_A": "0xeEf8B35eD538b6Ef7DbA82236377aDE4204e5115",
    "OPTIMISM_TELEPORT_BRIDGE": "0x920347f49a9dbe50865EB6161C3B2774AC046A7F",
    "OPTIMISM_TELEPORT_FEE": "0xA7C088AAD64512Eff242901E33a516f2381b8823",
    "ARBITRUM_TELEPORT_BRIDGE": "0x22218359E78bC34E532B653198894B639AC3ed72",
    "ARBITRUM_TELEPORT_FEE": "0xA7C088AAD64512Eff242901E33a516f2381b8823",
    "RWA007": "0x078fb926b041a816FaccEd3614Cf1E4bc3C723bD",
    "PIP_RWA007": "0x7bb4BcA758c4006998a2769776D9E4E6D86e0Dab",
    "MCD_JOIN_RWA007_A": "0x476aaD14F42469989EFad0b7A31f07b795FF0621",
    "RWA007_A_URN": "0x481bA2d2e86a1c41427893899B5B0cEae41c6726",
    "RWA007_A_JAR": "0xef1B095F700BE471981aae025f92B03091c3AD47",
    "RWA007_A_JAR_INPUT_CONDUIT": "0xc8bb4e2B249703640e89265e2Ae7c9D5eA2aF742",
    "RWA007_A_INPUT_CONDUIT": "0x58f5e979eF74b60a9e5F955553ab8e0e65ba89c9",
    "RWA007_A_OUTPUT_CONDUIT": "0x701C3a384c613157bf473152844f368F2d6EF191",
    "RETH": "0xae78736Cd615f374D3085123A210448E74Fc6393",
    "PIP_RETH": "0xeE7F0b350aA119b3d05DC733a4621a81972f7D47",
    "MCD_JOIN_RETH_A": "0xC6424e862f1462281B0a5FAc078e4b63006bDEBF",
    "MCD_CLIP_RETH_A": "0x27CA5E525ea473eD52Ea9423CD08cCc081d96a98",
    "MCD_CLIP_CALC_RETH_A": "0xc59B62AFC96cf9737F717B5e5815070C0f154396",
    "STARKNET_DAI_BRIDGE_LEGACY": "0x659a00c33263d9254Fed382dE81349426C795BB6",
    "STARKNET_TELEPORT_BRIDGE": "0x95D8367B74ef8C5d014ff19C212109E243748e28",
    "STARKNET_TELEPORT_FEE": "0x2123159d2178f07E3899d9d22aad2Fb177B59C48",
    "DIRECT_HUB": "0x12F36cdEA3A28C35aC8C6Cc71D9265c17C74A27F",
    "RWA013_A_OUTPUT_CONDUIT": "0x615984F33604011Fcd76E9b89803Be3816276E61",
    "DIRECT_COMPV2_DAI_POOL": "0x621fE4Fde2617ea8FFadE08D0FF5A862aD287EC2",
    "DIRECT_COMPV2_DAI_PLAN": "0xD0eA20f9f9e64A3582d569c8745DaCD746274AEe",
    "DIRECT_COMPV2_DAI_ORACLE": "0x0e2bf18273c953B54FE0a9dEC5429E67851D9468",
    "STARKNET_GOV_RELAY_LEGACY": "0x9eed6763BA8D89574af1478748a7FDF8C5236fE0",
    "GNO": "0x6810e776880C02933D47DB1b9fc05908e5386b96",
    "PIP_GNO": "0xd800ca44fFABecd159c7889c3bf64a217361AEc8",
    "MCD_JOIN_GNO_A": "0x7bD3f01e24E0f0838788bC8f573CEA43A80CaBB5",
    "MCD_CLIP_GNO_A": "0xd9e758bd239e5d568f44D0A748633f6a8d52CBbb",
    "MCD_CLIP_CALC_GNO_A": "0x17b6D0e4237ea7F880aF5F58257cd232a04171D9",
    "RWA010": "0x20C72C1fdd589C4Aaa8d9fF56a43F3B17BA129f8",
    "PIP_RWA010": "0xfBAa6a09A39D485a5Be9F5ebfe09C602E63b21EF",
    "MCD_JOIN_RWA010_A": "0xde2828c3F7B2161cF2a1711edc36c73C56EA72aE",
    "RWA010_A_URN": "0x4866d5d24CdC6cc094423717663b2D3343d4EFF9",
    "RWA010_A_INPUT_CONDUIT": "0x1F5C294EF3Ff2d2Da30ea9EDAd490C28096C91dF",
    "RWA010_A_OUTPUT_CONDUIT": "0x1F5C294EF3Ff2d2Da30ea9EDAd490C28096C91dF",
    "RWA011": "0x0b126F85285d1786F52FC911AfFaaf0d9253e37a",
    "PIP_RWA011": "0x8bDC64d73da9631C962C4932a391CB78065ce7a9",
    "MCD_JOIN_RWA011_A": "0x9048cb84F46e94Ff312DcC50f131191c399D9bC3",
    "RWA011_A_URN": "0x32C9bBA0841F2557C10d3f0d30092f138251aFE6",
    "RWA011_A_INPUT_CONDUIT": "0x8e74e529049bB135CF72276C1845f5bD779749b0",
    "RWA011_A_OUTPUT_CONDUIT": "0x8e74e529049bB135CF72276C1845f5bD779749b0",
    "RWA012": "0x3c7f1379B5ac286eB3636668dEAe71EaA5f7518c",
    "PIP_RWA012": "0x4FA7c611bD25DA38bC929C2A67290FbE49DDFF56",
    "MCD_JOIN_RWA012_A": "0x75646F68B8c5d8F415891F7204978Efb81ec6410",
    "RWA012_A_URN": "0xB22E9DBF60a5b47c8B2D0D6469548F3C2D036B7E",
    "RWA012_A_INPUT_CONDUIT": "0x795b917eBe0a812D406ae0f99D71caf36C307e21",
    "RWA012_A_OUTPUT_CONDUIT": "0x795b917eBe0a812D406ae0f99D71caf36C307e21",
    "DIRECT_AAVEV2_DAI_POOL": "0x66aE0574Eb28B92c82569b293B856BB99f80F040",
    "DIRECT_AAVEV2_DAI_PLAN": "0x5846Aee09298f8F3aB5D837d540232d19e5d5813",
    "DIRECT_AAVEV2_DAI_ORACLE": "0x634051fbA31829E245C616e79E289f89c8B851c2",
    "LINE_MOM": "0x9c257e5Aaf73d964aEBc2140CA38078988fB0C10",
    "DIRECT_SPARK_DAI_POOL": "0xAfA2DD8a0594B2B24B59de405Da9338C4Ce23437",
    "DIRECT_SPARK_DAI_PLAN": "0x104FaDbb7e17db1A685bBa61007DfB015206a4D2",
    "DIRECT_SPARK_DAI_ORACLE": "0xCBD53B683722F82Dc82EBa7916065532980d4833",
    "RWA014": "0x75dCa04C4aCC1FfB0AEF940e5b49e2C17416008a",
    "PIP_RWA014": "0xfeDAB3d532Af95b10F064c73bebEF68a0d0A5f36",
    "MCD_JOIN_RWA014_A": "0xAd722E51569EF41861fFf5e11942a8E07c7C309e",
    "RWA014_A_URN": "0xf082566Ac42566cF7B392C8e58116a27eEdcBe63",
    "RWA014_A_JAR": "0x71eC6d5Ee95B12062139311CA1fE8FD698Cbe0Cf",
    "RWA014_A_INPUT_CONDUIT_URN": "0x6B86bA08Bd7796464cEa758061Ac173D0268cf49",
    "RWA014_A_INPUT_CONDUIT_JAR": "0x391470cD3D8307AdC051d878A95Fa9459F800Dbc",
    "RWA014_A_OUTPUT_CONDUIT": "0xD7cBDFdE553DE2063caAfBF230Be135e5DbB5064",
    "RWA015": "0xf5E5E706EfC841BeD1D24460Cd04028075cDbfdE",
    "PIP_RWA015": "0xDa28e04514E718271b37c9F36fbaf45b4BF42dF4",
    "MCD_JOIN_RWA015_A": "0x8938988f7B368f74bEBdd3dcd8D6A3bd18C15C0b",
    "RWA015_A_URN": "0xebFDaa143827FD0fc9C6637c3604B75Bbcfb7284",
    "RWA015_A_JAR": "0xc27C3D3130563C1171feCC4F76C217Db603997cf",
    "RWA015_A_INPUT_CONDUIT_JAR_PAX": "0x79Fc3810735959db3C6D4fc64F7F7b5Ce48d1CEc",
    "RWA015_A_INPUT_CONDUIT_URN_PAX": "0x4f7f76f31CE6Bb20809aaCE30EfD75217Fbfc217",
    "RWA015_A_OUTPUT_CONDUIT": "0x1E86CB085f249772f7e7443631a87c6BDba2aCEb",
    "CRON_SEQUENCER": "0x238b4E35dAed6100C6162fAE4510261f88996EC9",
    "CRON_AUTOLINE_JOB": "0x67AD4000e73579B9725eE3A149F85C4Af0A61361",
    "CRON_LERP_JOB": "0x8F8f2FC1F0380B9Ff4fE5c3142d0811aC89E32fB",
    "CRON_D3M_JOB": "0x1Bb799509b0B039345f910dfFb71eEfAc7022323",
    "CRON_CLIPPER_MOM_JOB": "0xc3A76B34CFBdA7A3a5215629a0B937CBDEC7C71a",
    "CRON_ORACLE_JOB": "0xe717Ec34b2707fc8c226b34be5eae8482d06ED03",
    "FLAPPER_MOM": "0xee2058A11612587Ef6F5470e7776ceB0E4736078",
    "PIP_MKR": "0xdbBe5e9B1dAa91430cF0772fCEbe53F6c6f137DF",
    "CRON_FLAP_JOB": "0xc32506E9bB590971671b649d9B8e18CB6260559F",
    "RWA015_A_INPUT_CONDUIT_URN_GUSD": "0xAB80C37cB5b21238D975c2Cea46e0F12b3d84B06",
    "MCD_PSM_GUSD_A_JAR": "0xf2E7a5B83525c3017383dEEd19Bb05Fe34a62C27",
    "MCD_PSM_GUSD_A_INPUT_CONDUIT_JAR": "0x6934218d8B3E9ffCABEE8cd80F4c1C4167Afa638",
    "MCD_PSM_PAX_A_JAR": "0x8bF8b5C58bb57Ee9C97D0FEA773eeE042B10a787",
    "MCD_PSM_PAX_A_INPUT_CONDUIT_JAR": "0xDa276Ab5F1505965e0B6cD1B6da2A18CcBB29515",
    "RWA009_A_INPUT_CONDUIT_URN_USDC": "0x08012Ec53A7fAbf6F33318dfb93C1289886eBBE1",
    'Contract_8': '0x8e2a84d6ade1e7fffee039a35ef5f19f13057152',
    'Contract_1': '0x448a5065aebb8e423f0896e6c5d525c040f59af3',
}

maker_addresses = (
    pl.DataFrame(
        [
            [label, address]
            for label, address in raw_maker_addresses.items()
            if label.startswith("RWA")
            or label.startswith("MCD")
            or label.startswith("CDP")
            or label.startswith("PIP")
            or label.startswith("DIRECT")
            or label.startswith("CRON")
            or label.startswith("PROXY")
            or label.startswith("Contract")
        ],
        orient="row",
        schema=["name", "address"],
    )
    .select(
        address="address",
        label=pl.lit("maker"),
        name="name",
    )
    .group_by("address")
    .agg(pl.last("*"))
)

maker_addresses

In [ ]:
pl.Series(
    name.split("_")[0]
    for name, address in raw_maker_addresses.items()
    if address not in maker_addresses['address']
).value_counts(
    sort=True
).head(20)

In [ ]:
maker_addresses = maker_addresses.with_columns(
    maker_addresses['address'].evm.hex_to_binary()
)

In [ ]:
maker_addresses.write_parquet(    
    get_label_path('maker', min_block, max_block)
)

# synthetix

https://developer.synthetix.io/addresses/

In [ ]:
synthetix_addresses = {
    "AddressResolver": "0x823bE81bbF96BEc0e25CA13170F5AaCb5B79ba83",
    "CircuitBreaker": "0x9f231dBE53D460f359B2B8CC47574493caA5B7Bf",
    "CollateralErc20": "0xaa03aB31b55DceEeF845C8d17890CC61cD98eD04",
    "CollateralEth": "0x5c8344bcdC38F1aB5EB5C1d4a35DdEeA522B5DfA",
    "CollateralManager": "0x067e398605E84F2D0aEEC1806e62768C5110DCc6",
    "CollateralManagerState": "0x573E5105c4B92416D1544A188F1bf77d442Bb52d",
    "CollateralShort": "0x1F2c3a1046c32729862fcB038369696e3273a516",
    "CollateralStateErc20": "0x8A1A00Df7aA0102497C7591912bA8301e8dB8CdB",
    "CollateralStateEth": "0xbe5B5a7c198bC156474ed5c33CBf2F3F604F8fF8",
    "CollateralStateShort": "0x13A114a3Fa8A6CE03a0C5488BE9e614d78eDdb0c",
    "DappMaintenance": "0xAb0B2f1Cf979cdbF4676251F35353eC5AF2732Dd",
    "DebtCache": "0x1620Aa736939597891C1940CF0d28b82566F9390",
    "DebtMigratorOnEthereum": "0x94f864e55c77E07C2C7BF7bFBc334b7a8123442A",
    "DelegateApprovals": "0x15fd6e554874B9e70F832Ed37f231Ac5E142362f",
    "DelegateApprovalsEternalStorage": "0x8F586F063ffbb89b186C8e604FC6614766f9C9d1",
    "Depot": "0xE1f64079aDa6Ef07b03982Ca34f1dD7152AA3b86",
    "DirectIntegrationManager": "0x5F41718459955217df6A02AA8f5C19614ECfA6E7",
    "EscrowChecker": "0x3b399e00AFd8201ACf8A5a0EcCF1C47d8D5E41da",
    "EtherWrapper": "0xC1AAE9d18bBe386B102435a8632C8063d31e747C",
    "ExchangeCircuitBreaker": "0xeAcaEd9581294b1b5cfb6B941d4B8B81B2005437",
    "ExchangeRates": "0x648280dD2db772CD018A0CEC72fab5bF8B7683AB",
    "ExchangeSettlementLib": "0xAa5A3D7F04E15b22Eb3664B56310aA18A3527Ec7",
    "ExchangeState": "0x545973f28950f50fc6c7F52AAb4Ad214A27C0564",
    "Exchanger": "0xaeA0065E146FD75Dc24465961a583827284D405a",
    "FeePool": "0x83105D7CDd2fd9b8185BFF1cb56bB1595a618618",
    "FeePoolEternalStorage": "0xC9DFff5fA5605fd94F8B7927b892F2B57391e8bB",
    "FeePoolState": "0x11164F6a47C3f8472D19b9aDd516Fc780cb7Ee02",
    "FlexibleStorage": "0xc757aCBa3c0506218b3022266a9DC7F3612d85f5",
    "FuturesMarketManager": "0xd3f527F47A9DF2f6cBf631081315b6e2FE4e4521",
    "IssuanceEternalStorage": "0x631E93A0fb06B5eC6d52c0A2D89a3f9672d6Ba64",
    "Issuer": "0xca68a3D663483515a9D434E854AB59A41b3A523c",
    "Liquidator": "0x8e9757479D5ad4E7f9d951B60d39F5220b893d6c",
    "LiquidatorRewards": "0xf79603a71144e415730C1A6f57F366E4Ea962C00",
    "Math": "0x385e1Eb2FF28F4A637DA2C9971F8CAF5F5b1E77c",
    "NativeEtherWrapper": "0x7A3d898b717e50a96fd8b232E9d15F0A547A7eeb",
    "OneNetAggregatorDebtRatio": "0x977d0DD7eA212E9ca1dcD4Ec15cd7Ceb135fa68D",
    "OneNetAggregatorIssuedSynths": "0xcf1405b18dBCEA2893Abe635c88359C75878B9e1",
    "OneNetAggregatorsDEFI": "0x646F23085281Dbd006FBFD211FD38d0743884864",
    "OwnerRelayOnEthereum": "0x0e16A6876210841577b233C4165d7B7EdF640b8a",
    "ProxyERC20": "0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F",
    "ProxyERC20sUSD": "0x57Ab1ec28D129707052df4dF418D58a2D46d5f51",
    "ProxyFeePool": "0xb440DD674e1243644791a4AdfE3A2AbB0A92d309",
    "ProxySynthetixUse": "0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F",
    "ProxysAAVE": "0xd2dF355C19471c8bd7D8A3aa27Ff4e26A21b4076",
    "ProxysADA": "0xe36E2D3c7c34281FA3bC737950a68571736880A1",
    "ProxysAUD": "0xF48e200EAF9906362BB1442fca31e0835773b8B4",
    "ProxysBTC": "0xfE18be6b3Bd88A2D2A7f928d00292E7a9963CfC6",
    "ProxysCHF": "0x0F83287FF768D1c1e17a42F44d644D7F22e8ee1d",
    "ProxysDOT": "0x1715AC0743102BF5Cd58EfBB6Cf2dC2685d967b6",
    "ProxysETH": "0x5e74C9036fb86BD7eCdcb084a0673EFc32eA31cb",
    "ProxysETHBTC": "0x104eDF1da359506548BFc7c25bA1E28C16a70235",
    "ProxysEUR": "0xD71eCFF9342A5Ced620049e616c5035F1dB98620",
    "ProxysGBP": "0x97fe22E7341a0Cd8Db6F6C021A24Dc8f4DAD855F",
    "ProxysJPY": "0xF6b1C627e95BFc3c1b4c9B825a032Ff0fBf3e07d",
    "ProxysKRW": "0x269895a3dF4D73b077Fc823dD6dA1B95f72Aaf9B",
    "ProxysLINK": "0xbBC455cb4F1B9e4bFC4B73970d360c8f032EfEE6",
    "ProxysUSDUse": "0x57Ab1ec28D129707052df4dF418D58a2D46d5f51",
    "ReadProxyAddressResolver": "0x4E3b31eB0E5CB73641EE1E65E7dCEFe520bA3ef2",
    "RewardEscrow": "0xb671F2210B1F6621A2607EA63E6B2DC3e2464d1F",
    "RewardEscrowV2": "0xAc86855865CbF31c8f9FBB68C749AD5Bd72802e3",
    "RewardEscrowV2Storage": "0x182738BD9eE9810BC11f1c81b07Ec6F3691110BB",
    "RewardsDistribution": "0x94433f0DA8B5bfb473Ea8cd7ad10D9c8aef4aB7b",
    "SafeDecimalMath": "0x84D626B2BB4D0F064067e4BF80FCe7055d8F3E7B",
    "ShortingRewardssBTC": "0xCed4055b47cfD0421f3727a35F69CE659c8bAF7a",
    "ShortingRewardssETH": "0x12DC1273915A16ab8BD47bA7866B240c253e4c12",
    "SignedSafeDecimalMath": "0x728A2B79Cad691531CC1146eF802617FF50c7095",
    "StakingRewardsSNXBalancer": "0xFBaEdde70732540cE2B11A8AC58Eb2dC0D69dE10",
    "StakingRewardsiBTC": "0x167009dcDA2e49930a71712D956f02cc980DcC1b",
    "StakingRewardsiETH": "0x3f27c540ADaE3a9E8c875C61e3B970b559d7F65d",
    "StakingRewardssAAPLBalancer": "0x7af65f1740c0eB816A27FD808EaF6Ab09F6Fa646",
    "StakingRewardssAMZNBalancer": "0xDC338C7544654c7dadFEb7E44076E457963113B0",
    "StakingRewardssBTCCurve": "0x13C1542A468319688B89E323fe9A3Be3A90EBb27",
    "StakingRewardssCOINBalancer": "0x1C1D97f6338759AB814a5A717aE359573Ab5D5d4",
    "StakingRewardssETHUniswapV1": "0x48D7f315feDcaD332F68aafa017c7C158BC54760",
    "StakingRewardssEURCurve": "0xc0d8994Cd78eE1980885DF1A0C5470fC977b5cFe",
    "StakingRewardssFBBalancer": "0x26Fa0665660c1D3a3666584669511d3c66Ad37Cb",
    "StakingRewardssGOOGBalancer": "0x6fB7F0E78582746bd01BcB6dfbFE62cA5F4F9175",
    "StakingRewardssMSFTBalancer": "0x9D003Cc298E7Ea141A809C241C0a703176DA3ba3",
    "StakingRewardssNFLXBalancer": "0x8Ef8cA2AcAaAfEc19fB366C11561718357F780F2",
    "StakingRewardssTSLABalancer": "0xF0de877F2F9E7A60767f9BA662F10751566AD01c",
    "StakingRewardssUSDCurve": "0xDCB6A51eA3CA5d3Fd898Fd6564757c7aAeC3ca92",
    "StakingRewardssXAUUniswapV2": "0x8302FE9F0C509a996573D3Cc5B0D5D51e4FDD5eC",
    "SupplySchedule": "0x8d203C458d536Fe0F97e9f741bC231EaC8cd91cf",
    "SynthRedeemer": "0xe533139Af961c9747356D947838c98451015e234",
    "SynthUtil": "0x81Aee4EA48f678E172640fB5813cf7A96AFaF6C3",
    "Synthetix": "0xd0dA9cBeA9C3852C5d63A95F9ABCC4f6eA0F9032",
    "SynthetixBridgeEscrow": "0x5Fd79D46EBA7F351fe49BFF9E87cdeA6c821eF9f",
    "SynthetixBridgeToOptimism": "0x39Ea01a0298C315d149a490E34B59Dbf2EC7e48F",
    "SynthetixDebtShare": "0x89FCb32F29e509cc42d0C8b6f058C993013A843F",
    "SynthetixEscrow": "0x971e78e0C92392A4E39099835cF7E6aB535b2227",
    "SynthetixState": "0x4b9Ca5607f1fF8019c1C6A3c2f0CC8de622D5B82",
    "SynthsAAVE": "0x942Eb6e8c029EB22103743C99985aF4F4515a559",
    "SynthsADA": "0x91b82d62Ff322b8e02b86f33E9A99a813437830d",
    "SynthsAUD": "0xfb020CA7f4e8C4a5bBBe060f59a249c6275d2b69",
    "SynthsBTC": "0x1cB004a8e84a5CE95C1fF895EE603BaC8EC506c7",
    "SynthsCHF": "0xBb5b03E920cF702De5A3bA9Fc1445aF4B3919c88",
    "SynthsDOT": "0x75A0c1597137AA36B40b6a515D997F9a6c6eefEB",
    "SynthsETH": "0x5D4C724BFe3a228Ff0E29125Ac1571FE093700a4",
    "SynthsETHBTC": "0x07C1E81C345A7c58d7c24072EFc5D929BD0647AD",
    "SynthsEUR": "0xa8E31E3C38aDD6052A9407298FAEB8fD393A6cF9",
    "SynthsGBP": "0xdc883b9d9Ee16f74bE08826E68dF4C9D9d26e8bD",
    "SynthsJPY": "0xE1cc2332852B2Ac0dA59A1f9D3051829f4eF3c1C",
    "SynthsKRW": "0xdAe6C79c46aB3B280Ca28259000695529cbD1339",
    "SynthsLINK": "0xDF69bC4541b86Aa4c5A470B4347E730c38b2c3B2",
    "SynthsUSD": "0x10A5F7D9D65bCc2734763444D4940a31b109275f",
    "SystemSettings": "0x202ae40Bed1640b09e2AF7aC5719D129A498B7C8",
    "SystemSettingsLib": "0x4a39AEf2281ac0D192a9c4783604833bA8F31174",
    "SystemStatus": "0x696c905F8F8c006cA46e9808fE7e00049507798F",
    "TokenStateSynthetix": "0x5b1b5fEa1b99D83aD479dF0C222F0492385381dD",
    "TokenStatesAAVE": "0x9BcED8A8E3Ad81c9b146FFC880358f734A06f7c0",
    "TokenStatesADA": "0x9956c5019a24fbd5B506AD070b771577bAc5c343",
    "TokenStatesAUD": "0xCb29D2cf2C65d3Be1d00F07f3441390432D55203",
    "TokenStatesBTC": "0x4F6296455F8d754c19821cF1EC8FeBF2cD456E67",
    "TokenStatesCHF": "0x52496fE8a4feaEFe14d9433E00D48E6929c13deC",
    "TokenStatesDOT": "0x73B1a2643507Cd30F11Dfcf2D974f4373E5BC077",
    "TokenStatesETH": "0x34A5ef81d18F3a305aE9C2d7DF42beef4c79031c",
    "TokenStatesETHBTC": "0x042A7A0022A7695454ac5Be77a4860e50c9683fC",
    "TokenStatesEUR": "0x6568D9e750fC44AF00f857885Dfb8281c00529c4",
    "TokenStatesGBP": "0x7e88D19A79b291cfE5696d496055f7e57F537A75",
    "TokenStatesJPY": "0x4dFACfB15514C21c991ff75Bc7Bf6Fb1F98361ed",
    "TokenStatesKRW": "0x93B6e9FbBd2c32a0DC3C2B943B7C3CBC2fE23730",
    "TokenStatesLINK": "0x577D4a7395c6A5f46d9981a5F83fa7294926aBB0",
    "TokenStatesUSD": "0x05a9CBe762B36632b3594DA4F082340E0e5343e8",
    "TradingRewards": "0x62922670313bf6b41C580143d1f6C173C5C20019",
    "VirtualSynthMastercopy": "0xf02ce48fD47D7FA1B7a45a0444805d320D035775",
    "WrapperFactory": "0x02f9bC46beD33acdB9cb002fe346734CeF8a9480",
}

synthetix_addresses = pl.DataFrame(
    list(synthetix_addresses.items()),
    orient="row",
    schema=["name", "address"],
).with_columns(label=pl.lit("synthetix")).select(
    pl.col.address.str.to_lowercase(),
    pl.col.label,
    pl.col.name,
).group_by('address').agg(pl.last('*'))

synthetix_addresses

In [ ]:
synthetix_addresses = synthetix_addresses.with_columns(
    synthetix_addresses['address'].evm.hex_to_binary()
)

In [ ]:
synthetix_addresses.write_parquet(    
    get_label_path('synthetix', min_block, max_block)
)

# instadapp

In [ ]:
instadapp_addresses = {
    'Index': '0x2971adfa57b20e5a416ae5a708a8655a9c74f723',
    'InstaList': '0x4c8a1BEb8a87765788946D6B19C6C6355194AbEb',
    'InstaAccount': '0xFE02a32Cbe0CB9ad9A945576A5bb53A3C123A3A3',
    'InstaConnectors': '0x97b0B3A8bDeFE8cB9563a3c610019Ad10DB8aD11',
    'InstaMemory': '0x8a5419CfC711B2343c17a6ABf4B2bAFaBb06957F',
    'Implementations': '0xCBA828153d3a85b30B5b912e1f2daCac5816aE9D',
    'InstaFlashAggregator': '0x619Ad2D02dBeE6ebA3CDbDA3F98430410e892882',
    'InstaFlashResolver': '0x10c7B513b7d37f40bdBCE77183b9112ec35CAec1',
    'Treasury': '0x28849D2b63fA8D361e5fc15cB8aBB13019884d09',
}

instadapp_addresses = pl.DataFrame(
    list(instadapp_addresses.items()),
    orient="row",
    schema=["name", "address"],
).with_columns(label=pl.lit("instadapp")).select(
    pl.col.address.str.to_lowercase(),
    pl.col.label,
    pl.col.name,
)

instadapp_addresses

In [ ]:
instadapp_addresses = instadapp_addresses.with_columns(
    instadapp_addresses['address'].evm.hex_to_binary()
)

In [ ]:
instadapp_addresses.write_parquet(    
    get_label_path('instadapp', min_block, max_block)
)

# VDice

In [ ]:
raw_vdice_addresses = [
    "0xdd98b423dc61a756e1070de151b1485425505954",
    "0x49fddeae0b521dab8d0c4b77e7161094f971320d",
    "0x7da90089a73edd14c75b0c827cb54f4248d47ecc",
    "0x1e2fbe6be9eb39fc894d38be976111f332172d83",
    "0xe642b6f79041c60d8447679b3a499f18d8b03b81",
    "0xe8a51be86ad96447d45ddeddc55013f25157688c",
    "0x4e646a576917a6a47d5b0896c3e207693870869d",
    "0x2ab9f67a27f606272189b307052694d3a2b158ba",
    "0xb95dd00b76c15b11ae82e875e9719029cd4d2110",
    "0x4b92a948ced9d457b4655abf62ed930a090f8566",
    "0xd3c9836230d43e85c643607cec509138f2955da8",
    "0x75a931567048edd4f349fa1a1cfbc4b4dca352c9",
    "0xc3b643bb85318b6a0e1b46c30365adacdf9469c2",
    "0x6dd38916b8c3780fd487980c54c90c175017664c",
    "0x108284702b23504de29d3293f2530255c6b3ccea",
    "0xd4160356a4e81f008b86f9867e71d1e1404c5292",
    "0x84bfd848717d60e01805b34d38a0d8b1f2bc4e85",
    "0xcf7cba701ded5c4d8e588e64f70b6ba1c17ea277",
    "0x50739060a2c32dc076e507ae1a893aab28ecfe68",
    "0x83cfd2d33bb9ed552a1e4b6af1bc3ea73bdee4f0",
    "0x60aa80fe41b43c180870a137c7a87334e5c3cb5d",
    "0xf4a0d461d5ba272469361f9319596548b5cdb126",
    "0x110c11de11cce962604c28bb05bbc399cbaa7bda",
    "0x55a4574e57bd2d7e01daa4a81d6e52ae5220c418",
    "0x7f0fd8fa402d56dca1f4baa9bf3880721c274d53",
    "0xaae10c1223df062e2df6982a80621977fa8b2d34",
    "0xee9147dd0307e4a663d0f756549d47d6f8bd356b",
    "0xe840e4007987cd4c84446fa270bf87b1c98ca086",
    "0x96843f4763b86fefd1f56aa7133e24acf2a88efd",
    "0x09d4b4014db27ddfbc73393af4f0a8f444e539ea",
    "0x78f6a1d4b94d2e6c4ce3c5cc858b9dfb6e98b50f",
    "0x4e87a723a3a75c41e34ad6e6995f724d9abdc745",
    "0x230f66692f5cb2cf0eb3cf50520b04b75a520f0a",
    "0x869db2680600fbf9913df90ef0fd057935c248b6",
    "0x7da247644c9f5bc0ea4e2c789ebf69569cdd3f1a",
    "0xc7d784274aaa0010f0c3dedd10eb66d49b7e4098",
    "0xf56a188f3f8d02685ab14f8a89be47df5e87af32",
    "0xccf13891822bf2de0cacd300d2ce6f27abce8abe",
    "0xa341d53e64198cba938465236d1b3d9a929e0732",
    "0xfdfd8a46146bcc7b12bcee778c4562937849181f",
    "0xb760a46e5f1cf1dd53a01aa33dec363042e9d71d",
    "0x89a4eb99c404d236a8f62dd3389a4bf96833bb45",
    "0xc76e765aa9d5cc462c17f8860fed3cdab37adfe6",
    "0xc8da125b61fe356239321309ca07465d28887778",
    "0xb507a4c2b07aad8e19f99d179d8f90b2b72b4f72",
    "0x193ff455560318fc749e2ffbb0e6472e5192e427",
    "0xfdc1d17c59320f35816de0ffad0a019c9718dd88",
    "0xeb927a56588d17d551e72ef0a2be460a4ddb7060",
    "0xdf7734f8dbd40b392128993d847e9ede790dd1c9",
]

vdice_addresses = pl.DataFrame(
    {
        'address': pl.Series('address', [bytes.fromhex(address[2:]) for address in raw_vdice_addresses]),
    },
).with_columns(label=pl.lit('vdice'))

vdice_addresses

In [ ]:
vdice_addresses.write_parquet(
    get_label_path('vdice', min_block, max_block)
)